In [ ]:
# %%
import re
from typing import Dict

# 输出的最高性价比勋章数量
TOP_N = 1000
# 输出的最高收益勋章数量
TOP_INCOME_N = 1000

# 过滤的勋章类别
FILTER_MEDALS = ['装饰', '赠礼', '奖品']

# 定义个人的发帖比例，计算更符合自己发帖习惯的收益（个人资料中的主题帖/回帖比例）
PERSONAL_POST_WEIGHT = {
    "reply": 1,
    "topic": 30,
}


# 属性名称映射（中文到英文）
ATTRIBUTE_CN_TO_EN = {
    "金币": "gold",
    "血液": "blood",
    "旅程": "journey",
    "咒术": "curse",
    "知识": "knowledge",
    "灵魂": "soul",
    "堕落": "corruption",
}

# 调整各个属性的权重， 金币和血液权重为1，其他属性根据个人偏好调整，不计算堕落属性，不想要堕落的可以设置为-1（-1000可直接挑选减堕落的勋章)
INCOME_WEIGHT_MAP = {
    "gold": 1,
    "blood": 1,
    "journey": 30,
    "curse": 5,
    "knowledge": 50,
    "soul": 1000,
    "corruption": 0,
}

# 后面的不用看

ATTRIBUTE_MAP = {
    "gold": {"color": "#FFBF00", "emoji": "💰"},
    "blood": {"color": "#ff0000", "emoji": "🩸"},
    "journey": {"color": "#008000", "emoji": "✈️"},
    "curse": {"color": "#a52a2a", "emoji": "🔮"},
    "knowledge": {"color": "#0000ff", "emoji": "📖"},
    "soul": {"color": "#add8e6", "emoji": "✡️"},
    "corruption": {"color": "#800080", "emoji": "😈"},
    "total": {"color": "#ffa500", "emoji": "🈴"},
}

MEDAL_LIBRARY = {
      "詹姆斯·维加": "12025",
      "詹姆斯‧维加": "12025",
      "詹姆斯·维加（James Vega）": "12025",
      "奧倫": "12027",
      "奧倫（Auron）": "12027",
      "奥伦": "12027",
      "奥伦（Auron）": "12027",
      "希德‧海温特": "12028",
      "希德‧海温特（Cid Highwind）": "12028",
      "吉姆‧雷诺": "12030",
      "吉姆‧雷诺（Jim Raynor）": "12030",
      "法卡斯": "12032",
      "法卡斯（Farkas）": "12032",
      "维吉尔": "12033",
      "维吉尔（Vergil）": "12033",
      "皮尔斯‧尼凡斯": "12034",
      "皮尔斯‧尼凡斯（Piers Nivans）": "12034",
      "文森特·瓦伦丁": "12035",
      "文森特‧瓦伦丁": "12035",
      "文森特·瓦伦丁（Vincent Valentine）": "12035",
      "巴尔弗雷亚": "12036",
      "巴尔弗雷亚（Balthier）": "12036",
      "但丁": "12037",
      "但丁（Dante）": "12037",
      "盖里": "12040",
      "盖里（Gary）": "12040",
      "杰夫‧莫罗": "12041",
      "杰夫‧莫罗（Jeff Moreau）": "12041",
      "威尔卡斯": "12042",
      "威尔卡斯（Vilkas）": "12042",
      "克里斯·雷德菲尔德": "12043",
      "克里斯‧雷德菲尔德": "12043",
      "克里斯·雷德菲尔德（Chris Redfield）": "12043",
      "裸体克里斯": "12044",
      "一丝不挂克里斯": "12044",
      "一丝不挂克里斯（Naked Chris）": "12044",
      "凯登‧阿兰科": "12045",
      "凯登‧阿兰科（Kaidan Alenko）": "12045",
      "肥皂": "12046",
      "肥皂（John“Soap”MacTavish）": "12046",
      "奥利弗‧奎恩": "12047",
      "奥利弗‧奎恩（Oliver Queen）": "12047",
      "收到情书": "12048",
      "送情书": "12048",
      "丢肥皂": "12049",
      "捡到了肥皂": "12049",
      "千杯不醉": "12050",
      "灵光补脑剂": "12051",
      "贞洁内裤": "12052",
      "炼金之心": "12053",
      "黑暗交易": "12054",
      "水泡术": "12055",
      "召唤古代战士": "12056",
      "祈祷术": "12057",
      "嗜血斩首斧": "12058",
      "符文披风": "12059",
      "净化手杖": "12060",
      "圣光法杖": "12060",
      "十字叶章": "12061",
      "刺杀者匕首": "12062",
      "药剂背袋": "12063",
      "知识大典": "12064",
      "怪笑的面具": "12065",
      "诡笑的面具": "12065",
      "奸笑的面具": "12065",
      "恐怖的面具": "12065",
      "微笑的面具": "12065",
      "邪恶的面具": "12065",
      "被踩烂的小草": "12066",
      "抖不停小草": "12066",
      "抖来抖去小草": "12066",
      "微抖小草": "12066",
      "种植小草": "12066",
      "聚魔花盆": "12067",
      "金钱马车": "12068",
      "牧羊人": "12069",
      "森林羊男": "12070",
      "被祝福の新旅程": "12071",
      "被祝福的新旅程": "12071",
      "另一个身份": "12072",
      "神之匠工": "12073",
      "Chris Redfield in Uroboros": "12074",
      "站员薪俸": "12075",
      "保卫领土": "12075",
      "站员: 保卫领土": "12075",
      "实习版主薪俸": "12076",
      "神的重量": "12076",
      "见习版主: 神的重量": "12076",
      "版主薪俸": "12077",
      "一国之主": "12077",
      "版主: 一国之主": "12077",
      "梅格": "12085",
      "梅格（Meg）": "12085",
      "里昂·S·甘乃迪": "12086",
      "里昂‧S‧甘乃迪": "12086",
      "里昂·S·甘乃迪（Leon·S·Kennedy）": "12086",
      "重磅手环": "12087",
      "亚力斯塔尔": "12088",
      "亚力斯塔尔（Alistair）": "12088",
      "罗伯‧史塔克": "12090",
      "罗伯‧史塔克（Robb Stark）": "12090",
      "亚当‧简森": "12091",
      "亚当‧简森（Adam Jensen）": "12091",
      "猫化弩哥": "12092",
      "猫化弩哥（Daryl Dixon）": "12092",
      "高端噬魂者": "12093",
      "魂之佳肴": "12093",
      "狼狈噬魂者": "12093",
      "吞食魂魄": "12093",
      "超级名贵无用宝剑": "12094",
      "念念往日士官盔": "12095",
      "老旧之椅": "12096",
      "灵魂之椅": "12096",
      "流失之椅": "12096",
      "漂洋小船": "12097",
      "菠菜人的尸体": "12098",
      "发现菠菜人": "12098",
      "元气菠菜人": "12098",
      "种植菠菜": "12098",
      "洞窟魔蛋": "12099",
      "吸血魔蝠": "12099",
      "夜灯": "12100",
      "诺曼底号": "12101",
      "堕落之舞": "12102",
      "黄色就是俏皮": "12103",
      "骑兽之子": "12104",
      "质量效应三部曲": "12105",
      "五花八门版块": "12106",
      "辐射：新维加斯": "12107",
      "TRPG版块": "12108",
      "TRPG版塊": "12108",
      "上古卷轴V：天际": "12109",
      "一只可爱的小猫": "12111",
      "猫眼": "12112",
      "謎の男": "12113",
      "站长の守护": "12113",
      "布莱恩·欧康纳": "12121",
      "布莱恩‧欧康纳": "12121",
      "布莱恩·欧康纳（Brian O`Conner）": "12121",
      "迪恩‧温彻斯特": "12122",
      "迪恩‧温彻斯特（Dean Winchester）": "12122",
      "山姆‧温彻斯特": "12123",
      "山姆‧温彻斯特（Sam Winchester）": "12123",
      "魔术师奥斯卡": "12124",
      "魔术师奥斯卡（Magician Oscar）": "12124",
      "卡斯迪奥": "12125",
      "卡斯迪奥（Castiel）": "12125",
      "虎克船长": "12126",
      "虎克船长（Captain Hook）": "12126",
      "卢西亚诺‧科斯塔": "12127",
      "卢西亚诺‧科斯塔（Luciano Costa）": "12127",
      "史蒂夫‧金克斯": "12128",
      "史蒂夫‧金克斯（Steve Jinks）": "12128",
      "遗忘之水": "12129",
      "咆哮诅咒": "12130",
      "充满魔力的种子": "12131",
      "闪耀种子": "12131",
      "夜魔果实": "12131",
      "夜魔护符": "12131",
      "夜魔藤": "12131",
      "柏木炭": "12132",
      "柏树枝": "12132",
      "木柴堆": "12132",
      "木精灵短弓": "12132",
      "暗红矿土": "12133",
      "炼化龙血石": "12133",
      "龙血水晶": "12133",
      "龙血指环": "12133",
      "阿呆": "12134",
      "鼻涕精灵": "12134",
      "不灭的蓝宝石": "12134",
      "奇怪的刮刮卡": "12134",
      "神秘的邀请函": "12134",
      "天然有鸡": "12134",
      "指路幽灵": "12134",
      "吃饱金币的Doge": "12135",
      "吃掉金币的Doge": "12135",
      "迷のDoge": "12135",
      "喜欢金币的Doge": "12135",
      "龙腾世纪：审判": "12137",
      "堕落飨宴": "12139",
      "TRPG纪念章": "12140",
      "德拉克神圣灵魂瓶": "12141",
      "德拉克圣瓶": "12141",
      "戴蒙‧萨尔瓦托": "12143",
      "戴蒙‧萨尔瓦托（Damon Salvatore）": "12143",
      "库伦 (起源)": "12144",
      "库伦(起源)": "12144",
      "库伦(起源)（Cullen (Origins)）": "12144",
      "安德森‧戴维斯": "12145",
      "安德森‧戴维斯（Anderson Davis）": "12145",
      "圣英灵秘银甲": "12146",
      "圣英灵凯尔": "12146",
      "禽兽扒手": "12147",
      "奥兹大陆": "12148",
      "海上明月": "12149",
      "葛莱分多": "12150",
      "赫夫帕夫": "12150",
      "霍格沃茨五日游": "12150",
      "雷文克劳": "12150",
      "麻瓜": "12150",
      "史莱哲林": "12150",
      "铁杆影迷": "12151",
      "泡沫浮髅(Squirt)": "12152",
      "铁牛": "12153",
      "铁牛（Iron Bull）": "12153",
      "康纳‧沃什": "12154",
      "康纳‧沃什（Connor Walsh）": "12154",
      "尼克·贝特曼": "12155",
      "尼克‧贝特曼": "12155",
      "尼克·贝特曼（Nick Bateman）": "12155",
      "离去的尤利西斯": "12156",
      "尤利西斯": "12156",
      "尤利西斯（Ulises）": "12156",
      "布衣": "12157",
      "布衣+1 良好的": "12157",
      "布衣+10 史诗的": "12157",
      "布衣+11 史诗的": "12157",
      "布衣+12 传奇的": "12157",
      "布衣+13 歷战的": "12157",
      "布衣+13 歴战的": "12157",
      "布衣+2 上等的": "12157",
      "布衣+3 精致的": "12157",
      "布衣+4 精致的": "12157",
      "布衣+5 精致的": "12157",
      "布衣+6 精致的": "12157",
      "布衣+7 无暇的": "12157",
      "布衣+8 无暇的": "12157",
      "布衣+9 无暇的": "12157",
      "锻造卷轴": "12163",
      "巴特·贝克": "12245",
      "巴特‧贝克": "12245",
      "巴特·贝克（Bart Baker）": "12245",
      "被释放的灵魂": "12246",
      "诡异的灵魂石": "12246",
      "奇怪的紫水晶": "12246",
      "预知水晶球": "12248",
      "艾尔尤因": "12348",
      "名剑艾尔尤因": "12348",
      "灵剑艾尔尤因": "12348",
      "圣剑艾尔尤因": "12348",
      "杰森·斯坦森": "12349",
      "杰森‧斯坦森": "12349",
      "杰森·斯坦森（Jason Statham）": "12349",
      "神圣十字章": "12350",
      "哈尔·乔丹": "12351",
      "哈尔‧乔丹": "12351",
      "哈尔·乔丹（Hal Jordan）": "12351",
      "新月护符": "12352",
      "发芽的种子": "12353",
      "可爱的三叶草": "12353",
      "破土而出的嫩芽": "12353",
      "神奇四叶草": "12353",
      "艾德尔": "12376",
      "艾德尔（Edér Teylecg）": "12376",
      "盖拉斯‧瓦卡瑞安": "12377",
      "盖拉斯‧瓦卡瑞安（Garrus Vakarian）": "12377",
      "史莱姆牧场": "12378",
      "史莱姆养殖证书/史莱姆牧场": "12378",
      "史莱姆养殖证书": "12378",
      "丛林的鸟飞走了": "12379",
      "丛林露出来的鸟": "12379",
      "非常茂盛的丛林": "12379",
      "这是一片丛林": "12379",
      "种植菊花": "12380",
      "菊花发芽了": "12380",
      "菊花等着开花": "12380",
      "啊！菊花开了！": "12380",
      "把菊花卖了": "12380",
      "秘密森林": "12547",
      "戴尔·芭芭拉": "12645",
      "戴尔‧芭芭拉": "12645",
      '戴尔·芭芭拉（Dale "Barbie" Barbara）': "12645",
      "婴儿泪之瓶": "12646",
      "雪王的心脏": "12647",
      "萨赫的蛋糕": "12653",
      "勇者与龙之书": "12655",
      "落雪勇者与龙的传说-封面": "12655",
      "落雪勇者与龙的传说-第一页": "12655",
      "落雪勇者与龙的传说-第二页": "12655",
      "落雪勇者与龙的传说-第三页": "12655",
      "落雪勇者与龙的传说-第四页": "12655",
      "落雪勇者与龙的传说-第五页": "12655",
      "落雪勇者与龙的传说-第六页": "12655",
      "落雪勇者与龙的传说-第七页": "12655",
      "落雪勇者与龙的传说-第八页": "12655",
      "落雪勇者与龙的传说-书尾": "12655",
      "破损的旧书": "12655",
      "雪勇者与龙的传说-书尾": "12655",
      "Frank （LBF）": "12658",
      "Frank (LBF)": "12658",
      "BIG BOSS": "12659",
      "詹米·多南": "12670",
      "詹米·多南（Jamie Dornan）": "12670",
      "库伦（审判）": "12700",
      "库伦(审判)": "12700",
      "库伦 (审判)": "12700",
      "神秘商店贵宾卡": "12701",
      "史莱姆蛋": "12931",
      "心之水晶": "13183",
      "阿尔萨斯·米奈希尔": "13286",
      "阿尔萨斯‧米奈希尔": "13286",
      "阿尔萨斯·米奈希尔（Arthas Menethil）": "13286",
      "低阶战斗牧师": "13287",
      "高阶神圣祭司": "13287",
      "渐逝忠诚之魂": "13287",
      "维克多·天火": "13287",
      "维克多‧天火": "13287",
      "优雅的圣职者": "13287",
      "云游中的助祭": "13287",
      "天涯.此时": "13562",
      "天涯‧此时": "13562",
      "快破掉的蛋": "13563",
      "偷情的月兔男": "13563",
      "小月兔": "13563",
      "月亮的蛋": "13563",
      "月兔男": "13563",
      "骑士遗盔": "13572",
      "背弃之证": "14029",
      "传承之证": "14029",
      "新年小猴": "15171",
      "变形软泥": "15284",
      "冥界之眼": "15284",
      "魔眼护符": "15284",
      "冥界魔眼": "15284",
      "克里斯·埃文斯": "15285",
      "克里斯·埃文斯（Chris Evans）": "15285",
      "安德鲁·库珀": "15286",
      "安德鲁·库珀（Andrew Cooper）": "15286",
      "罗宾·西克": "15580",
      "罗宾·西克（Robin Thicke）": "15580",
      "章鱼小丸子": "15581",
      "华灯初上": "15660",
      "岛田半藏": "17946",
      "岛田半藏（Shimada Hanzo）": "17946",
      "归途": "68056",
      "浪潮之歌": "68056",
      "奇怪的鱼": "68056",
      "奇异的光线": "68056",
      "驱逐": "68056",
      "人鱼男子": "68056",
      "人鱼之泪": "68056",
      "泄密": "68056",
      "许愿": "68056",
      "罪恶": "68056",
      "安静的海边": "68056",
      "生化危机：复仇": "68057",
      "亚瑟·摩根": "68058",
      "亚瑟‧摩根": "68058",
      "亚瑟·摩根（Arthur Morgan）": "68058",
      "亚瑟·库瑞": "68059",
      "亚瑟‧库瑞": "68059",
      "亚瑟·库瑞（海王）": "68059",
      "亚瑟·库瑞（Arthur Curry）": "68059",
      "变骚喷雾": "68060",
      "网中的皮卡丘": "68061",
      "红龙蛋": "68272",
      "驯化红龙幼崽": "68272",
      "红龙幼崽": "68272",
      "黑龙蛋": "68273",
      "驯化黑龙幼崽": "68273",
      "黑龙幼崽": "68273",
      "腐化龙蛋": "68274",
      "驯化腐化龙幼崽": "68274",
      "腐化龙幼崽": "68274",
      "龙之魂火": "68448",
      "粗糙的骨锤": "68449",
      "红龙精华": "68449",
      "华丽的骨锤": "68449",
      "烤龙肉": "68449",
      "龙骨残余": "68449",
      "红龙宝珠碎片": "68451",
      "珊瑚色捕梦网": "68451",
      "红龙秘宝": "68451",
      "倒吊人(The Hanged Man , XII)": "68452",
      "战车(The Chariot , VII)": "68453",
      "泰凯斯·芬得利": "68497",
      "泰凯斯·芬得利（Tychus Findlay）": "68497",
      "【夏日限定】夏日的泰凯斯": "68498",
      "夏日的泰凯斯": "68498",
      "野兽之子": "68499",
      "康纳": "68500",
      "康纳/Connor": "68500",
      "康纳（Conner）": "68500",
      "德拉克魔瓶": "68501",
      "迷之瓶": "68501",
      "石肤术": "68502",
      "恋人(The Lovers，VI)": "68594",
      "恋人": "68594",
      "魔术师（The Magician，I）": "68733",
      "恋恋小烹锅": "69034",
      "黑曜石赤螯蝎": "69215",
      "漆黑的蝎卵": "69215",
      "蛮族战士": "69216",
      "山猫图腾": "69217",
      "猎鹰图腾": "69218",
      "眼镜蛇图腾": "69219",
      "布莱克沃尔": "69251",
      "黑墙": "69251",
      "守望者雷涅尔": "69251",
      "汤姆雷涅尔": "69251",
      "黑墙（Blackwall）": "69251",
      "汤姆 雷涅尔": "69251",
      "守望者徽章": "69252",
      "高阶守望者徽章": "69252",
      "守望者指挥官徽章": "69252",
      "满是血迹的徽章": "69252",
      "没有梦想的咸鱼": "69254",
      "内森·德雷克": "69255",
      "内森·德雷克（Nathan Drake）": "69255",
      "丹尼尔·纽曼": "69256",
      "丹尼爾·紐曼": "69256",
      "丹尼尔·纽曼（Daniel Newman）": "69256",
      "石鬼面": "69257",
      "秋水长天": "69302",
      "落霞孤鹜": "69302",
      "贝优妮塔": "69503",
      "GM村金蛋": "69504",
      "生金蛋的鹅": "69504",
      "兴奋的鹅": "69504",
      "休息的鹅": "69504",
      "【年中限定】GM村金蛋": "69504",
      "模拟人生4": "69505",
      "模擬人生4": "69505",
      "晃晃悠悠小矿车": "69660",
      "空矿车-日": "69660",
      "日-黑曜石": "69660",
      "日-琥珀": "69660",
      "日-精金": "69660",
      "日-蓝宝石 ": "69660",
      "日-煤": "69660",
      "日-山铜": "69660",
      "日-石头": "69660",
      "日-铁": "69660",
      "日-亚历山大石": "69660",
      "夜-电气石": "69660",
      "夜-金": "69660",
      "夜-铁": "69660",
      "夜-亚历山大石": "69660",
      "夜-月光石": "69660",
      "英普瑞斯": "69799",
      "杰西·麦克雷": "69800",
      "卡德加": "69801",
      "卡德加（Khadgar）": "69801",
      "麦迪文": "69802",
      "麦迪文（Medivh）": "69802",
      "达拉然": "69803",
      "德拉克魂匣": "69804",
      "圣甲虫秘典": "69805",
      "安杜因·乌瑞恩": "70367",
      "安杜因·乌瑞恩(Anduin Wrynn)": "70367",
      "羅素·托維": "70368",
      "冒险专用绳索": "70370",
      "红石": "70372",
      "可疑的红石": "70372",
      "伪造的红石": "70372",
      "结晶火鹰幼崽": "70373",
      "结晶卵": "70373",
      "暮光独角兽幼崽": "70375",
      "暮色卵": "70375",
      "发光的暮色卵": "70375",
      "狂暴的书籍": "70715",
      "莱托文本残页": "70715",
      "躁动的书籍": "70715",
      "赫尔墨斯·看守者之杖": "70996",
      "闪耀的赫尔墨斯之杖": "70996",
      "失控的赫尔墨斯之杖": "70996",
      "阿列克西欧斯(Alexios)": "70997",
      "阿列克西欧斯（Alexios）": "70997",
      "冰冷的遗骸": "70998",
      "蓝礼·拜拉席恩": "70998",
      "蓝礼·重生鹿三": "70998",
      "亲和的列王": "70998",
      "英俊的王弟": "70998",
      "青鸾": "71001",
      "青鸾蛋": "71001",
      "电磁巨鳄": "71002",
      "电磁卵": "71002",
      "风暴磁场之鳄": "71002",
      "凶猛的鳄鱼": "71002",
      "一只普通的鳄鱼": "71002",
      "荒野大镖客:救赎 II": "71004",
      "荒野大镖客：救赎 II": "71004",
      "鬼火竹筒": "71005",
      "神秘的竹筒": "71005",
      "幽灵竹筒": "71005",
      "阿帕茶": "71006",
      "强者の茶": "71006",
      "神秘的红茶": "71006",
      "种植土豆": "71007",
      "挖了个坑": "71007",
      "土豆发芽了": "71007",
      "土豆开花了": "71007",
      "收获土豆": "71007",
      "静谧的洞窟": "71007",
      "巴啦啦小魔仙棒": "71008",
      "充能的魔仙棒": "71008",
      "闪耀的魔仙棒": "71008",
      "失效的魔仙棒": "71008",
      "白猪猪储蓄罐": "71514",
      "爆炸的储蓄罐": "71514",
      "白猪猪储蓄罐㊖": "71514",
      "粉猪猪储蓄罐": "71515",
      "碎裂的储蓄罐": "71515",
      "粉猪猪储蓄罐㊖": "71515",
      "撑破的储蓄罐": "71516",
      "金猪猪储蓄罐": "71516",
      "金猪猪储蓄罐㊖": "71516",
      "德拉克的遗物": "71578",
      "虹之女神像": "71578",
      "破旧的石像": "71578",
      "遗物搜寻": "71578",
      "展开地图": "71578",
      "不败之花": "71579",
      "雾都血医": "71885",
      "春之歌": "71886",
      "冬之歌": "71886",
      "秋之歌": "71886",
      "四季之歌": "71886",
      "夏之歌": "71886",
      "喷涌的粪桶": "71887",
      "用过的粪桶": "71887",
      "箭术卷轴": "71889",
      "心心念念小雪人": "71890",
      "【圣诞限定】心心念念小雪人": "71890",
      "十字军护盾": "72051",
      "龙血之斧": "72052",
      "魔法石碑": "72053",
      "远古石碑": "72054",
      "莱因哈特·威尔海姆": "72137",
      "吃了一半的面包": "72138",
      "吃完的面包": "72138",
      "冒险用面包": "72138",
      "香喷喷的面包": "72138",
      "海螺号角": "72139",
      "迷之天鹅": "72140",
      "天鹅鬼琴": "72140",
      "初衷的力量": "72200",
      "临危受命": "72200",
      "尼克斯·乌尔里克": "72200",
      "曙光下的灰烬": "72200",
      "铁汉柔情": "72200",
      "王者之剑": "72200",
      "乔纳森·里德": "72201",
      "蔷薇骑士之刃": "72202",
      "狩猎用小刀": "72203",
      "阿拉喵？神灯": "72204",
      "沙漠神灯": "72204",
      "老旧的怀表": "72225",
      "发光的怀表": "72225",
      "无尽的怀表": "72225",
      "冒险用指南针": "72226",
      "珊瑚泡泡鱼": "72227",
      "珊瑚色礁石蛋": "72227",
      "草原孤狼": "72228",
      "月影蛋": "72228",
      "月影狼": "72228",
      "藤田優馬": "72920",
      "莫瑞甘": "72921",
      "荒野女巫": "72921",
      "被狩猎的女巫": "72921",
      "轻蔑的女术士": "72921",
      "秘法顾问": "72921",
      "谜踪女巫": "72921",
      "钢铁勇士弯刀": "72922",
      "海盗弯钩": "72923",
      "生锈的海盗刀枪": "72924",
      "洗刷的了海盗刀枪": "72924",
      "意气风发的海盗刀枪": "72924",
      "草原之象": "72925",
      "马戏团灰蛋": "72925",
      "马戏团狂欢象": "72925",
      "迈克尔迈尔斯": "73626",
      "Doc": "73627",
      "火柴 - Gamemale": "73628",
      "火柴 - 果体美男子": "73628",
      "火柴 - 美食大全": "73628",
      "暖心小火柴": "73628",
      "暴风雨中的漂流瓶": "73629",
      "繁忙船道上的漂流瓶": "73629",
      "孤岛旁的漂流瓶": "73629",
      "极地穿越的漂流瓶": "73629",
      "秘密森林的漂流瓶": "73629",
      "密林仙境中的漂流瓶": "73629",
      "秋日麦田的漂流瓶": "73629",
      "沙滩上的漂流瓶": "73629",
      "神秘的漂流瓶": "73629",
      "神秘古城的漂流瓶": "73629",
      "驶出水渠的漂流瓶": "73629",
      "信仰之心": "73629",
      "森林鹿": "73630",
      "一只小鹿": "73630",
      "郁苍卵": "73630",
      "熔岩蛋": "73631",
      "熔岩鹰": "73631",
      "杰克·莫里森": "74196",
      "杰克·莫里森/士兵 76": "74196",
      "士兵 76": "74196",
      "索林·橡木盾": "74198",
      "索林·此生挚爱": "74198",
      "索林·孤山之王": "74198",
      "索林·临终一役": "74198",
      "陷阱杀手": "74199",
      "日荒戒指": "74201",
      "月陨戒指": "74202",
      "灵鹫蛋": "74203",
      "圣光灵鹫": "74203",
      "心领神会": "74203",
      "御风之灵": "74203",
      "滴血认亲": "74204",
      "猩红魔鹫": "74204",
      "血鹫蛋": "74204",
      "血色威仪": "74204",
      "月上柳梢": "74206",
      "吉姆·霍普": "75042",
      "接近阴谋": "75042",
      "警长": "75042",
      "老父亲": "75042",
      "谜": "75042",
      "十一": "75042",
      "沃特·沙利文": "75043",
      "塞巴斯蒂安·斯坦": "75044",
      "魯杰羅·弗雷迪": "75045",
      "星芒戒指": "75046",
      "冒险用绷带": "75047",
      "日渐染血的绷带": "75047",
      "逐渐用完的绷带": "75047",
      "绷带内裤？": "75047",
      "宝箱内的球": "75048",
      "空荡荡的精灵球": "75048",
      "迷拟Q": "75048",
      "皮卡丘": "75048",
      "和你一起飞行的皮卡丘": "75048",
      "软泥怪": "75049",
      "软泥怪蛋": "75049",
      "寶可夢 Pokémon": "75057",
      "血石": "75131",
      "恶魔阿蕾莎": "75750",
      "莎伦": "75750",
      "消失的阿蕾莎": "75750",
      "疾风剑豪": "75751",
      "亚索": "75751",
      "【新手友好】昆進": "75752",
      "昆進": "75752",
      "萨菲罗斯": "75753",
      "SCP-s-1889": "75754",
      "SCP-s-1889-第二页": "75754",
      "SCP-s-1889-第六页": "75754",
      "SCP-s-1889-第七页": "75754",
      "SCP-s-1889-第三页": "75754",
      "SCP-s-1889-第四页": "75754",
      "SCP-s-1889-第五页": "75754",
      "SCP-s-1889-第一页": "75754",
      "GHOST": "75755",
      "最终幻想XIV": "75756",
      "掌中雪球瓶": "75757",
      "猫猫幽灵": "76082",
      "万圣南瓜": "76083",
      "丹·安博尔": "76095",
      "汤姆·赫兰德": "76096",
      "超人": "76097",
      "阿尔伯特·威斯克": "76556",
      "鬼王酒吞童子": "76557",
      "Scott Ryder": "76558",
      "GM论坛初心者毕业证书": "76559",
      "GM论坛初心者勋章": "76559",
      "GM论坛进阶勋章": "76559",
      "GM论坛荣誉勋章": "76559",
      "GM论坛勋章": "76559",
      "GM論壇初心者勛章": "76559",
      "GM論壇勛章": "76559",
      "GM論壇進階勛章": "76559",
      "GM論壇榮譽勛章": "76559",
      "GM論壇初心者畢業證書": "76559",
      "社畜专用闹钟": "76560",
      "暗纹鲨": "76561",
      "螺旋纹卵": "76561",
      "邦尼尼": "76562",
      "万圣彩蛋": "76562",
      "风雪之家": "76563",
      "赛博朋克2077": "76564",
      "气球带来的礼物": "76565",
      "闪亮拐杖糖": "76565",
      "神秘的礼物": "76565",
      "圣诞袜": "76565",
      "汉克/Hank": "77158",
      "Hank - 推倒": "77158",
      "Hank - 抉择": "77158",
      "Hank - 重逢": "77158",
      "三角头": "77159",
      "守护者三角头": "77159",
      "刽子手三角头": "77159",
      "寂静岭三角头": "77159",
      "幻象": "77160",
      "琉璃玉坠": "77161",
      "蠢蠢欲动的宝箱": "77162",
      "冒险用宝箱": "77162",
      "奇怪的宝箱": "77162",
      "不曾寄出的信件": "77163",
      "羽毛笔": "77163",
      "星光彩虹小粉驼": "77164",
      "幽光彩蛋": "77164",
      "沙漠鸵鸟": "77165",
      "沙漠羽蛋": "77165",
      "恶魔城": "77166",
      "老旧的书籍": "78151",
      "伊波恩之书": "78151",
      "凝视夜空": "78151",
      "金刚狼": "78152",
      "罗根": "78152",
      "暮狼归来": "78152",
      "逆转未来": "78152",
      "克苏鲁": "78153",
      "诡秘的雕像": "78153",
      "翻卷的海潮": "78153",
      "遥远的呼唤": "78153",
      "无尽的深渊": "78153",
      "旧日支配者—克苏鲁": "78153",
      "士官长": "78154",
      "约翰-117": "78154",
      "繁星花": "78711",
      "枯黄的种苗": "78711",
      "微光苗": "78711",
      "托尼·史塔克": "79451",
      "钢铁侠": "79451",
      "加勒特·霍克": "79453",
      "艾吉奥": "79454",
      "Chris Mazdzer": "79456",
      "武士之魂": "79457",
      "削铁如泥的武士刀": "79457",
      "乘风破浪的武士刀": "79457",
      "超级无敌名贵金卡": "79458",
      "林中松鼠": "79459",
      "林中之蛋": "79459",
      "云上之光": "79525",
      "湖面云光": "79525",
      "魔法灵药": "80303",
      "波板糖": "80304",
      "幼儿波板糖": "80304",
      "彩虹許願星": "80304",
      "索尔·奥丁森": "81117",
      "绯红女巫": "81119",
      "泰比里厄斯": "81120",
      "大古": "81121",
      "GM村蛋糕": "81123",
      "缘定仙桥": "81124",
      "小小行星": "82408",
      "炙热的格拉迪欧拉斯": "82409",
      "格拉迪欧拉斯": "82410",
      "王者之盾": "82410",
      "卡洛斯·奥利维拉": "82411",
      "巴基 (猎鹰与冬兵)": "82412",
      "巴基": "82412",
      "冬兵": "82412",
      "碎旗者": "82412",
      "英雄联盟": "82413",
      "Joker": "83981",
      "V (DMC5)": "83983",
      "Dante": "83984",
      "Vergil": "83985",
      "力量腕带": "83986",
      "物理学圣剑": "83987",
      "男巫之歌": "83988",
      "很久很久以前": "83988",
      "xx之歌": "83988",
      "××之歌": "83988",
      "闪耀圣诞球": "84015",
      "压箱底的泡面": "84534",
      "岛田源氏": "85513",
      "阿拉贡": "85514",
      "阿拉贡·护戒使者": "85514",
      "阿拉贡·疾风之足": "85514",
      "阿拉贡·亡魂之约": "85514",
      "阿拉贡·星辰之鹰": "85514",
      "阿拉贡·王者归来": "85514",
      "瑟兰迪尔": "85516",
      "异形": "85517",
      "“Space jockey”": "85517",
      "“普罗米修斯”": "85517",
      "威克多尔·克鲁姆": "85519",
      "赫敏·格兰杰": "85520",
      "男用贞操带": "85521",
      "One Ring": "85522",
      "巴拉多要塞": "85522",
      "厄运山谷": "85522",
      "烽火": "85522",
      "黑暗消融": "85522",
      "黑门": "85522",
      "卡兰德拉斯": "85522",
      "米纳斯提力斯": "85522",
      "千军万马": "85522",
      "瑞文戴尔": "85522",
      "夏尔": "85522",
      "成年独角兽": "85523",
      "五彩斑斓的蛋": "85523",
      "幼年独角兽": "85523",
      "大恶魔": "85524",
      "小恶魔": "85524",
      "血红色的蛋": "85524",
      "男色风暴": "85525",
      "男色时代": "85525",
      "男色诱惑": "85525",
      "孔明灯": "87022",
      "生命树叶": "88872",
      "菀叶狸猫": "88872",
      "不起眼的空瓶": "88874",
      "大脚板": "88875",
      "小天狼星·布莱克": "88875",
      "至情至性": "88875",
      "甘道夫": "88876",
      "甘道夫·护戒使者": "88876",
      "甘道夫·涅槃重生": "88876",
      "甘道夫·曙光军师": "88876",
      "甘道夫·先圣仁心": "88876",
      "灰袍甘道夫": "88876",
      "莱戈拉斯": "88877",
      "神灯": "88878",
      "黑豹": "88879",
      "时间变异管理局": "88880",
      "时间尽头的虚空": "88880",
      "遗留之人的城堡": "88880",
      "缘起星空": "88880",
      "普隆普特·阿金塔姆": "91413",
      "诺克提斯·路西斯·伽拉姆": "91414",
      "阿不思·邓布利多": "91415",
      "魔法不朽·传奇不熄": "91415",
      "孤注一掷": "91416",
      "混血王子": "91416",
      "青梅竹马": "91416",
      "西弗勒斯·斯内普": "91416",
      "至死不渝": "91416",
      "奥利维尼斯辉石头罩": "91418",
      "卡勒罗斯辉石头罩": "91418",
      "双贤辉石头罩": "91418",
      "贤者头盔": "91418",
      "恩惠护符": "91419",
      "黄金护符": "91419",
      "黄金树的恩惠": "91419",
      "秘密空瓶": "91420",
      "诺克史黛拉之月": "91420",
      "星星泪滴": "91420",
      "霍格沃兹魔法学校": "91424",
      "霍格沃兹特快列车": "91424",
      "梦中的列车": "91424",
      "霍格沃茨特快列车": "91424",
      "海边的蛋": "91425",
      "夏日柯基": "91425",
      "海边的邻居": "91427",
      "远亲不如近邻？": "91427",
      "1984": "91428",
      "疯人院": "91428",
      "畸形秀": "91428",
      "洛亚洛克": "91428",
      "旅馆": "91428",
      "美恐：启程": "91428",
      "美恐：新的开始": "91428",
      "女巫集会": "91428",
      "启示录": "91428",
      "邪教": "91428",
      "玄生万物": "91429",
      "莱托·厄崔迪": "91431",
      "海的记忆": "92246",
      "独眼章鱼": "92248",
      "海与天之蛋": "92248",
      "豹王": "94945",
      "为承父志": "94945",
      "化敌为友": "94945",
      "双重身份": "94945",
      "再会恩师": "94945",
      "不灭狂雷": "94946",
      "不灭狂雷-沃利贝尔": "94946",
      "霹天雳地": "94946",
      "苏醒巨熊": "94946",
      "初入黑道-桐生一马": "94947",
      "传说的黑道-桐生一马": "94947",
      "堂岛之龙-桐生一马": "94947",
      "桐生一马": "94947",
      "大黄蜂": "94948",
      "大黄蜂（ChevroletCamaro）": "94948",
      "擎天柱（Peterbilt389）": "94949",
      "博伊卡": "94950",
      "重回格斗场": "94950",
      "回旋踢才是男人的浪漫": "94950",
      "The Brave Boyka": "94950",
      "自由": "94950",
      "飞天小糖果": "94952",
      "闪光糖果盒": "94952",
      "茉香啤酒": "94953",
      "雷霆晶球": "94956",
      "思绪骤聚": "94957",
      "真理世界": "94957",
      "超级幸运无敌辉石": "95016",
      "卡利亚权杖": "95016",
      "这是怎么辉石呢": "95016",
      "香喷喷的烤鸡": "95018",
      "小菜鸟": "95018",
      "小凤凰": "95018",
      "新手蛋": "95018",
      "深渊遗物": "95019",
      "星辰龙": "95019",
      "星尘龙": "95019",
      "街头霸王": "95020",
      "奇思妙想": "97633",
      "旅行骰子！": "99612",
      "埃及": "101343",
      "马克": "101343",
      "马克·史贝特": "101343",
      "信任": "101343",
      "月光化身": "101343",
      "月光骑士": "101343",
      "史蒂文·格兰特": "101345",
      "史蒂文": "101345",
      "幻觉": "101345",
      "光化身": "101345",
      "骑士先生": "101345",
      "果体76": "101775",
      "竹村五郎": "103957",
      "内在": "103957",
      "忠诚": "103957",
      "前身": "103957",
      "暗影烈焰": "103958",
      "苍穹禁城": "103958",
      "光之战士": "103958",
      "红莲狂潮": "103958",
      "晓月终焉": "103958",
      "重生之境": "103958",
      "Drover": "103959",
      "流行巨星GM": "103960",
      "乔治·迈克尔": "103960",
      "威猛主唱GM": "103960",
      "性感男神GM": "103960",
      "蛰伏中的GM": "103960",
      "吃饱的小阿尔": "103961",
      "饥饿的小阿尔": "103961",
      "小阿尔的蛋": "103961",
      "幸福的小阿尔": "103961",
      "红心玉": "104061",
      "『还乡歌』": "108197",
      "『落樱缤纷』": "108197",
      "『樱花树灵』": "108197",
      "『日心说』": "108200",
      "『星象监测』": "108200",
      "『任天堂Switch』红蓝√": "108201",
      "『崭新的红蓝游戏机』": "108201",
      "『随时随地开启！』": "108201",
      "『任天堂Switch』灰黑√": "108202",
      "『崭新的灰黑游戏机』": "108202",
      "『私有海域』": "108899",
      "『钜鲸』": "108901",
      "『召唤好运的角笛』": "108903",
      "『圣洁化身』": "108905",
      "『矩阵谜钥Ⓖ』": "108908",
      "『确认购置新居?』": "108911",
      "『新居手册Ⓖ』": "108911",
      "『交钥匙了!』": "108911",
      "『户口本: Lv2~6』": "108918",
      "『居住证: Lv2~6』": "108918",
      "『正在入住GM村』": "108918",
      "『户口本: Lv7+』": "108920",
      "『居住证: Lv7+』": "108920",
      "『住在GM村』": "108920",
      "牌中小丑": "109760",
      "牌中小丑 · 呼之欲出": "109760",
      "牌中小丑·呼之欲出": "109760",
      "诡案谜集·黑夜之星": "111098",
      "黑夜之星": "111098",
      "诡案谜集·追击者": "111099",
      "追击者": "111099",
      "艾利克斯": "111280",
      "勒维恩·戴维斯": "111282",
      "麻烦": "111282",
      "失败": "111282",
      "意外": "111282",
      "远行": "111282",
      "辗转": "111282",
      "芝加哥": "111282",
      "醉乡民谣": "111282",
      "丹·雷诺斯": "111283",
      "蒂法·洛克哈特": "111292",
      "山村贞子": "111293",
      "破旧打火机": "111294",
      "我的冶金打火机": "111294",
      "静置的双向圣杯": "111295",
      "双向圣杯：待焕活": "111295",
      "双向圣杯：焕然意志": "111295",
      "双向圣杯：血液循环仪式I": "111295",
      "双向圣杯：血液循环仪式II": "111295",
      "双向圣杯：咒术循环仪式I": "111295",
      "双项圣杯": "111295",
      "双向圣杯：咒术循环仪式II": "111295",
      "传送镜": "111680",
      "镜中小鸟": "111680",
      "神奇传送镜": "111680",
      "『叫价牌』": "111775",
      "『瓶中信』": "111775",
      "『弗霖的琴』": "112426",
      "『 弗霖的琴』": "112426",
      "『伊黎丝的祝福』": "112510",
      "『伊黎丝的赞词』": "112510",
      "『伊黎丝的赞美词』": "112510",
      "【周年限定】克里斯(8)": "112540",
      "人到中年": "112540",
      "上作剧情": "112540",
      "永远的克叔": "112540",
      "再次回归": "112540",
      "不屈之枪·阿特瑞斯": "112547",
      "Futūrum（未来）": "112555",
      "弗图AI": "112555",
      "弗图博士": "112555",
      "乐园防御程式": "112555",
      "恐惧气味": "112639",
      "雷夜啸声": "112639",
      "死神": "112639",
      "死亡": "112639",
      "死亡化身": "112639",
      "业火死斗": "112639",
      "業火死鬥": "112639",
      "雷夜嘯聲": "112639",
      "恐懼氣味": "112639",
      "九尾妖狐·阿狸": "112640",
      "丹妮莉丝·坦格利安": "112642",
      "希尔瓦娜斯·风行者": "112645",
      "炽天使之拥": "112646",
      "大天使之杖": "112646",
      "女神之泪": "112646",
      "遍江云霞": "112663",
      "壶中冰心": "112663",
      "千里潋滟": "112663",
      "散佚的文集": "112663",
      "狱炎蛋": "112666",
      "狱炎魔犬": "112666",
      "风物长宜": "113518",
      "小小舞台": "115919",
      "『灰域来音』": "116029",
      "『开裂囊状物』": "117000",
      "『迷翳结晶：聚合颗粒』": "117000",
      "『迷翳结晶：囊状物』": "117000",
      "『迷翳结晶：收集颗粒』": "117000",
      "『迷翳之中』": "117000",
      "『眼榴』": "117000",
      "『迷翳森林回忆录』": "117679",
      "探险三杰士": "117680",
      "『星河碎片』": "118047",
      "桑克瑞德·沃特斯": "118371",
      "白野威": "118372",
      "不知名的石像": "118372",
      "复苏的天照": "118372",
      "净化污秽的天照": "118372",
      "失去力量的白狼": "118372",
      "天照大神": "118372",
      "刀锋女王": "118374",
      "刀锋女王 - 诞生": "118374",
      "刀锋女王 - 复仇": "118374",
      "刀锋女王 - 归宿": "118374",
      "刀锋女王 - 晋升": "118374",
      "刀锋女王 - 重生": "118374",
      "海德林": "118375",
      "你的答案": "118375",
      "生死答问": "118375",
      "维涅斯": "118375",
      "我已倾听，我已感受，我已思考": "118375",
      "萨勒芬妮": "118376",
      "星籁歌姬": "118376",
      "令人不安的契约书": "118378",
      "沼泽黏附者": "118378",
      "灵藤蛋": "118380",
      "迷你蔓生灵树": "118380",
      "白巧克力蛋": "118475",
      "猪庇特": "118475",
      "猛虎贴贴": "118478",
      "情难自抑": "118478",
      "永浴爱河": "118478",
      "绿茵宝钻": "119472",
      "肉垫手套": "119473",
      "百相千面": "120387",
      "百相千面-晦": "120387",
      "百相千面-殇": "120387",
      "百相千面-戏": "120387",
      "阿齐斯": "120388",
      "保加利亚妖王": "120388",
      "纯爷们阿齐斯": "120388",
      "温暖的小屋": "120388",
      "真正的阿齐斯": "120388",
      "白骨露于野": "120390",
      "感召": "120390",
      "涅槃": "120390",
      "叛逆者": "120390",
      "缘起": "120390",
      "纣王·子受": "120390",
      "阿尔瓦罗·索莱尔": "120391",
      "喝奶茶的罗罗": "120391",
      "认真练琴的罗罗": "120391",
      "认真练琴的罗罗]": "120391",
      "走出失恋阴影的罗罗": "120391",
      "自在旅行的罗罗": "120391",
      "开心到旋转的罗罗": "120391",
      "莫甘娜": "120393",
      "凯尔": "120395",
      "回忆之物": "120396",
      "露娜弗蕾亚·诺克斯·芙尔雷": "120396",
      "未来的黎明": "120396",
      "约定守护": "120396",
      "众神祭祀": "120396",
      "干涸的圣杯": "120397",
      "和谐圣杯": "120397",
      "邪恶圣杯": "120397",
      "被尘封的诅咒之书": "120398",
      "被尘封之书": "120398",
      "沉睡之神，拉莱耶之主—克苏鲁": "120398",
      "海德拉": "120398",
      "黄衣之主—哈斯塔": "120398",
      "黄衣之主召唤仪式": "120398",
      "解封的死灵之书": "120398",
      "旧日之印": "120398",
      "蠕动的混沌—奈亚拉托提普": "120398",
      "森之黑山羊母神—纱布尼古拉斯": "120398",
      "死灵之书": "120398",
      "万物归一者—犹格·索托斯": "120398",
      "可爱的小伯": "120400",
      "睡着的小伯": "120400",
      "我的天使": "120400",
      "享受美食的小伯": "120400",
      "兴奋的小伯": "120400",
      "棕色条纹蛋": "120400",
      "沉睡的格罗姆": "120401",
      "苏醒的格罗姆": "120401",
      "长花的蛋": "120401",
      "龙鳞护盾": "121125",
      "龙眼指环": "121125",
      "龙爪王冠": "121125",
      "龙之秘宝": "121125",
      "炎龙火舌": "121125",
      "图腾饼干": "123613",
      "重建熊屋": "123614",
      "虎头怪": "126105",
      "妙手空空": "126105",
      "神力无穷": "126105",
      "十年一梦": "126105",
      "鹰击": "126105",
      "追忆": "126105",
      "克劳斯·迈克尔森": "126107",
      "Klaus": "126107",
      "Niklaus Mikaelson": "126107",
      "The Great Evil": "126107",
      "The original vampire": "126107",
      "The hybrid": "126107",
      "As a father": "126107",
      "The king": "126107",
      "凯特尼斯·伊夫狄恩": "126109",
      "禁果": "126109",
      "破壁": "126109",
      "爱丽丝·盖恩斯巴勒": "126111",
      "你没事吧？": "126111",
      "请收下这朵花": "126111",
      "辣手姐妹花": "126111",
      "跨过这道墙": "126111",
      "原来他一直在我身边": "126111",
      "喋血日记本": "126113",
      "杀意人偶": "126113",
      "无法消解的怨恨": "126113",
      "邪骸转生": "126113",
      "烈焰天使弓": "126115",
      "天使之赐": "126115",
      "真·天使之赐": "126115",
      "【二阶】最终棱镜": "126116",
      "【一阶】棱镜": "126116",
      "棱镜": "126116",
      "哀嚎恶灵": "126119",
      "厄运骸骨": "126119",
      "黑暗水晶": "126119",
      "骷髅勇士": "126119",
      "天灾骑士": "126119",
      "不朽之恋": "126120",
      "创生之柱": "126120",
      "璀璨之焰": "126120",
      "诞星之所": "126120",
      "光子之海": "126120",
      "荒原": "126120",
      "枯荣明灭": "126120",
      "猎户座的明珠": "126120",
      "王座的毗邻": "126120",
      "无光余烬": "126120",
      "无垠": "126120",
      "可怖的眼球": "126121",
      "可疑的触手": "126121",
      "可疑的肉蛋": "126121",
      "可疑的眼睛": "126121",
      "克苏鲁的眼球": "126121",
      "最终幻想XVI": "126122",
      "『金色车票』": "126634",
      "『梦旅存根』": "126634",
      "梦旅存根": "126634",
      "六出冰花": "126968",
      "特供热巧": "126969",
      "极地特快车厢特供 · 热巧 450ml": "126969",
      "HOT CHOC！": "126969",
      "车厢特供 · 热巧 450ml": "126969",
      "『列车长』": "127026",
      "『极地特快列车长』": "127026",
      "『即将抵达终点站』": "127026",
      "『终点站：极地』": "127026",
      "岛屿探险家": "128473",
      "征服之王": "128474",
      "尼克·王尔德": "129113",
      "初遇朱迪": "129113",
      "重归于好": "129113",
      "实现梦想": "129113",
      "朱迪·霍普斯": "129115",
      "初入动物城": "129115",
      "卸下警徽": "129115",
      "官复原职": "129115",
      "“米凯拉的锋刃”玛莲妮亚": "129118",
      "圣树底部的半神": "129118",
      "米凯拉的锋刃": "129118",
      "猩红恐惧": "129118",
      "破碎战争": "129118",
      "“腐败女神”玛莲妮亚": "129118",
      "改造后的火枪": "129119",
      "黄金之旅": "129119",
      "极·龙の意": "129119",
      "射手的火枪": "129119",
      "银色飞鹰": "129119",
      "冰海钓竿": "129120",
      "步行鮟鱇": "129120",
      "克苏鲁的仆从": "129120",
      "虚空之海的鲸": "129120",
      "崩朽龙卵": "129122",
      "崩朽青铜龙幼崽": "129122",
      "崩朽之青铜龙王": "129122",
      "波纹蓝蛋": "129123",
      "幽光蓝龙": "129123",
      "注入能量的波纹蓝蛋": "129123",
      "雄躯的昇格": "129126",
      "极客的晚宴": "129127",
      "里昂（RE4）": "129158",
      "致命危机": "129158",
      "特工里昂的奇幻漂流": "129158",
      "衣衫褴褛": "129158",
      "终归一人": "129158",
      "苇名弦一郎": "129159",
      "巴流·苇名弦一郎": "129159",
      "飞渡轻舟": "129159",
      "巴流·雷之矢": "129159",
      "不死斩【开门】": "129159",
      "克莱夫・罗兹菲尔德": "129161",
      "约书亚・罗兹菲尔德": "129162",
      "『不败之花』": "129610",
      "龙鳞石": "129790",
      "『先知灵药:真视』": "132585",
      "『先知灵药』": "132585",
      " 『先知灵药：真视』": "132585",
      " 『先知灵药』": "132585",
      "幽浮起司堡": "133898",
      "幽浮起司煲": "133898",
      "一只陶瓮": "133899",
      "瓮中能言蛙": "133899",
      "『流星赶月：宙刃』": "134127",
      "『流星赶月』": "134127",
      "阿怪": "134663",
      "生活拍立得": "135530",
      "照相机": "135530",
      "巴比伦辞典": "135738",
      "巴比伦辞典㊥": "135738",
      "金翼使": "136035",
      "金翼使(30d)": "136035",
      "金翼使㊊": "136035",
      "丹雀衔五穗，人间始丰登": "137106",
      "烈日载雨、风禾尽起": "137106",
      "瑞雪兆丰年，生灵万物新": "137106",
      "五谷丰年": "137106",
      "寻觅": "137106",
      "吉尔·沃瑞克": "137107",
      "大罪人希德": "137108",
      "雷之显化者": "137108",
      "希德法斯·特拉蒙": "137108",
      "召唤兽拉姆": "137108",
      "狄翁·勒萨若": "137110",
      "狄翁・勒萨若": "137110",
      "皇子与侍从": "137110",
      "弑父者": "137110",
      "百万火光": "137110",
      "十垓火光": "137110", 
      "Act of Ultimate Trust": "137117",
      "Awakening Earth": "137117",
      "Birth of Sky": "137117",
      "Emerging Force": "137117",
      "Life of Ice": "137117",
      "Life of Water": "137117",
      "Life of Wind": "137117",
      "Master of Six Lives": "137117",
      "神秘挑战书": "137117",
      "尾崎八项": "137117",
      "敖蜃星": "137113",
      "格林星": "137113",
      "太空列车": "137113",
      "太空列车票": "137113",
      "甜点星": "137113",
      "坏掉的月亮提灯": "137134",
      "月亮提灯": "137134",
      "GM吸血伯爵": "137135",
      "可爱黑猫": "137135",
      "可爱毛团": "137135",
      "吸血猫蛋": "137135",
      "穿过的白袜": "137137",
      "崭新的白袜": "137137",
      "装了衣物的纸盒": "137137",
      "变身器": "137697",
      "月棱镜": "137697",
      "近地夜航": "138136",
      "『分析天平』": "139404",
      "巨力橡果(30d)": "139406",
      "巨力橡果": "139406",
      "巨力橡果㊊": "139406",
      "古老金币": "141060",
      "璀璨金币": "141060",
      "不洁圣子": "141062",
      "『钟楼日暮』": "141820",
      "『钟楼盐水棒冰』": "141820",
      "小小安全帽": "142917",
      "金牌矿工": "142919",
      "『泥潭颂唱者』": "143409",
      "莱昂纳多·迪卡普里奥": "144099",
      "了不起的盖茨比": "144099",
      "盗梦空间": "144099",
      "Forever Titanic": "144099",
      "叶卡捷琳娜": "144101",
      "叶卡捷琳娜大帝": "144101",
      "肃弓": "144103",
      "最初的轻语": "144103",
      "轻语": "144103",
      "最后的轻语": "144103",
      "半生黄金种": "144104",
      "托莉娜的睡莲": "144104",
      "托莉娜的微笑": "144104",
      "米凯拉的花": "144104",
      "神人的编制发": "144104",
      "Zootopia": "144106",
      "启程": "144106",
      "全景": "144106",
      "沙漠": "144106",
      "冰原": "144106",
      "雨林": "144106",
      "御医神兔": "144664",
      "脉律辐石": "144944",
      "劫掠核芯": "144945",
      "幸运女神的微笑": "146144",
      "『逆境中的幸运女神』": "146145",
      "『南瓜拿铁』": "146962",
      "梅克军徽": "147987",
      "贵族与野兽": "147987",
      "占卜与日月": "147987",
      "羽人与蜕变": "147987",
      "小丑与格雷与星光璀璨": "147987",
      "奎兰": "150222",
      "无瑕的回忆": "150222",
      "水银日报社特约调查员": "150223",
      "朴素的誓言": "150223",
      "古烈": "150911",
      "奇异博士": "150913",
      "史蒂芬·斯特兰奇": "150913",
      "午夜悲剧": "150913",
      "时间牢笼": "150913",
      "至尊法师": "150913",
      "托比·马奎尔": "150914",
      "抉择": "150914",
      "改变": "150914",
      "蜘蛛侠": "150914",
      "阿丽塔": "150915",
      "重生": "150915",
      "天人永隔": "150915",
      "剑指撒冷": "150915",
      "圣诞有铃": "150917",
      "圣诞寻铃": "150917",
      "羽毛胸针": "150919",
      "苏格兰圆脸胖鸡": "150919",
      "苏格兰圆脸胖鸡[Pro Max]": "150919",
      "都市：天际线2": "150921",
      "银色溜冰鞋": "152367",
      "『冰雕马拉橇』": "152369",
      "【新春限定】果体 隆": "153403",
      "汉尼拔": "153404",
      "晚宴": "153404",
      "掌中命运": "153404",
      "未散的宴席": "153404",
      "梅琳娜Melina": "153407",
      "约定立下": "153407",
      "明月知心": "153407",
      "亭亭如盖": "153407",
      "使命终成": "153407",
      "宵眼定死": "153407",
      "黄粱一梦": "153407",
      "贝儿(Belle)": "153409",
      "Belle": "153409",
      "Come across": "153409",
      "Crush": "153409",
      "Beauty and the Beast": "153409",
      "普通羊毛球": "153411",
      "整理好的线团": "153411",
      "织好的布料": "153411",
      "暖心小斗篷": "153411",
      "神秘天球": "153413",
      "乐园之初": "153413",
      "繁盛之景": "153413",
      "人世之始": "153413",
      "堕落之实": "153413",
      "原罪之初": "153413",
      "审判终至": "153413",
      "历练人间": "153413",
      "堕入地狱": "153413",
      "百鬼夜行": "153413",
      "婚姻登记册": "153415",
      "爱情是一只小鸟": "153415",
      "寻求恋爱是我的自由": "153415",
      "孤独是一种常态": "153415",
      "我有一个暗恋对象": "153415",
      "这天我们之间是粉色的": "153415",
      "狂热恋爱中": "153415",
      "520认证": "153415",
      "组成家庭": "153415",
      "健忘礼物盒": "153416",
      "打开一个礼物盒": "153416",
      "打开又一个礼物盒": "153416",
      "打开又亿个礼物盒": "153416",
      "变色的礼物盒": "153416",
      "智力鉴定书": "153416",
      "脏兮兮的蛋": "153419",
      "爱你的小蟑螂": "153419",
      "螂的诱惑": "153419",
      "神秘Bug！": "153419",
      "战螂": "153419",
      "一只来自远方的螂": "153419",
      "Jo太螂": "153419",
      "小狮欢舞": "155113",
      "『搓粉团珠』": "155117",
      "永亘环": "155195",
      "神奇宝贝大师球": "157540",
      "神奇宝贝图鉴": "157542",
      "『道具超市』": "157750",
      "猫咪点唱机㊊": "157787",
      "猫咪点唱机": "157787",
      "肉乖乖": "159166",
      "眠眠茧": "159166",
      "璀璨闪蝶": "159166",
      "哀恸魔蝶": "159166",
      "灵魂残絮聚合法": "159167",
      "脏器再生手术": "159167",
      "实验体出逃事故": "159167",
      "自定义男从Homunculus": "159167",
      "John Reese": "160193",
      "穿靴子的猫": "160194",
      "万众瞩目": "160194",
      "遭遇死神": "160194",
      "醉生梦死": "160194",
      "迷途羔羊": "160194",
      "许愿之星": "160194",
      "传奇": "160194",
      "狗狗": "160195",
      "dog": "160195",
      "perrito": "160195",
      "阿加莎·哈克尼斯": "160196",
      "女巫起源": "160196",
      "深陷混沌": "160196",
      "女巫之路传说": "160196",
      "圣水瓶": "160197",
      "受赐福的圣水瓶": "160197",
      "破碎的圣水瓶": "160197",
      "重铸的圣水瓶": "160197",
      "新神的赐福": "160197",
      "弯钩与连枷": "160198",
      "木乃伊巨人": "160198",
      "巴斯泰托信徒": "160198",
      "索贝克信徒": "160198",
      "阿努比斯信徒": "160198",
      "黑神话:悟空": "160199",
      "猫头鹰守卫": "160795",
      "守卫: 不可选中": "160795",
      "守卫: 轮班侦察": "160795",
      "守卫: 坚守眼位": "160795",
      "『凯旋诺书』": "160797",
      "『林中过夜』": "160798",
      "『绿茵甘露』": "159704",
      "位于左侧的随从已派遣远征": "159704",
      "胡子贴纸": "161623",
      "香蕉特饮": "111748",
      "迷之香蕉特饮": "111748",
      " 迷之香蕉特饮": "111748",
      "特殊-纯真护剑": "112214",
      "纯真护剑": "112214",
      "纯真护剑㊕": "112214",
      "纯真护剑 · 这把剑守护每个孩子无论他有没有被神选中": "112214",
      "『活动代币』": "120137",
      "『叫价牌』": "120137",
      "『活动代币兑金币』": "120137",
      "(人)血球蛋白": "121881",
      "小型流动血瓶": "121881",
      "失去生机的血瓶": "121881",
      "中型储蓄血瓶": "121881",
      "失去生机的血瓶": "121881",
      "大型贮藏血瓶": "121881",
      "失活的血瓶": "121881",
      "『酒馆蛋煲』": "122417",
      "格雷的扑克牌": "130782",
      "格雷的扑克牌": "130782",
      "『厢庭望远』": "161935",
      "十周年扭蛋 - 绿": "113501",
      "十周年扭蛋 - 蓝": "113501",
      "十周年扭蛋 - 橙": "113501",
      "十周年扭蛋 - 紫": "113501",
      "十周年扭蛋 - 红": "113501",
      "巴哈姆特": "162881",
      "愤怒的巴哈姆特": "162881",
      "失败的封印仪式": "162881",
      "百万核爆": "162881",
      "巴哈姆特大迷宫": "162881",
      "龙神巴哈姆特": "162881",
      "至尊巴哈姆特": "162881",
      "约翰·康斯坦丁": "162882",
      "牛局长博戈": "162885",
      "勃然大怒的牛局长": "162885",
      "不为人知的小秘密": "162885",
      "亨利.卡维尔": "162886",
      "亨利·卡维尔": "162886",
      "仇恋": "162886",
      "都铎王朝": "162886",
      "惊天战神": "162886",
      "明日之子": "162886",
      "秘密特工": "162886",
      "bvs": "162886",
      "猎魔人": "162886",
      "死侍": "162886",
      "人间之神": "162886",
      "红夫人": "162888",
      "玛丽": "162888",
      "希望的残像": "162888",
      "破镜": "162888",
      "荣光的残像": "162888",
      "莉莉娅·考尔德（Lilia Calderu）": "162889",
      "QUEEN of CUPS.【圣杯皇后】": "162889",
      "THREE of PENTACLES.【钱币三】": "162889",
      "KNIGHT of WANDS.【权杖骑士】": "162889",
      "THE HIGH PRIESTESS.【女祭司】": "162889",
      "THREE of SWORDS.【宝剑三】": "162889",
      "THE TOWER (REVERSED).【高塔·逆位】": "162889",
      "DEATH.【死神】": "162889",
      "THE TOWER (UP RIGHT).【高塔·正位】": "162889",
      "我的路从时间蜿蜒而出": "162889",
      "尼特公仔": "162891",
      "會動的尼特公仔": "162891",
      "健壯的尼特公仔": "162891",
      "威猛尼特": "162891",
      "黑暗之魂系列": "162893",
      "女巫之路": "162894",
      "『狄文卡德的残羽』": "163415",
      "鎏彩万幢": "164432",
      "呆猫": "167219",
      "传说中的黑龙": "167220",
      "检定场": "168856",
      "命运的轮廓": "168858",
      "灯载情绵": "169887",
      "“半狼”布莱泽": "171601",
      "合作": "171601",
      "相约于群星": "171601",
      "发狂": "171601",
      "You Can Pet Blaidd": "171601",
      "炽焰咆哮虎": "171602",
      "火斑喵": "171602",
      "炽焰咆哮虎的训练": "171602",
      "咆哮虎的冠军之路": "171602",
      "基努·里维斯": "171603",
      "生死时速": "171603",
      "极速追杀": "171603",
      "The One": "171603",
      "黑客帝国": "171603",
      "地狱神探": "171603",
      "琴.葛蕾": "171605",
      "牺牲": "171605",
      "Honey B Lovely": "171606",
      "Honey Bee": "171606",
      "蜜蜂魔物": "171606",
      "蜂蜂小甜心": "171606",
      "黑暗封印": "171607",
      "梅贾的窃魂卷": "171607",
      "饱食之戒": "171607",
      "躁动之戒": "171607",
      "秘法之书": "171607",
      "光之少女の魔法书": "171607",
      "枯木法杖": "171608",
      "蓄能法杖": "171608",
      "元灵化法杖": "171608",
      "水晶化法杖": "171608",
      "结晶化法杖": "171608",
      "千年积木": "171609",
      "寄宿法老王灵魂的容器": "171609",
      "辉夜姬的五难题": "171610",
      "佛御石之钵": "171610",
      "蓬莱的玉枝": "171610",
      "火鼠的皮衣": "171610",
      "龙头的宝玉": "171610",
      "燕的子安贝": "171610",
      "竹取物语": "171610",
      "末影珍珠": "171611",
      "原初の珍珠": "171611",
      "更多的珍珠": "171611",
      "隔墙有眼": "171611",
      "The End": "171611",
      "双生蛋": "171614",
      "双生": "171614",
      "三羽蚀鸮": "171614",
      "三羽蚀鸮": "171614",
      "六翼神鸮": "171614",
    }

# 勋章数据（简化示例）
MEDAL_MAPPINGS = {
  "史莱姆蛋": "史莱姆蛋\n【勋章类型】宠物\n【入手条件】发帖数≥30\n【商店售价】220血液\n【等级1】3% 回帖咒术+1▕▏升级条件：消耗50金币\n【等级2】6% 回帖咒术+1▕▏升级条件：消耗75金币\n【等级3】9% 回帖咒术+1▕▏升级条件：消耗100金币\n【 Max 】12% 回帖咒术+1、发帖金币+2",
  "迷のDoge": "迷のDoge\n【勋章类型】宠物\n【入手条件】主题数 >= 6\n【商店售价】100金币\n【等级1】2% 回帖血液+1 金币+1▕▏升级条件：消耗50金币\n【等级2】4% 回帖血液+1 金币+1▕▏升级条件：消耗50金币\n【 Max 】6% 回帖血液+1 金币+2 咒术+1",
  "洞窟魔蛋": "洞窟魔蛋\n【勋章类型】宠物\n【入手条件】咒术≥40\n【商店售价】1金币\n【等级1】无属性▕▏升级条件：消耗30血液\n【等级2】5% 回帖咒术+1▕▏升级条件：消耗70血液\n【 Max 】14% 回帖咒术+2 血液-2、发帖咒术+2",
  "红龙蛋": "红龙蛋\n【勋章类型】宠物\n【入手条件】血液≥200\n【商店售价】150金币\n【等级1】无属性▕▏升级条件：血液≥20\n【等级2】5% 回帖血液+1 金币-1▕▏升级条件：血液≥250\n【 Max 】10% 回帖血液+2 金币-1",
  "黑龙蛋": "黑龙蛋\n【勋章类型】宠物\n【入手条件】咒术≥20\n【商店售价】150金币\n【等级1】无属性▕▏升级条件：升级条件：咒术≥10\n【等级2】5% 回帖咒术+1 金币-1▕▏升级条件：升级条件：咒术≥40\n【 Max 】10% 回帖咒术+1 金币-1",
  "腐化龙蛋": "腐化龙蛋\n【勋章类型】宠物\n【入手条件】金币≥200\n【商店售价】150金币\n【等级1】无属性▕▏升级条件：堕落≥10\n【等级2】5% 回帖金币+1 血液-1▕▏升级条件：堕落≥40\n【 Max 】10% 回帖金币+2 血液-1",
  "漆黑的蝎卵": "漆黑的蝎卵\n【勋章类型】宠物\n【入手条件】血液≥300\n【商店售价】200金币\n【等级1】4% 回帖咒术+1、发帖咒术+1▕▏升级条件：灵魂≥1\n【 Max 】2% 回帖咒术+1、发帖灵魂+1",
  "【年中限定】GM村金蛋": "【年中限定】GM村金蛋\n【勋章类型】宠物\n【入手条件】知识≥20\n【商店售价】618金币\n【等级1】1% 发帖金币+1▕▏升级条件：知识≥35\n【等级2】12% 回帖金币+1、发帖金币+2▕▏升级条件：知识≥60\n【 Max 】25% 回帖金币+2、发帖金币+4",
  "GM村金蛋": "GM村金蛋\n【入手条件】知识≥20\n【商店售价】618金币\n【等级1】1% 发帖金币+1▕▏升级条件：知识≥35\n【等级2】12% 回帖金币+1、发帖金币+2▕▏升级条件：知识≥35\n【 Max 】25% 回帖金币+2、发帖金币+4",
  "詹姆斯‧维加": "詹姆斯‧维加\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】450金币\n【等级1】5% 发帖血液+1▕▏升级条件：追随≥20\n【等级2】10% 回帖血液+1、发帖血液+1▕▏升级条件：追随≥55\n【 Max 】15% 回帖血液+1、发帖血液+1",
  "奧倫": "奧倫\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】300金币\n【等级1】无属性▕▏升级条件：50旅程\n【等级2】10% 发帖金币+1▕▏升级条件：120旅程\n【 Max 】20% 发帖金币+3",
  "希德‧海温特": "希德‧海温特\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】450金币\n【等级1】无属性▕▏升级条件：消耗50金币\n【等级2】4% 回帖金币+1▕▏升级条件：消耗100金币\n【等级3】8% 回帖金币+1▕▏升级条件：消耗150金币\n【 Max 】12% 回帖金币+2",
  "吉姆‧雷诺": "吉姆‧雷诺\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】350金币\n【 Max 】12% 发帖旅程+1",
  "法卡斯": "法卡斯\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】300金币\n【 Max 】10% 回帖血液+1、发帖血液+1",
  "皮尔斯‧尼凡斯": "皮尔斯‧尼凡斯\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】无属性▕▏升级条件：消耗50金币\n【等级2】无属性▕▏升级条件：消耗50血液\n【等级3】无属性▕▏升级条件：消耗50咒术\n【等级4】无属性▕▏升级条件：血液≥150\n【 Max 】15% 回帖血液+2、发帖血液+2",
  "文森特‧瓦伦丁": "文森特‧瓦伦丁\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】350金币\n【 Max 】10% 回帖咒术+1、发帖堕落+1",
  "巴尔弗雷亚": "巴尔弗雷亚\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】350金币\n【等级1】4% 回帖金币+2、发帖金币+2▕▏升级条件：知识≥10\n【等级2】8% 回帖金币+2、发帖金币+2▕▏升级条件：知识≥30\n【 Max 】12% 回帖金币+2、发帖金币+2",
  "但丁": "但丁\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】450金币\n【等级1】5% 发帖咒术+1▕▏升级条件：10血液\n【等级2】8% 发帖咒术+1▕▏升级条件：10咒术\n【等级3】10% 发帖咒术+2▕▏升级条件：1灵魂\n【等级4】15% 发帖咒术+2▕▏升级条件：30咒术\n【 Max 】20% 发帖咒术+3",
  "盖里": "盖里\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】420金币\n【等级1】2% 发帖旅程+1▕▏升级条件：好友≥10\n【等级2】5% 回帖血液+1、发帖旅程+1▕▏升级条件：好友≥30\n【 Max 】7% 回帖血液+1 金币+1、发帖旅程+1",
  "梅格": "梅格\n【勋章类型】女从\n【入手条件】无\n【商店售价】300金币\n【等级1】5% 回帖堕落+1▕▏升级条件：追随≥22\n【等级2】8% 回帖堕落+1▕▏升级条件：追随≥44\n【 Max 】12% 回帖堕落+1 血液+1、发帖堕落+1 血液+1",
  "亚力斯塔尔": "亚力斯塔尔\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】450金币\n【等级1】5% 发帖血液+2▕▏升级条件：发帖数≥200\n【等级2】10% 发帖血液+2▕▏升级条件：发帖数≥500\n【等级3】15% 发帖血液+2▕▏升级条件：消耗1灵魂\n【 Max 】40% 发帖血液+3",
  "罗伯‧史塔克": "罗伯‧史塔克\n【勋章类型】真人男从\n【入手条件】堕落＜30\n【商店售价】500金币\n【等级1】1% 回帖金币+1 知识+1▕▏升级条件：堕落≥10\n【等级2】2% 回帖金币+1 知识+1▕▏升级条件：消耗200金币\n【等级3】5% 回帖金币+1、发帖旅程+1▕▏升级条件：主题≥35\n【等级4】8% 回帖金币+1、发帖旅程+1▕▏升级条件：消耗300血液\n【等级5】10% 回帖金币+1、发帖旅程+1▕▏升级条件：积分≥300\n【等级6】11% 回帖金币+1、发帖旅程+1 金币+3▕▏升级条件：消耗500金币\n【等级7】13% 回帖金币+2、发帖旅程+1 金币+3▕▏升级条件：消耗700血液\n【 Max 】50% 发帖金币+7 咒术+1",
  "亚当‧简森": "亚当‧简森\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】350金币\n【等级1】5% 回帖金币+1、发帖血液+1▕▏升级条件：消耗20咒术\n【等级2】6% 回帖金币+1、发帖血液+2▕▏升级条件：知识≥15\n【等级3】7% 回帖金币+2、发帖血液+3▕▏升级条件：消耗50咒术\n【等级4】8% 回帖金币+2、发帖血液+4▕▏升级条件：知识≥45\n【 Max 】9% 回帖金币+3、发帖血液+5",
  "布莱恩·欧康纳": "布莱恩·欧康纳\n【入手条件】追随≥20\n【商店售价】400金币\n【等级1】4% 回帖金币+1▕▏升级条件：消耗100金币\n【等级2】6% 回帖金币+1▕▏升级条件：消耗100血液\n【等级3】8% 回帖金币+1▕▏升级条件：追随≥101\n【 Max 】10% 回帖金币+2、发帖旅程+1",
  "迪恩·温彻斯特": "迪恩·温彻斯特\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】400金币\n【等级1】2% 回帖血液+2、发帖血液+2▕▏升级条件：血液≥100\n【等级2】4% 回帖血液+2、发帖血液+2▕▏升级条件：血液≥250\n【等级3】6% 回帖血液+2、发帖血液+2▕▏升级条件：血液≥450\n【等级4】8% 回帖血液+2、发帖血液+2▕▏升级条件：消耗350金币\n【 Max 】10% 回帖血液+2、发帖血液+2",
  "山姆·温彻斯特": "山姆·温彻斯特\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】400金币\n【等级1】2% 回帖金币+2、发帖金币+2▕▏升级条件：咒术≥10\n【等级2】4% 回帖金币+2、发帖金币+2▕▏升级条件：咒术≥50\n【等级3】6% 回帖金币+2、发帖金币+2▕▏升级条件：咒术≥100\n【等级4】8% 回帖金币+2、发帖金币+2▕▏升级条件：消耗350金币\n【 Max 】10% 回帖金币+2、发帖金币+2",
  "魔术师奥斯卡": "魔术师奥斯卡\n【勋章类型】真人男从\n【入手条件】旅程≥10\n【商店售价】490金币\n【等级1】5% 回帖堕落+1▕▏升级条件：消耗30咒术\n【等级2】6% 回帖堕落-1▕▏升级条件：消耗75咒术\n【等级3】7% 回帖堕落+1▕▏升级条件：堕落≥20\n【等级4】8% 回帖堕落-1、发帖知识+1▕▏升级条件：消耗120咒术\n【等级5】9% 回帖堕落+1 咒术+1 发帖咒术+3 知识+1▕▏升级条件：咒术≥100\n【 Max 】10% 回帖堕落-1 咒术+1、发帖知识+1 咒术+3",
  "戴蒙‧萨尔瓦托": "戴蒙‧萨尔瓦托\n【勋章类型】真人男从\n【入手条件】堕落≥10\n【商店售价】450金币\n【等级1】5% 回帖血液+5 堕落+1、发帖血液+5 堕落+1▕▏升级条件：血液≥5\n【等级2】无属性▕▏升级条件：血液≥20\n【等级3】5% 回帖血液+1▕▏升级条件：血液≥50\n【等级4】20% 回帖血液+1、发帖血液+3▕▏升级条件：血液≥55\n【等级5】5% 回帖血液+1、发帖血液+3▕▏升级条件：血液≥100\n【等级6】10% 回帖血液-1、发帖堕落+2 血液+2▕▏升级条件：血液≥108\n【等级7】100% 回帖血液+2 堕落+1、发帖血液+2 堕落+1▕▏升级条件：血液≥110\n【等级8】无属性▕▏升级条件：血液≥150\n【等级9】2% 回帖血液-5 堕落+5、发帖灵魂+1▕▏升级条件：血液≥160\n【等级10】20% 回帖血液-1 堕落+1▕▏升级条件：血液≥300\n【 Max 】无属性",
  "库伦(起源)": "库伦(起源)\n【入手条件】咒术≤25\n【商店售价】480金币\n【等级1】18% 回帖血液+2、发帖血液+6▕▏升级条件：咒术≥11\n【 Max 】7% 发帖血液+8 咒术-1",
  "铁牛": "铁牛\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】5% 回帖血液-1 金币+2▕▏升级条件：血液≥350\n【等级2】7% 回帖血液-1 金币+2▕▏升级条件：消耗200血液\n【 Max 】13% 回帖血液-1 金币+3 堕落+1、发帖血液-1 金币+3 堕落+1",
  "康纳‧沃什": "康纳‧沃什\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】400金币\n【等级1】5% 回帖血液+1▕▏升级条件：消耗50血液\n【等级2】6% 回帖血液+1、发帖血液+1▕▏升级条件：堕落≥50\n【等级3】7% 发帖知识+1 血液+1▕▏升级条件：知识≥50\n【 Max 】8% 回帖堕落+1 血液+1、发帖知识+1 血液+1",
  "尼克·贝特曼": "尼克·贝特曼\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】500金币\n【等级1】无属性▕▏升级条件：血液≥600\n【等级2】3% 发帖血液+3▕▏升级条件：金币≥600\n【等级3】5% 发帖血液+3 咒术+2▕▏升级条件：血液≥900\n【等级4】7% 发帖血液+3 旅程+1▕▏升级条件：金币≥900\n【等级5】15% 发帖血液+3 金币+2▕▏升级条件：灵魂≥1\n【 Max 】20% 发帖血液+4 金币+3",
  "杰夫‧莫罗": "杰夫‧莫罗\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】无属性▕▏升级条件：消耗20血液\n【等级2】3% 发帖知识+1▕▏升级条件：血液≥100\n【等级3】3% 回帖知识+1、发帖知识+1▕▏升级条件：血液≥200\n【等级4】5% 回帖知识+1、发帖知识+1▕▏升级条件：消耗600金币\n【 Max 】7% 回帖知识+1、发帖知识+1 旅程+1",
  "维吉尔": "维吉尔\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】无属性▕▏升级条件：消耗20血液\n【等级2】无属性▕▏升级条件：消耗80血液\n【等级3】4% 回帖咒术+1、发帖咒术+1▕▏升级条件：堕落-10\n【 Max 】6% 回帖咒术+1、发帖咒术+1 知识+1",
  "威尔卡斯": "威尔卡斯\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】无属性▕▏升级条件：消耗10血液\n【等级2】无属性▕▏升级条件：消耗30血液\n【等级3】无属性▕▏升级条件：消耗60血液\n【 Max 】2% 发帖灵魂+1",
  "卡斯迪奥": "卡斯迪奥\n【勋章类型】真人男从\n【入手条件】堕落≤10\n【商店售价】520金币\n【等级1】2% 回帖堕落-1、发帖堕落-1▕▏升级条件：消耗50血液\n【等级2】3% 回帖堕落-1、发帖堕落-1▕▏升级条件：消耗80血液\n【等级3】4% 回帖堕落-1、发帖堕落-1 血液+1▕▏升级条件：消耗100血液\n【等级4】5% 回帖堕落-1、发帖堕落-1 血液+1▕▏升级条件：消耗1灵魂\n【等级5】无属性▕▏升级条件：堕落≥1\n【等级6】2% 回帖血液+5 堕落-3、发帖灵魂+1 堕落-5▕▏升级条件：堕落≥16\n【等级7】15% 回帖血液+2 堕落-1、发帖血液+3 堕落-3▕▏升级条件：堕落≥66\n【等级8】15% 回帖金币+2 堕落+1、发帖金币+3 堕落+3▕▏升级条件：堕落≥116\n【等级9】2% 回帖金币+5 堕落+3、发帖灵魂+1 堕落+5▕▏升级条件：堕落≥131\n【 Max 】无属性",
  "虎克船长": "虎克船长\n【勋章类型】真人男从\n【入手条件】旅程≥15\n【商店售价】320金币\n【等级1】1% 发帖旅程+1▕▏升级条件：消耗150金币\n【等级2】1% 回帖旅程+1、发帖旅程+1▕▏升级条件：消耗200血液\n【等级3】2% 回帖旅程+1、发帖旅程+1▕▏升级条件：消耗-5堕落\n【等级4】2% 回帖旅程+1、发帖旅程+1▕▏升级条件：消耗-10咒术\n【等级5】3% 回帖旅程+1、发帖旅程+1▕▏升级条件：消耗-1知识\n【 Max 】3% 回帖旅程+1、发帖旅程+1",
  "卢西亚诺‧科斯塔": "卢西亚诺‧科斯塔\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】380金币\n【等级1】5% 回帖血液+1、发帖血液+1▕▏升级条件：知识≥100\n【 Max 】30% 回帖血液+1、发帖血液+1",
  "安德森‧戴维斯": "安德森‧戴维斯\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】500金币\n【等级1】2% 回帖血液+1、发帖咒术+1▕▏升级条件：堕落≥5\n【等级2】4% 回帖血液+1、发帖咒术+1▕▏升级条件：堕落≥10\n【等级3】6% 回帖血液+1、发帖咒术+1▕▏升级条件：堕落≥25\n【等级4】8% 回帖血液+1、发帖咒术+1▕▏升级条件：堕落≥50\n【等级5】10% 回帖金币-1 血液+1、发帖咒术+1▕▏升级条件：堕落≥100\n【 Max 】15% 回帖金币-1 血液+1 咒术+1、发帖咒术+2",
  "尤利西斯": "尤利西斯\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】250金币\n【等级1】无属性▕▏升级条件：消耗200血液\n【等级2】2% 回帖血液+1▕▏升级条件：消耗-10堕落\n【等级3】7% 回帖金币+1▕▏升级条件：金币≥600\n【 Max 】无属性",
  "克里斯‧雷德菲尔德": "克里斯‧雷德菲尔德\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】550金币\n【等级1】3% 回帖血液+1、发帖血液+1▕▏升级条件：消耗30血液\n【等级2】3% 回帖血液+2、发帖血液+2▕▏升级条件：追随≥50\n【等级3】4% 回帖血液+2、发帖血液+2 旅程+1▕▏升级条件：追随≥100\n【等级4】5% 回帖血液+2 旅程+1、发帖血液+2 旅程+1▕▏升级条件：消耗100血液\n【 Max 】7% 回帖血液+2 旅程+1 金币+1、发帖血液+2 旅程+1",
  "裸体克里斯": "裸体克里斯\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】888金币\n【 Max 】100% 回帖金币+1、发帖金币+1",
  "凯登‧阿兰科": "凯登‧阿兰科\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】550金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗20血液\n【等级2】5% 回帖金币+1 血液+1▕▏升级条件：血液≥60\n【等级3】7% 回帖金币+1 血液+1▕▏升级条件：消耗150金币\n【等级4】10% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：灵魂≥1\n【等级5】20% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：消耗800金币\n【 Max 】50% 回帖金币+1 血液+1、发帖金币+3 血液+3",
  "肥皂": "肥皂\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】无属性▕▏升级条件：消耗60金币\n【等级2】3% 回帖金币+1、发帖金币+5▕▏升级条件：消耗150金币\n【等级3】7% 回帖金币+1、发帖金币+5▕▏升级条件：追随≥50\n【等级4】10% 回帖金币+1、发帖金币+5▕▏升级条件：追随≥80\n【等级5】12% 回帖金币+1、发帖金币+5▕▏升级条件：消耗-200金币\n【 Max 】12% 回帖金币+2、发帖金币+5 旅程+1",
  "奥利弗‧奎恩": "奥利弗‧奎恩\n【勋章类型】真人男从\n【入手条件】堕落＜50\n【商店售价】400金币\n【等级1】5% 发帖血液+2▕▏升级条件：消耗40血液\n【等级2】7% 发帖血液+2▕▏升级条件：消耗65血液\n【等级3】9% 发帖血液+2▕▏升级条件：主题数≥15\n【等级4】11% 回帖血液+1、发帖血液+2▕▏升级条件：追随≥45\n【等级5】13% 回帖血液+1、发帖血液+2▕▏升级条件：消耗30堕落\n【等级6】13% 回帖血液+1、发帖血液+2▕▏升级条件：消耗500血液\n【 Max 】15% 回帖血液+1 堕落-1、发帖血液+2 堕落-3",
  "猫化弩哥": "猫化弩哥\n【勋章类型】真人男从\n【入手条件】堕落＞5\n【商店售价】200金币\n【 Max 】6% 发帖知识+1 咒术+1",
  "史蒂夫‧金克斯": "史蒂夫‧金克斯\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】230金币\n【等级1】3% 回帖咒术+1、发帖咒术+1▕▏升级条件：咒术≥120\n【 Max 】5% 回帖堕落+2、发帖堕落+2",
  "巴特‧贝克": "巴特‧贝克\n【勋章类型】真人男从\n【入手条件】堕落＜20\n【商店售价】180金币\n【等级1】2% 回帖堕落+1▕▏升级条件：20堕落\n【等级2】4% 回帖堕落+1▕▏升级条件：50堕落\n【 Max 】6% 回帖堕落+1 金币+1、发帖堕落+1 金币+1",
  "杰森·斯坦森": "杰森·斯坦森\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】490金币\n【等级1】无属性▕▏升级条件：金币≥50\n【等级2】1% 回帖血液+1▕▏升级条件：金币≥300\n【等级3】5% 回帖血液+1、发帖血液+1▕▏升级条件：金币≥750\n【等级4】10% 回帖堕落-1 血液+1、发帖堕落-1 血液+1▕▏升级条件：血液≥1000\n【等级5】15% 回帖堕落-1 血液+2、发帖堕落-1 血液+2▕▏升级条件：消耗1灵魂\n【 Max 】25% 回帖堕落-1 血液+2、发帖堕落-1 血液+2",
  "哈尔‧乔丹": "哈尔‧乔丹\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】550金币\n【等级1】无属性▕▏升级条件：知识≥15\n【等级2】2% 回帖咒术+1▕▏升级条件：追随≥75\n【等级3】5% 回帖咒术+1、发帖咒术+1▕▏升级条件：积分≥100（手动升级，但是无消耗）\n【 Max 】7% 回帖咒术+1、发帖咒术+1 旅程+1",
  "艾德尔": "艾德尔\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】280金币\n【 Max 】8% 发帖血液+3 旅程+1",
  "盖拉斯‧瓦卡瑞安": "盖拉斯‧瓦卡瑞安\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】450金币\n【等级1】5% 回帖金币+1、发帖金币+1▕▏升级条件：灵魂≥2\n【等级2】5% 回帖知识+1、发帖知识+1▕▏升级条件：血液≥50\n【等级3】8% 回帖血液+1、发帖血液+1▕▏升级条件：血液≥600\n【 Max 】12% 回帖金币+1、发帖金币+2",
  "戴尔‧芭芭拉": "戴尔‧芭芭拉\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】280金币\n【等级1】无属性▕▏升级条件：250血液\n【等级2】3% 发帖堕落+1▕▏升级条件：10堕落\n【 Max 】3% 回帖堕落-1 血液+1、发帖旅程+1",
  "Frank (LBF)": "Frank (LBF)\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】270金币\n【等级1】3% 回帖血液+1▕▏升级条件：10主题数\n【等级2】6% 回帖血液+1▕▏升级条件：30追随\n【等级3】10% 回帖血液+1▕▏升级条件：90旅程\n【 Max 】10% 回帖血液+1 堕落-1、发帖旅程+1",
  "BIG BOSS": "BIG BOSS\n【勋章类型】游戏男从\n【入手条件】堕落＞20\n【商店售价】600金币\n【等级1】3% 回帖堕落+1、发帖堕落+1▕▏升级条件：消耗500血液\n【等级2】5% 回帖金币+1 堕落+1、发帖堕落+1▕▏升级条件：消耗1灵魂\n【等级3】10% 回帖金币+1 堕落+1、发帖堕落+1▕▏升级条件：金币≥1000\n【 Max 】20% 回帖金币+1 血液+1、发帖金币+3 血液+3",
  "詹米·多南": "詹米·多南\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】320金币\n【等级1】5% 回帖血液+1 堕落+1、发帖血液+2 堕落+1▕▏升级条件：消耗60咒术\n【 Max 】10% 回帖血液+1 堕落+1、发帖血液+2 堕落+1",
  "阿尔萨斯‧米奈希尔": "阿尔萨斯‧米奈希尔\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】350金币\n【等级1】7% 回帖堕落+1、发帖咒术+2▕▏升级条件：堕落≥50\n【 Max 】15% 回帖血液+1、发帖咒术+2",
  "克里斯·埃文斯": "克里斯·埃文斯\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】300金币\n【等级1】2% 回帖血液+1、发帖血液+1▕▏升级条件：追随≥50\n【等级2】3% 回帖血液+2、发帖血液+2▕▏升级条件：消耗300金币\n【 Max 】5% 回帖血液+2 旅程+1、发帖血液+2 旅程+1",
  "安德鲁·库珀": "安德鲁·库珀\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】400金币\n【等级1】2% 回帖金币+1 血液-1▕▏升级条件：消耗200血液\n【等级2】5% 回帖金币+2 血液-1▕▏升级条件：消耗300血液\n【 Max 】10% 回帖金币+3 血液-1、发帖金币+3 血液-1",
  "罗宾·西克": "罗宾·西克\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】300金币\n【等级1】5% 回帖堕落+1▕▏升级条件：堕落≥20\n【等级2】5% 回帖堕落+2▕▏升级条件：堕落≥50\n【等级3】10% 回帖堕落+2 血液-1▕▏升级条件：消耗200血液\n【 Max 】15% 回帖堕落-1 血液+2",
  "岛田半藏": "岛田半藏\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】500金币\n【等级1】2% 回帖血液+1▕▏升级条件：知识≥10\n【等级2】5% 回帖血液+1▕▏升级条件：旅程≥25\n【等级3】5% 回帖金币+1 血液+1▕▏升级条件：消耗100咒术\n【 Max 】10% 回帖金币+1 咒术+1",
  "泰凯斯·芬得利": "泰凯斯·芬得利\n【勋章类型】游戏男从\n【入手条件】追随>=10\n【商店售价】320金币\n【等级1】4% 回帖血液+1▕▏升级条件：血液≥100\n【等级2】6% 回帖血液+1、发帖金币+1▕▏升级条件：消耗150血液\n【等级3】8% 回帖血液+1、发帖金币+2▕▏升级条件：消耗200金币\n【 Max 】8% 回帖血液+2、发帖金币+2 知识+1",
  "【夏日限定】夏日的泰凯斯": "【夏日限定】夏日的泰凯斯\n【勋章类型】游戏男从\n【入手条件】堕落>=30\n【商店售价】666金币\n【等级1】5% 回帖堕落+1▕▏升级条件：堕落≥50\n【等级2】10% 回帖堕落+1 血液+1▕▏升级条件：消耗200血液\n【等级3】15% 回帖堕落+1 血液+2、发帖血液+3 堕落+1▕▏升级条件：消耗200金币\n【 Max 】20% 回帖金币-1 血液+3 堕落+1、发帖血液+5 堕落+1",
  "夏日的泰凯斯": "夏日的泰凯斯\n【入手条件】堕落>=30\n【商店售价】666金币\n【等级1】5% 回帖堕落+1▕▏升级条件：堕落≥50\n【等级2】10% 回帖堕落+1 血液+1▕▏升级条件：消耗200血液\n【等级3】15% 回帖堕落+1 血液+2、发帖血液+3 堕落+1▕▏升级条件：消耗200金币\n【 Max 】20% 回帖金币-1 血液+3 堕落+1、发帖血液+5 堕落+1",
  "康纳/Connor": "康纳/Connor\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】2% 回帖金币+1▕▏升级条件：消耗75金币\n【等级2】4% 回帖金币+1▕▏升级条件：消耗75血液\n【等级3】6% 回帖金币+1、发帖金币+1▕▏升级条件：消耗250金币\n【等级4】8% 回帖金币+1、发帖金币+1 血液+1▕▏升级条件：消耗250血液\n【等级5】8% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：追随≥50\n【 Max 】12% 回帖金币+2 血液+1、发帖金币+3 旅程+1",
  "亚瑟·库瑞（海王）": "亚瑟·库瑞（海王）\n【勋章类型】真人男从\n【入手条件】咒术≤30\n【商店售价】450金币\n【等级1】2% 回帖血液+1▕▏升级条件：旅程≥30\n【等级2】3% 回帖血液+1、发帖血液+1▕▏升级条件：追随≥100\n【等级3】5% 回帖血液+1、发帖血液+2▕▏升级条件：消耗200血液\n【等级4】8% 回帖血液+2、发帖血液+2▕▏升级条件：消耗50咒术\n【 Max 】12% 回帖血液+3、发帖血液+3 咒术+1",
  "亚瑟‧摩根": "亚瑟‧摩根\n【勋章类型】游戏男从\n【入手条件】堕落≥10\n【商店售价】500金币\n【等级1】4% 发帖金币+1▕▏升级条件：消耗200血液\n【等级2】4% 回帖金币+1、发帖金币+1▕▏升级条件：消耗300金币\n【等级3】6% 回帖金币+1、发帖金币+1▕▏升级条件：追随≥100\n【等级4】8% 回帖金币+1、发帖金币+2 血液+1▕▏升级条件：消耗-2旅程\n【等级5】12% 回帖金币+2、发帖金币+2 旅程+1▕▏升级条件：消耗-15堕落\n【 Max 】16% 回帖金币+3、发帖金币+3 旅程+1",
  "蛮族战士": "蛮族战士\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】50金币\n【等级1】勋章博物馆资料暂缺\n【 Max 】5% 回帖血液+1",
  "黑墙": "黑墙\n【勋章类型】游戏男从\n【入手条件】旅程≥15\n【商店售价】400金币\n【等级1】3% 回帖金币+1▕▏升级条件：追随≥20\n【等级2】6% 回帖金币+1 血液+1▕▏升级条件：消耗200金币\n【等级3】9% 回帖金币+1 血液+1▕▏升级条件：消耗200血液\n【等级4】13% 回帖血液+2、发帖金币+1 血液+3▕▏升级条件：堕落≥50\n【等级5】13% 回帖金币+1 血液+1、发帖金币+2 血液+2▕▏升级条件：堕落≥100\n【 Max 】13% 回帖金币+2、发帖金币+3 血液+1",
  "内森·德雷克": "内森·德雷克\n【勋章类型】游戏男从\n【入手条件】旅程≥10\n【商店售价】600金币\n【等级1】4% 回帖血液+1▕▏升级条件：消耗333血液\n【等级2】6% 回帖血液+1▕▏升级条件：消耗10堕落\n【等级3】8% 回帖血液+1▕▏升级条件：消耗333血液\n【等级4】10% 回帖血液+2▕▏升级条件：消耗333血液\n【等级5】20% 回帖血液+3、发帖旅程+1▕▏升级条件：消耗-299金币\n【 Max 】15% 回帖血液+3、发帖血液+2 旅程+1",
  "丹尼爾·紐曼": "丹尼爾·紐曼\n【勋章类型】真人男从\n【入手条件】主题数≥3\n【商店售价】350金币\n【等级1】5% 回帖血液+1▕▏升级条件：金币≥400\n【等级2】5% 回帖血液+1、发帖金币+1▕▏升级条件：血液≥400\n【等级3】8% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：追随≥100\n【 Max 】10% 回帖金币+1 血液+1、发帖金币+2 血液+2",
  "贝优妮塔": "贝优妮塔\n【勋章类型】女从\n【入手条件】咒术≥10\n【商店售价】280金币\n【等级1】5% 回帖堕落+1▕▏升级条件：堕落-10\n【等级2】5% 回帖堕落+1 血液+1▕▏升级条件：70追随\n【 Max 】5% 回帖堕落+1 血液+1 咒术+1",
  "英普瑞斯": "英普瑞斯\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】3% 回帖堕落-1▕▏升级条件：追随≥70\n【等级2】7% 回帖堕落-1 血液+1、发帖血液+1▕▏升级条件：消耗300金币\n【等级3】10% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：血液≥700\n【 Max 】17% 回帖金币+1 血液+1、发帖金币+2 血液+2",
  "杰西·麦克雷": "杰西·麦克雷\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】500金币\n【等级1】5% 回帖金币+1、发帖堕落+1▕▏升级条件：发帖数≥100\n【等级2】8% 回帖金币+1、发帖堕落+1▕▏升级条件：主题数≥10\n【等级3】10% 回帖金币+1 堕落+1、发帖堕落+1▕▏升级条件：堕落≥40\n【等级4】15% 回帖金币+1 堕落+1、发帖旅程+1▕▏升级条件：消耗400血液\n【 Max 】15% 回帖金币+2 堕落+1、发帖旅程+1 堕落+1",
  "卡德加（Khadgar）": "卡德加（Khadgar）\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】350金币\n【等级1】3% 发帖堕落-1▕▏升级条件：咒术≥40\n【等级2】6% 回帖堕落-1、发帖知识+1 堕落-1▕▏升级条件：消耗80咒术\n【等级3】9% 回帖血液+1 堕落-1、发帖知识+1 堕落-1▕▏升级条件：消耗15堕落\n【 Max 】12% 回帖血液+2 堕落-1、发帖知识+1 堕落-1",
  "麦迪文（Medivh）": "麦迪文（Medivh）\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】350金币\n【等级1】4% 发帖咒术+1▕▏升级条件: 消耗66咒术\n【等级2】6% 回帖咒术+1、发帖咒术+1▕▏升级条件: 堕落≥50\n【等级3】8% 回帖咒术+1、发帖咒术+1 堕落+1▕▏升级条件: 堕落≥99\n【 Max 】12% 回帖咒术+1、发帖咒术+2 堕落+1",
  "遗忘之水": "遗忘之水\n【勋章类型】赠礼\n【入手条件】无\n【商店售价】180金币\n【等级1】无属性▕▏升级条件：消耗10堕落\n【等级2】无属性▕▏升级条件：消耗10堕落\n【等级3】无属性▕▏升级条件：消耗10堕落\n【 Max 】无属性",
  "千杯不醉": "千杯不醉\n【勋章类型】赠礼\n【入手条件】无\n【商店售价】12金币\n【 Max 】5% 回帖血液+1 堕落+1、发帖血液+1 堕落+1",
  "送情书": "送情书\n【勋章类型】赠礼（只可赠送）\n【入手条件】无\n【商店售价】15金币\n【时效】5天\n【 Max 】10% 回帖咒术+1、发帖咒术+1",
  "丢肥皂": "丢肥皂\n【勋章类型】赠礼（只可赠送）\n【入手条件】无\n【商店售价】10金币\n【时效】5天\n【 Max 】10% 回帖堕落+1、发帖堕落+1",
  "灵光补脑剂": "灵光补脑剂\n【勋章类型】赠礼（只可赠送）\n【入手条件】无\n【商店售价】22金币\n【时效】3天\n【等级1】升级条件：消耗-1知识\n【 Max 】2% 回帖血液-1",
  "萨赫的蛋糕": "萨赫的蛋糕\n【勋章类型】赠礼（只可赠送）\n【入手条件】知识≥5\n【商店售价】40金币\n【时效】14天\n【 Max 】10% 回帖血液+2、发帖血液+2",
  "神秘商店贵宾卡": "神秘商店贵宾卡\n【勋章类型】赠礼（只可赠送）\n【入手条件】知识≥10\n【商店售价】50金币\n【时效】7天\n【等级1】10% 回帖金币+2▕▏升级条件：咒术≥20\n【等级2】20% 回帖金币+2、发帖旅程+1▕▏升级条件：咒术≥60\n【 Max 】30% 回帖金币+3、发帖旅程+1",
  "变骚喷雾": "变骚喷雾\n【勋章类型】赠礼（只可赠送）\n【入手条件】无\n【商店售价】13金币\n【时效】5天\n【 Max 】5% 回帖堕落+1 金币+1、发帖堕落+1 金币+1",
  "贞洁内裤": "贞洁内裤\n【勋章类型】赠礼（只可赠送）（已下架）\n【入手条件】无\n【商店售价】110金币\n【 Max 】13% 回帖堕落-1 血液+3",
  "没有梦想的咸鱼": "没有梦想的咸鱼\n【勋章类型】赠礼（只可赠送）\n【入手条件】无\n【商店售价】1金币\n【时效】1天\n【 Max 】1% 回帖金币+1、回帖血液+1",
  "石肤术": "石肤术\n【勋章类型】咒术\n【入手条件】血液≥15\n【商店售价】4咒术\n【时效】3天\n【等级1】10% 回帖血液+1、发帖血液+1▕▏升级条件：血液≥200\n【等级2】20% 回帖血液+1、发帖血液+1▕▏升级条件：血液≥400\n【 Max 】40% 回帖血液+1、发帖血液+1",
  "吞食魂魄": "吞食魂魄\n【勋章类型】咒术\n【入手条件】知识≥35\n【商店售价】1灵魂\n【时效】7天\n【等级1】1% 无属性▕▏升级条件：消耗-1001血液\n【等级2】10% 无属性▕▏升级条件：堕落≥10\n【 Max 】10% 回帖血液-1、发帖血液-1",
  "咆哮诅咒": "咆哮诅咒\n【勋章类型】咒术\n【入手条件】知识≥28\n【商店售价】8咒术\n【时效】3天\n【等级1】30% 回帖堕落+3 血液-1▕▏升级条件：积分≥35（手动升级，但是无消耗）\n【 Max 】无属性",
  "霍格沃茨五日游": "霍格沃茨五日游\n【勋章类型】咒术\n【入手条件】旅程≥10\n【商店售价】8咒术\n【时效】5天\n【等级1】5% 回帖咒术+2▕▏升级条件：旅程≥10\n【等级2】20% 发帖知识+1▕▏升级条件：知识≥20\n【等级3】20% 回帖血液+3、发帖知识+1 血液+3▕▏升级条件：追随≥100\n【等级4】20% 回帖血液+5、发帖旅程+1 血液+5▕▏升级条件：堕落≥50\n【 Max 】20% 回帖血液+3、发帖堕落+1 血液+3",
  "祈祷术": "祈祷术\n【勋章类型】咒术\n【入手条件】无\n【商店售价】8咒术\n【时效】3天\n【等级1】15% 回帖堕落-1、发帖堕落-1▕▏升级条件：知识≥5\n【等级2】15% 回帖堕落-1、发帖堕落-1▕▏升级条件：消耗1堕落\n【等级3】12% 回帖堕落-1、发帖堕落-1▕▏升级条件：消耗1堕落\n【等级4】10% 回帖堕落-1、发帖堕落-1▕▏升级条件：消耗1堕落\n【等级5】8% 回帖堕落-1、发帖堕落-1▕▏升级条件：消耗1堕落\n【等级6】6% 回帖堕落-1、发帖堕落-1▕▏升级条件：消耗1堕落\n【 Max 】4% 回帖堕落-1、发帖堕落-1",
  "黑暗交易": "黑暗交易\n【勋章类型】咒术\n【入手条件】堕落＞10 知识≥10\n【商店售价】8咒术\n【时效】3天\n【等级1】勋章博物馆数据暂缺\n【等级2】25% 回帖金币+2 血液-1、发帖金币+2 血液-1▕▏升级条件：堕落≥25\n【等级3】勋章博物馆数据暂缺\n【等级4】25% 回帖金币+4 血液-2、发帖金币+4 血液-2▕▏升级条件：堕落≥50\n【等级5】0%▕▏升级条件：血液≥3\n【 Max 】25% 回帖金币+6 血液-3、发帖金币+6 血液-3",
  "炼金之心": "炼金之心\n【勋章类型】咒术\n【入手条件】无\n【商店售价】4咒术\n【时效】3天\n【等级1】10% 回帖金币+2▕▏升级条件：知识≥3\n【等级2】15% 回帖金币+2▕▏升级条件：知识≥15\n【等级3】20% 回帖金币+2▕▏升级条件：知识≥30\n【 Max 】30% 回帖金币+2",
  "水泡术": "水泡术\n【勋章类型】咒术\n【入手条件】知识≥3\n【商店售价】3咒术\n【时效】5天\n【等级1】无属性▕▏升级条件：知识≥6\n【等级2】无属性▕▏升级条件：消耗-10血液\n【 Max 】1% 回帖血液+1",
  "召唤古代战士": "召唤古代战士\n【勋章类型】咒术\n【入手条件】知识≥6\n【商店售价】8咒术\n【时效】4天\n【等级1】10% 回帖血液+3、发帖血液+3▕▏升级条件：知识≥25\n【 Max 】30% 回帖血液+3、发帖血液+3",
  "念念往日士官盔": "念念往日士官盔\n【勋章类型】装备\n【入手条件】无\n【商店售价】125金币\n【 Max 】30% 回帖金币-1 血液+1",
  "药剂背袋": "药剂背袋\n【勋章类型】装备\n【入手条件】无\n【商店售价】180金币\n【 Max 】8% 回帖血液+1、发帖血液+1",
  "刺杀者匕首": "刺杀者匕首\n【勋章类型】装备\n【入手条件】堕落＞10\n【商店售价】80金币\n【 Max 】5% 回帖金币+3",
  "净化手杖": "净化手杖\n【勋章类型】装备\n【入手条件】堕落＜50\n【商店售价】400金币\n【等级1】10% 发帖堕落-1▕▏升级条件：在线时间≥800\n【 Max 】20% 回帖堕落-1、发帖堕落-1 咒术+1",
  "符文披风": "符文披风\n【勋章类型】装备\n【入手条件】无\n【商店售价】280金币\n【 Max 】8% 回帖咒术+1、发帖咒术+1",
  "嗜血斩首斧": "嗜血斩首斧\n【勋章类型】装备\n【入手条件】无\n【商店售价】100金币\n【 Max 】5% 回帖堕落+1",
  "圣英灵秘银甲": "圣英灵秘银甲\n【勋章类型】装备\n【入手条件】无\n【商店售价】1350金币\n【等级1】11% 回帖血液+2 堕落-1、发帖血液+2 堕落-1▕▏升级条件：消耗1灵魂\n【等级2】31% 回帖血液+2 堕落-1、发帖血液+2 堕落-1▕▏升级条件：咒术≥111\n【 Max 】41% 回帖血液+2 堕落-1、发帖血液+2 堕落-1 旅程+1",
  "重磅手环": "重磅手环\n【勋章类型】装备\n【入手条件】无\n【商店售价】250金币\n【 Max 】20% 发帖血液-5 旅程+1",
  "新月护符": "新月护符\n【勋章类型】装备\n【入手条件】血液≥50\n【商店售价】200金币\n【等级1】5% 回帖堕落-1、发帖堕落-1▕▏升级条件：旅程≥90\n【 Max 】10% 回帖堕落-2、发帖堕落-2 旅程+1 ",
  "布衣": "布衣\n【勋章类型】装备\n【入手条件】无\n【商店售价】45金币\n【等级1】1% 回帖血液-1▕▏升级条件：消耗2金币\n【等级2】无属性▕▏升级条件：消耗4金币\n【等级3】1% 回帖血液+1▕▏升级条件：消耗16金币\n【等级4】1% 回帖血液+1▕▏升级条件：消耗18金币\n【等级5】1% 回帖血液+1▕▏升级条件：消耗20金币\n【等级6】1% 回帖血液+1▕▏升级条件：消耗22金币\n【等级7】2% 回帖血液+1▕▏升级条件：消耗34金币\n【等级8】2% 回帖血液+1、发帖血液+1▕▏升级条件：消耗36金币\n【等级9】2% 回帖血液+2、发帖血液+2▕▏升级条件：消耗38金币\n【等级10】2% 回帖血液+2、发帖血液+3▕▏升级条件：消耗50金币\n【等级11】2% 回帖血液+2 咒术+1、发帖血液+2 咒术+2▕▏升级条件：消耗52金币\n【等级12】2% 回帖血液+2 咒术+1、发帖血液+2 咒术+3▕▏升级条件：消耗64金币\n【等级13】2% 回帖血液+3 知识+1、发帖血液+3 知识+1▕▏升级条件：消耗250血液\n【 Max 】3% 回帖血液+5 咒术-1、发帖血液+5 咒术-1",
  "骑士遗盔": "骑士遗盔\n【勋章类型】装备\n【入手条件】无\n【商店售价】275金币\n【等级1】1% 回帖咒术+1▕▏升级条件：消耗25金币\n【等级2】2% 回帖咒术+1、发帖知识+1▕▏升级条件：消耗50金币\n【等级3】3% 回帖咒术+1、发帖知识+1▕▏升级条件：消耗100金币\n【 Max 】5% 回帖咒术+1 血液+1、发帖知识+1 ",
  "艾尔尤因": "艾尔尤因\n【勋章类型】装备\n【入手条件】无\n【商店售价】290金币\n【等级1】5% 回帖血液+1、发帖血液+1▕▏升级条件：血液≥300\n【等级2】5% 回帖咒术+1、发帖咒术+1▕▏升级条件：血液≥500\n【 Max 】10% 回帖血液+1 堕落-1、发帖血液+1 堕落-1",
  "变形软泥": "变形软泥\n【勋章类型】装备\n【入手条件】无\n【商店售价】66金币\n【等级1】无属性▕▏升级条件：堕落≥13\n【等级2】3% 回帖堕落+2▕▏升级条件：消耗166金币\n【 Max 】6% 回帖堕落+2、发帖咒术+1",
  "神圣十字章": "神圣十字章\n【勋章类型】装备\n【入手条件】旅程≥12\n【商店售价】300金币\n【等级1】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗8咒术\n【等级2】10% 回帖血液+1、发帖血液+1▕▏升级条件：消耗-20血液\n【 Max 】5% 回帖血液+2、发帖血液+2",
  "重新充能的神圣十字章": "重新充能的神圣十字章\n【勋章类型】装备\n【入手条件】未知\n【商店售价】300金币\n【 Max 】10% 回帖血液+2，发帖血液+1、堕落-1",
  "超级名贵无用宝剑": "超级名贵无用宝剑\n【勋章类型】装备\n【入手条件】无\n【商店售价】1299金币\n【 Max 】无属性",
  "十字叶章": "十字叶章\n【勋章类型】装备\n【入手条件】无\n【商店售价】277金币\n【等级1】5% 回帖堕落-1▕▏升级条件：追随≥45\n【等级2】5% 回帖堕落-1 咒术+1、发帖咒术+1▕▏升级条件：消耗80血液\n【 Max 】1% 回帖堕落-5 咒术+5、发帖灵魂+1",
  "守望者徽章": "守望者徽章\n【勋章类型】装备\n【入手条件】血液≥200\n【商店售价】100金币\n【等级1】2% 回帖血液+1▕▏升级条件：消耗50血液\n【等级2】4% 回帖血液+1▕▏升级条件：消耗50血液\n【等级3】6% 回帖血液+2▕▏升级条件：堕落≥30\n【 Max 】8% 回帖血液+2 堕落-1",
  "山猫图腾": "山猫图腾\n【勋章类型】装备\n【入手条件】无\n【商店售价】88金币\n【 Max 】2% 回帖金币+1、发帖金币+1",
  "眼镜蛇图腾": "眼镜蛇图腾\n【勋章类型】装备\n【入手条件】无\n【商店售价】88金币\n【 Max 】1% 回帖咒术+1、发帖咒术+1",
  "猎鹰图腾": "猎鹰图腾\n【勋章类型】装备\n【入手条件】无\n【商店售价】88金币\n【 Max 】2% 回帖血液+1、发帖血液+1",
  "石鬼面": "石鬼面\n【勋章类型】装备\n【入手条件】血液≥30\n【商店售价】233金币\n【等级1】9% 回帖咒术+1 血液-1▕▏升级条件：血液≥400\n【 Max 】18% 回帖咒术+1 血液-3 堕落+1、发帖咒术+1",
  "锻造卷轴": "锻造卷轴\n【勋章类型】资产\n【入手条件】无\n【商店售价】99金币\n【 Max 】1% 回帖金币+1 知识+1、发帖金币+1 知识+1",
  "漂洋小船": "漂洋小船\n【勋章类型】资产\n【入手条件】无\n【商店售价】75金币\n【 Max 】2% 回帖旅程+1",
  "知识大典": "知识大典\n【勋章类型】资产\n【入手条件】无\n【商店售价】50金币\n【 Max 】1% 回帖知识+1",
  "预知水晶球": "预知水晶球\n【勋章类型】资产\n【入手条件】咒术≥10\n【商店售价】150金币\n【等级1】4% 回帖金币+1、发帖知识+1▕▏升级条件：消耗40咒术\n【 Max 】8% 回帖金币+1、发帖知识+1 旅程+1",
  "种植小草": "种植小草\n【勋章类型】资产\n【入手条件】无\n【商店售价】15金币\n【等级1】无属性▕▏升级条件：追随≥5\n【等级2】无属性▕▏升级条件：追随≥10\n【等级3】无属性▕▏升级条件：追随≥15\n【等级4】无属性▕▏升级条件：消耗-1血液\n【 Max 】1% 回帖血液+1",
  "聚魔花盆": "聚魔花盆\n【勋章类型】资产\n【入手条件】无\n【商店售价】500金币\n【等级1】10% 发帖咒术+3▕▏升级条件：知识≥5\n【等级2】20% 发帖咒术+3▕▏升级条件：知识≥10\n【等级3】30% 发帖咒术+3▕▏升级条件：知识≥15\n【等级4】40% 发帖咒术+3▕▏升级条件：知识≥30\n【等级5】50% 发帖咒术+3▕▏升级条件：知识≥50\n【等级6】60% 发帖咒术+3▕▏升级条件：堕落≥30\n【 Max 】30% 发帖咒术+3",
  "夜灯": "夜灯\n【勋章类型】资产\n【入手条件】无\n【商店售价】40金币\n【等级1】1% 回帖血液+1▕▏升级条件：消耗15金币\n【等级2】2% 回帖血液+1▕▏升级条件：消耗30金币\n【等级3】3% 回帖血液+1▕▏升级条件：消耗45金币\n【等级4】4% 回帖血液+1▕▏升级条件：消耗60金币\n【等级5】5% 回帖血液+1▕▏升级条件：消耗75金币\n【等级6】6% 回帖血液+1▕▏升级条件：消耗90金币\n【等级7】7% 回帖血液+1▕▏升级条件：消耗-6旅程\n【 Max 】7% 回帖血液+1",
  "种植菊花": "种植菊花\n【勋章类型】资产\n【入手条件】无\n【商店售价】110金币\n【等级1】1% 回帖堕落+1▕▏升级条件：消耗20血液\n【等级2】3% 回帖堕落+1▕▏升级条件：消耗50血液\n【等级3】5% 回帖血液+1 堕落+1▕▏升级条件：消耗-30金币\n【 Max 】3% 回帖金币+1 堕落+1、发帖-1血液",
  "发芽的种子": "发芽的种子\n【勋章类型】资产\n【入手条件】在线时间＞10小时\n【商店售价】77金币\n【等级1】无属性▕▏升级条件：消耗50血液\n【等级2】无属性▕▏升级条件：消耗50血液\n【等级3】无属性▕▏升级条件：消耗50血液\n【 Max 】7% 回帖金币+1、发帖金币+2",
  "奇怪的紫水晶": "奇怪的紫水晶\n【勋章类型】资产\n【入手条件】无\n【商店售价】299金币\n【等级1】5% 回帖血液-1 咒术+2 堕落+1▕▏升级条件：消耗35咒术\n【 Max 】1% 回帖知识+1 咒术+1、发帖灵魂+1",
  "婴儿泪之瓶": "婴儿泪之瓶\n【勋章类型】资产\n【入手条件】无\n【商店售价】200金币\n【等级1】2% 回帖堕落-1、发帖堕落-1▕▏升级条件：消耗50金币\n【等级2】2% 回帖堕落-1 知识+1、发帖堕落-1 知识+1▕▏升级条件：消耗100金币\n【 Max 】3% 回帖堕落-1 知识+1、发帖堕落-1 知识+1",
  "雪王的心脏": "雪王的心脏\n【勋章类型】资产\n【入手条件】旅程≥12\n【商店售价】180金币\n【等级1】2% 回帖旅程+1▕▏升级条件：消耗30咒术\n【等级2】2% 回帖知识+1 旅程+1、发帖知识+1 旅程+1▕▏升级条件：消耗50咒术\n【 Max 】3% 回帖知识+1 旅程+1、发帖知识+1 旅程+1",
  "这是一片丛林": "这是一片丛林\n【勋章类型】资产\n【入手条件】无\n【商店售价】120金币\n【等级1】无属性▕▏升级条件：堕落≥35\n【等级2】3% 回帖咒术+1▕▏升级条件：堕落≥70\n【 Max 】3% 回帖旅程+1",
  "暗红矿土": "暗红矿土\n【勋章类型】资产\n【入手条件】堕落≥20\n【商店售价】40金币\n【等级1】1% 回帖血液+1▕▏升级条件：消耗30金币\n【等级2】4% 回帖堕落+1▕▏升级条件：消耗70金币\n【等级3】1% 回帖血液+2▕▏升级条件：消耗200金币\n【 Max 】12% 回帖血液+1 堕落+1、发帖血液+2 咒术+2",
  "充满魔力的种子": "充满魔力的种子\n【勋章类型】资产\n【入手条件】旅程≥20 知识≥4\n【商店售价】250金币\n【等级1】无属性▕▏升级条件：消耗20咒术\n【等级2】5% 回帖知识+1▕▏升级条件：在线时间≥240\n【等级3】5% 回帖知识+1、发帖知识+2▕▏升级条件：消耗20咒术\n【等级4】无属性▕▏升级条件：消耗10咒术\n【 Max 】8% 回帖金币+2 咒术+1、发帖知识+2",
  "史莱姆养殖证书": "史莱姆养殖证书\n【勋章类型】资产\n【入手条件】无\n【商店售价】60金币\n【等级1】1% 回帖金币+1▕▏升级条件：消耗100金币\n【等级2】2% 回帖金币+1▕▏升级条件：消耗100金币\n【等级3】3% 回帖金币+1▕▏升级条件：消耗100金币\n【等级4】4% 回帖金币+1▕▏升级条件：消耗100金币\n【等级5】5% 回帖金币+1▕▏升级条件：消耗100金币\n【等级6】8% 回帖金币+1▕▏升级条件：知识≥8\n【等级7】8% 回帖金币+1▕▏升级条件：消耗100金币\n【等级8】12% 回帖金币+1▕▏升级条件：知识≥12\n【等级9】12% 回帖金币+1▕▏升级条件：消耗100金币\n【等级10】\n【等级11】15% 回帖金币+1▕▏升级条件：消耗100金币\n【等级12】\n【等级13】18% 回帖金币+1▕▏升级条件：消耗100金币\n【等级14】\n【等级15】21% 回帖金币+1▕▏升级条件：消耗100金币\n【等级16】25% 回帖金币+1▕▏升级条件：知识≥30\n【等级17】25% 回帖金币+1▕▏升级条件：消耗100金币\n【等级18】\n【等级19】28% 回帖金币+1▕▏升级条件：消耗100金币\n【等级20】\n【等级21】31% 回帖金币+1▕▏升级条件：消耗100金币\n【等级22】\n【等级23】35% 回帖金币+1▕▏升级条件：消耗100金币\n【等级24】\n【等级25】38% 回帖金币+1▕▏升级条件：消耗100金币\n【等级26】\n【等级27】42% 回帖金币+1▕▏升级条件：消耗100金币\n【等级28】\n【等级29】45% 回帖金币+1▕▏升级条件：消耗100金币\n【等级30】48% 回帖金币+1▕▏升级条件：消耗20知识\n【等级31】\n【等级32】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥160\n【等级33】25% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥170\n【等级34】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥320\n【等级35】70% 回帖金币+1 咒术+1、发帖金币+1▕▏升级条件：血液≥322\n【等级36】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥460\n【等级37】25% 回帖血液-1▕▏升级条件：血液≥470\n【等级38】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥580\n【等级39】100% 回帖金币+1 血液+1、发帖金币+1▕▏升级条件：血液≥620\n【等级40】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥700\n【等级41】15% 回帖咒术+1 金币+1、发帖金币+1▕▏升级条件：血液≥720\n【等级42】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥820\n【等级43】100% 回帖金币+1 血液+1、发帖金币+1▕▏升级条件：血液≥850\n【等级44】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥990\n【等级45】25% 回帖血液-1、咒术+1▕▏升级条件：血液≥1000\n【等级46】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥1130\n【等级47】35% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥1150\n【等级48】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥1280\n【等级49】100% 回帖金币+1 血液+1、发帖金币+1▕▏升级条件：血液≥1340\n【等级50】70% 回帖金币+1 咒术+1、发帖金币+1▕▏升级条件：血液≥1342\n【等级51】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥1470\n【等级52】10% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥1500\n【等级53】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥1610\n【等级54】100% 回帖金币+1 血液+1、发帖金币+1▕▏升级条件：血液≥1650\n【等级55】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥1780\n【等级56】25% 回帖血液-1▕▏升级条件：血液≥1790\n【等级57】70% 回帖金币+1 咒术+1、发帖金币+1▕▏升级条件：血液≥1792\n【等级58】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥1880\n【等级59】45% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥1900\n【等级60】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥1920\n【等级61】100% 回帖金币+1 血液+1、发帖金币+1▕▏升级条件：血液≥1950\n【等级62】70% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥2000\n【 Max 】无属性",
  "种植菠菜": "种植菠菜\n【勋章类型】资产\n【入手条件】主题数≥5\n【商店售价】30金币\n【等级1】无属性▕▏升级条件：知识≥1\n【等级2】无属性▕▏升级条件：消耗10金币\n【等级3】无属性▕▏升级条件：知识≥2\n【等级4】无属性▕▏升级条件：消耗10金币\n【等级5】无属性▕▏升级条件：知识≥3\n【等级6】无属性▕▏升级条件：消耗15金币\n【等级7】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗-35血液\n【 Max 】无属性",
  "勇者与龙之书": "勇者与龙之书\n【勋章类型】资产\n【入手条件】无\n【商店售价】300金币\n【等级1】无属性▕▏升级条件：消耗1旅程\n【等级2】无属性▕▏升级条件：消耗1旅程\n【等级3】无属性▕▏升级条件：消耗1旅程\n【等级4】无属性▕▏升级条件：消耗1旅程\n【等级5】无属性▕▏升级条件：消耗1旅程\n【等级6】无属性▕▏升级条件：消耗1旅程\n【等级7】无属性▕▏升级条件：消耗1旅程\n【等级8】无属性▕▏升级条件：消耗1旅程\n【等级9】无属性▕▏升级条件：消耗1旅程\n【等级10】无属性▕▏升级条件：消耗-9旅程\n【 Max 】3% 回帖旅程+1",
  "微笑的面具": "微笑的面具\n【勋章类型】资产\n【入手条件】无\n【商店售价】100金币\n【等级1】无属性▕▏升级条件：主题数≥10\n【等级2】无属性▕▏升级条件：主题数≥20\n【等级3】无属性▕▏升级条件：主题数≥30\n【等级4】无属性▕▏升级条件：主题数≥40\n【等级5】无属性▕▏升级条件：主题数≥50\n【 Max 】20% 回帖堕落+1、发帖堕落+1",
  "浪潮之歌": "浪潮之歌\n【勋章类型】资产\n【入手条件】旅程≥70\n【商店售价】300金币\n【等级1】无属性▕▏升级条件：消耗1旅程\n【等级2】无属性▕▏升级条件：消耗1旅程\n【等级3】无属性▕▏升级条件：消耗1旅程\n【等级4】无属性▕▏升级条件：消耗1旅程\n【等级5】无属性▕▏升级条件：消耗1旅程\n【等级6】无属性▕▏升级条件：消耗1旅程\n【等级7】无属性▕▏升级条件：消耗1旅程\n【等级8】无属性▕▏升级条件：消耗1旅程\n【等级9】无属性▕▏升级条件：消耗1旅程\n【等级10】无属性▕▏升级条件：消耗-9旅程\n【 Max 】3% 回帖金币+1 旅程+1、发帖金币+1 旅程+1",
  "章鱼小丸子": "章鱼小丸子\n【勋章类型】资产\n【入手条件】无（该勋章不可寄售，仅可回收）\n【商店售价】150金币\n【等级1】10% 回帖血液+1▕▏升级条件：消耗-10血液\n【等级2】7% 回帖血液+1▕▏升级条件：消耗-20血液\n【等级3】5% 回帖血液+1▕▏升级条件：消耗-20血液\n【等级4】3% 回帖血液+2▕▏升级条件：消耗-30血液\n【 Max 】无属性",
  "流失之椅": "流失之椅\n【勋章类型】资产\n【入手条件】无\n【商店售价】320金币\n【等级1】40% 回帖血液+1、发帖旅程+1▕▏升级条件：在线时间≥80\n【等级2】32% 回帖血液+1、发帖旅程+1▕▏升级条件：在线时间≥150\n【等级3】24% 回帖血液+1、发帖旅程+1▕▏升级条件：在线时间≥300\n【等级4】17% 回帖血液+1、发帖旅程+1▕▏升级条件：在线时间≥500\n【等级5】9% 回帖血液+1、发帖旅程+1▕▏升级条件：在线时间≥800\n【等级6】3% 回帖血液+1、发帖旅程+1▕▏升级条件：在线时间≥1000\n【等级7】无属性▕▏升级条件：消耗1灵魂\n【 Max 】50% 回帖血液+1、发帖旅程+1",
  "金钱马车": "金钱马车\n【勋章类型】资产\n【入手条件】旅程≥10\n【商店售价】200金币\n【等级1】无属性▕▏升级≥1金币\n【等级2】23% 回帖金币+1▕▏升级≥50金币\n【等级3】15% 回帖金币+2▕▏升级≥100金币\n【等级4】26% 回帖金币+2▕▏升级≥187金币\n【等级5】55% 回帖咒术+1▕▏升级≥189金币\n【等级6】40% 回帖金币+3▕▏升级≥250金币\n【等级7】7% 回帖金币+1▕▏升级≥345金币\n【等级8】60% 回帖金币+10▕▏升级≥346金币\n【等级9】10% 回帖血液-1▕▏升级≥360金币\n【等级10】15% 回帖金币+2▕▏升级≥501金币\n【等级11】18% 回帖知识+1▕▏升级≥502金币\n【等级12】7% 回帖金币+1▕▏升级≥617金币\n【等级13】26% 回帖金币+2▕▏升级≥749金币\n【等级14】2% 发帖灵魂+1▕▏升级≥750金币\n【等级15】30% 回帖金币+3 血液+1▕▏升级≥823金币\n【等级16】23% 回帖金币+1▕▏升级≥978金币\n【等级17】无属性▕▏升级≥1000金币\n【等级18】15% 回帖金币+2▕▏升级≥1092金币\n【等级19】26% 回帖金币+2▕▏升级≥1202金币\n【等级20】55% 回帖咒术+1▕▏升级≥1204金币\n【等级21】40% 回帖金币+3▕▏升级≥1250金币\n【等级22】7% 回帖金币+1▕▏升级≥1334金币\n【等级23】60% 回帖金币+10▕▏升级≥1335金币\n【等级24】23% 回帖金币+1▕▏升级≥1500金币\n【等级25】15% 回帖金币+2▕▏升级≥1666金币\n【等级26】2% 发帖灵魂+1▕▏升级≥1667金币\n【等级27】无属性▕▏升级≥1699金币\n【等级28】23% 回帖金币+1▕▏升级≥1750金币\n【等级29】10% 回帖血液-1▕▏升级≥1763金币\n【等级30】30% 回帖金币+3 血液+1▕▏升级≥1811金币\n【等级31】7% 回帖金币+1▕▏升级≥1888金币\n【等级32】18% 回帖知识+1▕▏升级≥1889金币\n【等级33】26% 回帖金币+2▕▏升级≥2000金币\n【等级34】23% 回帖金币+1▕▏升级≥2073金币\n【等级35】无属性▕▏升级≥2101金币\n【等级36】27% 回帖金币+3▕▏升级≥2250金币\n【等级37】15% 回帖金币+2▕▏升级≥2348金币\n【等级38】60% 回帖金币+10▕▏升级≥2349金币\n【等级39】40% 回帖金币+3▕▏升级≥2401金币\n【等级40】7% 回帖金币+1▕▏升级≥2500金币\n【等级41】18% 回帖知识+1▕▏升级≥2501金币\n【等级42】26% 回帖金币+2▕▏升级≥2615金币\n【等级43】23% 回帖金币+1▕▏升级≥2748金币\n【等级44】55% 回帖咒术+1▕▏升级≥2750金币\n【等级45】10% 回帖血液-1▕▏升级≥2866金币\n【等级46】7% 回帖金币+1▕▏升级≥2903金币\n【等级47】2% 发帖灵魂+1▕▏升级≥2904金币\n【等级48】15% 回帖金币+2▕▏升级≥2962金币\n【等级49】30% 回帖金币+3 血液+1▕▏升级≥3000金币\n【 Max 】无属性",
  "木柴堆": "木柴堆\n【勋章类型】资产\n【入手条件】旅程≥18\n【商店售价】90金币\n【等级1】1% 回帖金币+1▕▏升级条件：消耗5血液\n【等级2】5% 回帖金币+1▕▏升级条件：消耗150金币\n【等级3】1% 无属性▕▏升级条件：消耗30血液\n【 Max 】10% 回帖金币+1、发帖金币+3 旅程+1",
  "神秘的邀请函": "神秘的邀请函\n【勋章类型】资产\n【入手条件】主题≥3 在线时间＞12小时\n【商店售价】100金币\n【等级1】1% 回帖金币+1、发帖金币+1▕▏升级条件：消耗80血液\n【等级2】1% 回帖血液+1、发帖血液+1▕▏升级条件：消耗80金币\n【等级3】5% 回帖血液-1▕▏升级条件：追随≥30\n【等级4】5% 回帖金币+3 血液-1、发帖金币+3 血液-1▕▏升级条件：知识≥10\n【等级5】10% 回帖血液+1 金币+1、发帖血液+2 金币+3▕▏升级条件：消耗-100金币\n【 Max 】无属性",
  "诺曼底号": "诺曼底号\n【勋章类型】资产\n【入手条件】无\n【商店售价】8700金币\n【等级1】无属性▕▏升级条件：金币≥100\n【等级2】50% 发帖血液+7▕▏升级条件：金币≥1000\n【等级3】50% 发帖血液+7 旅程+1▕▏升级条件：金币≥5000\n【等级4】50% 发帖血液+7 旅程+1 咒术+3▕▏升级条件：金币≥10000\n【等级5】50% 发帖血液+7 旅程+1 咒术+3 知识+1▕▏升级条件：金币≥20000\n【 Max 】无属性",
  "德拉克魂匣": "德拉克魂匣\n【勋章类型】资产\n【入手条件】知识≥10\n【商店售价】350血液\n【等级1】5% 回帖血液+1、发帖血液+1▕▏升级条件：知识≥60（猜测）\n【等级2】8% 回帖血液+1、发帖血液+2▕▏升级条件：知识≥120\n【等级3】14% 回帖血液+1、发帖血液+2 咒术+1  ▕▏升级条件：知识≥180\n【 Max 】20% 回帖血液+2、发帖血液+3 咒术+1",
  "圣甲虫秘典": "圣甲虫秘典\n【勋章类型】资产\n【入手条件】知识≥10\n【商店售价】350金币\n【等级1】5% 回帖金币+1、发帖金币+1▕▏升级条件：消耗50咒术\n【等级2】10% 回帖金币+1、发帖金币+2▕▏升级条件：知识≥100\n【等级3】15% 回帖金币+2、发帖金币+3▕▏升级条件：知识≥200\n【 Max 】20% 回帖金币+3、发帖金币+4",
  "生化危机：复仇": "生化危机：复仇\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】1% 回帖金币+1 血液+1 ",
  "五花八门版块": "五花八门版块\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖咒术+1",
  "奥兹大陆": "奥兹大陆\n【勋章类型】板块\n【入手条件】旅程＞25 在线时间＞255小时\n【商店售价】100金币\n【 Max 】2% 回帖咒术+1、发帖咒术+1",
  "龙腾世纪：审判": "龙腾世纪：审判\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】1% 回帖旅程+1",
  "质量效应三部曲": "质量效应三部曲\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖血液+2",
  "辐射:新维加斯": "辐射:新维加斯\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖堕落+1",
  "TRPG纪念章": "TRPG纪念章\n【勋章类型】奖品\n【入手条件】完成TRPG版块内的跑团任务\n【商店售价】无\n【等级1】无属性▕▏升级条件:-100金币\n【 Max 】2% 回帖金币+1 血液+1",
  "上古卷轴V:天际": "上古卷轴V:天际\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖金币+2",
  "堕落飨宴": "堕落飨宴\n【勋章类型】板块\n【入手条件】旅程≥20 在线时间≥100小时 发帖数≥100 主题≥10\n【商店售价】9999金币\n【等级1】无属性▕▏升级条件：堕落-10\n【等级2】无属性▕▏升级条件：血液-500\n【等级3】无属性▕▏升级条件：堕落-20\n【等级4】无属性▕▏升级条件：咒术-200\n【等级5】无属性▕▏升级条件：堕落-30\n【等级6】无属性▕▏升级条件：旅程-30\n【等级7】无属性▕▏升级条件：堕落-40\n【等级8】无属性▕▏升级条件：知识-20\n【等级9】无属性▕▏升级条件：堕落-50\n【等级10】无属性▕▏升级条件：金币-500\n【等级11】无属性▕▏升级条件：堕落-60\n【等级12】无属性▕▏升级条件：血液-1500\n【等级13】无属性▕▏升级条件：堕落-70\n【等级14】无属性▕▏升级条件：咒术-300\n【等级15】无属性▕▏升级条件：堕落-80\n【等级16】无属性▕▏升级条件：旅程-50\n【等级17】无属性▕▏升级条件：堕落-90\n【等级18】无属性▕▏升级条件：知识-30\n【等级19】无属性▕▏升级条件：堕落-100\n【等级20】无属性▕▏升级条件：金币-2500\n【等级21】1% 回帖金币+5 血液+5 堕落+5、发帖灵魂+1 咒术+10▕▏升级条件：堕落-200\n【等级22】1% 回帖金币+5 血液+5 堕落+5、发帖灵魂+1 咒术+10▕▏升级条件：灵魂-2\n【 Max 】1% 回帖金币+5 血液+5 堕落+5、发帖灵魂+1 咒术+10",
  "模擬人生4": "模擬人生4\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖金币+1",
  "达拉然": "达拉然\n【勋章类型】板块\n【入手条件】旅程≥25 知识≥25\n【商店售价】100金币\n【 Max 】2% 回帖知识+1",
  "牧羊人": "牧羊人\n【勋章类型】天赋\n【入手条件】发帖数>=100\n【商店售价】无\n【等级1】5% 发帖旅程+1▕▏升级条件：发帖数>250\n【等级2】8% 发帖旅程+1▕▏升级条件：发帖数>500\n【 Max 】10% 发帖旅程+1",
  "森林羊男": "森林羊男\n【勋章类型】天赋\n【入手条件】注册天数>45 并且 在线时间(小时)>200 井且 血液>150井且 旅程>50\n【商店售价】无\n【 Max 】5% 回帖知识+1、发帖知识+1",
  "堕落之舞": "堕落之舞\n【勋章类型】天赋\n【入手条件】堕落>40\n【商店售价】无\n【 Max 】35% 发帖金币+3",
  "黄色就是俏皮": "黄色就是俏皮\n【勋章类型】天赋\n【入手条件】堕落>20 幷且 血液>60 并且主题数 >= 6\n【商店售价】无\n【等级1】3% 回帖血液+1、发帖血液+1▕▏升级条件：50主题数\n【 Max 】10% 回帖血液+1、发帖血液+1",
  "骑兽之子": "骑兽之子\n【勋章类型】天赋\n【入手条件】旅程 >= 120 井且 灵魂 >=1\n【商店售价】无\n【等级1】20% 回帖血液+1、发帖血液+3 咒术+1▕▏升级条件：灵魂 ≥3\n【 Max 】50% 回帖血液+1、发帖血液+3 咒术+1",
  "禽兽扒手": "禽兽扒手\n【勋章类型】天赋\n【入手条件】追随 >= 35 幷且 旅程 >= 45 并且 知识 >= 15\n【商店售价】无\n【等级1】10% 回帖血液-1 咒术+1▕▏升级条件：101在线时间\n【等级2】30% 回帖血液-1 咒术+1▕▏升级条件：110在线时间\n【等级3】10% 回帖血液-1 咒术+1▕▏升级条件：201在线时间\n【等级4】30% 回帖血液-1 咒术+1▕▏升级条件：210在线时间\n【等级5】10% 回帖血液-1 咒术+1▕▏升级条件：301在线时间\n【等级6】30% 回帖血液-1 咒术+1▕▏升级条件：310在线时间\n【等级7】10% 回帖血液-1 咒术+1▕▏升级条件：401在线时间\n【等级8】30% 回帖血液-1 咒术+1▕▏升级条件：410在线时间\n【等级9】10% 回帖血液-1 咒术+1▕▏升级条件：501在线时间\n【等级10】30% 回帖血液-1 咒术+1▕▏升级条件：510在线时间\n【等级11】10% 回帖血液-1 咒术+1▕▏升级条件：601在线时间\n【等级12】30% 回帖血液-1 咒术+1▕▏升级条件：610在线时间\n【等级13】10% 回帖血液-1 咒术+1▕▏升级条件：701在线时间\n【等级14】10% 回帖血液-1 咒术+1▕▏升级条件：710在线时间\n【等级15】10% 回帖血液-1 咒术+1▕▏升级条件：801在线时间\n【等级16】30% 回帖血液-1 咒术+1▕▏升级条件：810在线时间\n【等级17】10% 回帖血液-1 咒术+1▕▏升级条件：901在线时间\n【等级18】30% 回帖血液-1 咒术+1▕▏升级条件：910在线时间\n【等级19】10% 回帖血液-1 咒术+1▕▏升级条件：1001在线时间\n【等级20】30% 回帖血液-1 咒术+1▕▏升级条件：1100在线时间\n【等级21】10% 回帖血液-1 咒术+1▕▏升级条件：2001在线时间\n【等级22】30% 回帖血液-1 咒术+1▕▏升级条件：2100在线时间\n【等级23】10% 回帖血液-1 咒术+1▕▏升级条件：3001在线时间\n【 Max 】35% 回帖血液-1 咒术+1",
  "野兽之子": "野兽之子\n【勋章类型】天赋\n【入手条件】在线时间(小时) >= 200 井且 旅程 >= 20 井且 知识 >= 20 并且追随>= 20\n【商店售价】无\n【等级1】无属性▕▏升级条件：金币≥200\n【等级2】3% 回帖知识+1 血液-1▕▏升级条件：金币≥400\n【等级3】5% 回帖知识+1 血液-1▕▏升级条件：金币≥600\n【等级4】7% 回帖知识+1 血液-1▕▏升级条件：金币≥800\n【 Max 】9% 回帖知识+1 血液-1",
  "结晶卵": "结晶卵\n【勋章类型】宠物\n【入手条件】知识≥15\n【商店售价】280金币\n【等级1】2% 发帖知识+1▕▏升级条件：咒术≥40\n【等级2】4% 回帖咒术+1、发帖知识+1▕▏升级条件：咒术≥80\n【 Max 】8% 回帖咒术+2、发帖知识+1 咒术+2",
  "暮色卵": "暮色卵\n【勋章类型】宠物\n【入手条件】旅程≥15\n【商店售价】260金币\n【等级1】4% 回帖堕落+1▕▏升级条件：堕落≥80\n【等级2】7% 回帖堕落+1、发帖血液+2▕▏升级条件：堕落≥150\n【 Max 】15% 回帖堕落+1 血液+1、发帖血液+3",
  "青鸾蛋": "青鸾蛋\n【勋章类型】宠物\n【入手条件】旅程≥29\n【商店售价】220金币\n【等级1】6% 回帖金币+1▕▏升级条件：主题数≥20\n【 Max 】12% 回帖血液+1 金币+1、发帖旅程+1",
  "电磁卵": "电磁卵\n【勋章类型】宠物\n【入手条件】咒术≥30\n【商店售价】240金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗11咒术\n【等级2】7% 回帖金币+2▕▏升级条件：消耗22咒术\n【等级3】10% 回帖金币+2▕▏升级条件：消耗33咒术\n【 Max 】15% 回帖金币+2",
  "珊瑚色礁石蛋": "珊瑚色礁石蛋\n【勋章类型】宠物\n【入手条件】发帖数≥150\n【商店售价】260金币\n【等级1】4% 回帖咒术+1 血液+1、发帖咒术+2▕▏升级条件：消耗188金币\n【 Max 】8% 回帖咒术+1 血液+2、发帖咒术+2",
  "月影蛋": "月影蛋\n【勋章类型】宠物\n【入手条件】血液≥303\n【商店售价】310金币\n【等级1】5% 回帖血液+2、发帖咒术+1▕▏升级条件：消耗30咒术\n【等级2】10% 回帖血液+2、发帖咒术+1▕▏升级条件：血液≥388\n【 Max 】15% 回帖血液+2、发帖咒术+1",
  "马戏团灰蛋": "马戏团灰蛋\n【勋章类型】宠物\n【入手条件】旅程≥25\n【商店售价】270金币\n【等级1】5% 回帖金币+1、发帖堕落+1▕▏升级条件：消耗180血液\n【等级2】10% 回帖金币+2、发帖堕落+1▕▏升级条件：消耗150金币\n【 Max 】12% 回帖血液+2、发帖旅程+1",
  "郁苍卵": "郁苍卵\n【勋章类型】宠物\n【入手条件】旅程≥35\n【商店售价】350金币\n【等级1】5% 回帖血液+2▕▏升级条件：消耗369金币\n【 Max 】15% 回帖血液+2、发帖血液+3",
  "熔岩蛋": "熔岩蛋\n【勋章类型】宠物\n【入手条件】旅程≥35\n【商店售价】350金币\n【等级1】5% 回帖金币+2▕▏升级条件：消耗369血液\n【 Max 】15% 回帖金币+2、发帖金币+3",
  "灵鹫蛋": "灵鹫蛋\n【勋章类型】宠物\n【入手条件】堕落≤19\n【商店售价】310金币\n【等级1】3% 回帖血液+1、发帖知识+1▕▏升级条件：主题≥10\n【等级2】6% 回帖血液+1、发帖知识+1▕▏升级条件：消耗210金币\n【等级3】10% 回帖血液+2 堕落-1、发帖知识+1▕▏升级条件：主题≥50\n【 Max 】15% 回帖血液+2 堕落-1、发帖知识+1",
  "血鹫蛋": "血鹫蛋\n【勋章类型】宠物\n【入手条件】堕落≥25\n【商店售价】310金币\n【等级1】3% 回帖金币+1、发帖旅程+1▕▏升级条件：咒术≥50\n【等级2】6% 回帖金币+1、发帖旅程+1▕▏升级条件：消耗210血液\n【等级3】10% 回帖金币+2 堕落+1、发帖旅程+1▕▏升级条件：咒术≥150\n【 Max 】15% 回帖金币+2 堕落+1、发帖旅程+1",
  "软泥怪蛋": "软泥怪蛋\n【勋章类型】宠物\n【入手条件】主题≥30\n【商店售价】150金币\n【等级1】4% 回帖血液+2▕▏升级条件：消耗220金币\n【 Max 】4% 回帖血液+1 旅程+1",
  "螺旋纹卵": "螺旋纹卵\n【勋章类型】宠物\n【入手条件】旅程≥33\n【商店售价】270金币\n【等级1】5% 回帖堕落+1 金币+1▕▏升级条件：消耗188血液\n【 Max 】11% 回帖堕落+1 金币+1",
  "万圣彩蛋": "万圣彩蛋\n【勋章类型】宠物\n【入手条件】追随≥35\n【商店售价】310金币\n【等级1】6% 回帖金币+2、发帖金币+2▕▏升级条件：消耗33咒术\n【 Max 】12% 回帖金币+2、发帖金币+2",
  "幽光彩蛋": "幽光彩蛋\n【勋章类型】宠物\n【入手条件】知识≥22\n【商店售价】300金币\n【等级1】5% 回帖血液+1、发帖咒术+1▕▏升级条件：消耗25咒术\n【等级2】8% 回帖血液+2、发帖咒术+1▕▏升级条件：旅程≥77\n【 Max 】14% 回帖血液+2、发帖咒术+1",
  "沙漠羽蛋": "沙漠羽蛋\n【勋章类型】宠物\n【入手条件】旅程≥25\n【商店售价】250金币\n【等级1】5% 回帖血液+2、发帖旅程+1▕▏升级条件：消耗173金币\n【 Max 】10% 回帖血液+2、发帖旅程+1",
  "林中之蛋": "林中之蛋\n【勋章类型】宠物\n【入手条件】旅程≥35\n【商店售价】275金币\n【等级1】6% 回帖金币+1 血液+1▕▏升级条件：消耗213血液\n【 Max 】12% 回帖金币+1 血液+1",
  "五彩斑斓的蛋": "五彩斑斓的蛋\n【勋章类型】宠物\n【入手条件】堕落≤100 旅程≥50\n【商店售价】220金币\n【等级1】2% 回帖血液+1▕▏升级条件：消耗100血液\n【等级2】5% 回帖咒术+1 血液-1▕▏升级条件：消耗50咒术\n【 Max 】8% 回帖咒术+2 堕落-1",
  "血红色的蛋": "血红色的蛋\n【勋章类型】宠物\n【入手条件】堕落≥60\n【商店售价】290金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗75血液\n【等级2】10% 回帖金币+1 堕落+1▕▏升级条件：消耗100血液\n【 Max 】15% 回帖金币+1 堕落+1",
  "海边的蛋": "海边的蛋\n【勋章类型】宠物\n【入手条件】在线时间≥300小时\n【商店售价】288金币\n【等级1】10% 回帖金币+1▕▏升级条件：消耗100金币\n【 Max 】20% 回帖金币+1、发帖金币+3",
  "新手蛋": "新手蛋\n【勋章类型】宠物\n【入手条件】发帖数＞＝50 主题数＞＝ 2\n【商店售价】120金币\n【等级1】10% 回帖金币+1▕▏升级条件：发帖数≥150\n【等级2】20% 回帖金币+1▕▏升级条件：发帖数≥300\n【等级3】30% 回帖金币+1▕▏升级条件：发帖数≥500\n【 Max 】5% 回帖金币+1",
  "深渊遗物": "深渊遗物\n【勋章类型】奖品\n【入手条件】【星尘月陨】活动奖励\n【等级1】1% 回帖血液+1▕▏升级条件：消耗50血液\n【等级2】1% 回帖血液+2▕▏升级条件：消耗10咒术\n【 Max 】1% 回帖血液+3",
  "【限定】深渊遗物": "【限定】深渊遗物\n【勋章类型】宠物\n【入手条件】【星尘月陨】活动奖励\n【等级1】1% 回帖血液+3▕▏升级条件：消耗-1旅程\n【 Max 】10% 回帖血液+3、发帖旅程+1",
  "小阿尔的蛋": "小阿尔的蛋\n【勋章类型】宠物\n【入手条件】在线时间≥200小时\n【商店售价】388金币\n【等级1】5% 回帖血液+1、发帖金币+2▕▏升级条件：消耗150金币\n【等级2】8% 回帖血液+1、发帖金币+2▕▏升级条件：消耗200血液\n【等级3】10% 回帖血液+2、发帖金币+2▕▏升级条件：在线时间≥1000小时\n【 Max 】15% 回帖血液+2、发帖金币+3",
  "红石": "红石\n【勋章类型】资产\n【入手条件】血液≥30\n【商店售价】177金币\n【等级1】4% 回帖血液+1、发帖血液+1▕▏升级条件：血液≥233\n【 Max 】8% 回帖血液+2、发帖旅程+1",
  "幽灵竹筒": "幽灵竹筒\n【勋章类型】资产\n【入手条件】在线时间≥20小时\n【商店售价】280金币\n【等级1】4% 回帖血液+1、发帖血液+1▕▏升级条件：在线时间≥100小时\n【等级2】8% 回帖血液+2、发帖血液+1▕▏升级条件：在线时间≥300小时\n【 Max 】4% 回帖旅程+1 血液+1、发帖旅程+1 血液+1",
  "神秘的红茶": "神秘的红茶\n【勋章类型】资产\n【入手条件】无\n【商店售价】77金币\n【等级1】3% 回帖血液+1、发帖血液+1▕▏升级条件：消耗30血液\n【等级2】7% 回帖血液+1▕▏升级条件：堕落≥33\n【 Max 】7% 回帖血液+1 堕落+1",
  "种植土豆": "种植土豆\n【勋章类型】资产\n【入手条件】无\n【商店售价】140金币\n【等级1】2% 回帖血液+1▕▏升级条件：消耗25血液\n【等级2】4% 回帖血液+1▕▏升级条件：知识≥5\n【等级3】6% 回帖血液+1▕▏升级条件：消耗35血液\n【等级4】8% 回帖血液+1▕▏升级条件：知识≥15\n【等级5】10% 回帖血液+1、发帖血液+2▕▏升级条件：消耗-88金币\n【 Max 】3% 回帖血液+1",
  "用过的粪桶": "用过的粪桶\n【勋章类型】资产\n【入手条件】旅程≥10\n【商店售价】111金币\n【等级1】3% 回帖血液-2 金币+2▕▏升级条件：消耗30血液\n【 Max 】6% 回帖血液-1 金币+2",
  "箭术卷轴": "箭术卷轴\n【勋章类型】资产\n【入手条件】无\n【商店售价】70金币\n【 Max 】4% 回帖血液+1、发帖血液+1",
  "【圣诞限定】心心念念小雪人": "【圣诞限定】心心念念小雪人\n【勋章类型】资产\n【入手条件】无\n【商店售价】666金币\n【 Max 】1% 发帖旅程+1 金币+5 血液+5 咒术+3 知识+1 灵魂+1",
  "心心念念小雪人": "心心念念小雪人\n【勋章类型】资产\n【入手条件】无\n【商店售价】666金币\n【 Max 】1% 发帖旅程+1 金币+5 血液+5 咒术+3 知识+1 灵魂+1",
  "魔法石碑": "魔法石碑\n【勋章类型】资产\n【入手条件】无\n【商店售价】130金币\n【 Max 】7% 回帖血液+1，发帖咒术+1",
  "远古石碑": "远古石碑\n【勋章类型】资产\n【入手条件】无\n【商店售价】140金币\n【 Max 】7% 回帖金币+1，发帖咒术+1",
  "冒险用面包": "冒险用面包\n【勋章类型】资产\n【入手条件】无\n【商店售价】150金币\n【等级1】6% 回帖血液+1▕▏升级条件：旅程≥18\n【等级2】6% 回帖血液+2▕▏升级条件：消耗-25血液\n【 Max 】2% 回帖血液+2",
  "海螺号角": "海螺号角\n【勋章类型】资产\n【入手条件】旅程≥15\n【商店售价】277金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗177血液\n【等级2】10% 回帖金币+1▕▏升级条件：旅程≥77\n【 Max 】10% 回帖金币+1、发帖旅程+1 金币+3",
  "沙漠神灯": "沙漠神灯\n【勋章类型】资产\n【入手条件】知识≥20\n【商店售价】250金币\n【等级1】2% 回帖咒术+1▕▏升级条件：消耗25咒术\n【等级2】3% 回帖咒术+1、发帖知识+1▕▏升级条件：金币≥433\n【 Max 】3% 回帖咒术+1 知识+1、发帖咒术+1 知识+1",
  "冒险用指南针": "冒险用指南针\n【勋章类型】资产\n【入手条件】无\n【商店售价】150金币\n【 Max 】3% 回帖旅程+1",
  "暖心小火柴": "暖心小火柴\n【勋章类型】资产\n【入手条件】旅程≥15\n【商店售价】333金币\n【等级1】勋章博物馆资料暂缺\n【等级2】11% 回帖金币+1 血液-1、发帖金币+2▕▏升级条件：消耗33血液\n【等级3】11% 回帖金币-1 血液+1、发帖血液+2▕▏升级条件：消耗33血液\n【 Max 】11% 回帖金币+1 血液+1、发帖血液+1 金币+1",
  "神秘的漂流瓶": "神秘的漂流瓶\n【勋章类型】资产\n【入手条件】旅程≥66\n【商店售价】500金币\n【等级1】无属性▕▏升级条件：消耗1旅程\n【等级2】无属性▕▏升级条件：消耗1旅程\n【等级3】无属性▕▏升级条件：消耗1旅程\n【等级4】无属性▕▏升级条件：消耗1旅程\n【等级5】无属性▕▏升级条件：消耗1旅程\n【等级6】无属性▕▏升级条件：消耗1旅程\n【等级7】无属性▕▏升级条件：消耗1旅程\n【等级8】无属性▕▏升级条件：消耗1旅程\n【等级9】无属性▕▏升级条件：消耗1旅程\n【等级10】无属性▕▏升级条件：消耗-9旅程\n【 Max 】12% 发帖旅程+1 知识+1",
  "冒险用绷带": "冒险用绷带\n【勋章类型】资产\n【入手条件】无\n【商店售价】211金币\n【等级1】10% 回帖血液+1▕▏升级条件：消耗-10血液\n【等级2】8% 回帖血液+1▕▏升级条件：消耗-15血液\n【等级3】6% 回帖血液+1▕▏升级条件：消耗-20血液\n【 Max 】4% 回帖血液+2",
  "宝箱内的球": "宝箱内的球\n【勋章类型】资产\n【入手条件】旅程≥15\n【商店售价】350金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗100金币\n【等级2】6% 回帖金币+1、发帖金币+1▕▏升级条件：消耗150血液\n【等级3】2% 回帖旅程+1 金币-1、发帖金币+2▕▏升级条件：知识≥60\n【等级4】10% 回帖金币+2 血液-1、发帖金币+3▕▏升级条件：旅程≥77\n【 Max 】12% 回帖金币+3 血液-1、发帖金币+3",
  "SCP-s-1889": "SCP-s-1889\n【勋章类型】资产\n【入手条件】旅程≥20\n【商店售价】450金币\n【等级1】无属性▕▏升级条件：消耗1旅程\n【等级2】无属性▕▏升级条件：消耗1旅程\n【等级3】无属性▕▏升级条件：消耗1旅程\n【等级4】无属性▕▏升级条件：消耗1旅程\n【等级5】无属性▕▏升级条件：消耗1旅程\n【等级6】无属性▕▏升级条件：消耗-5旅程\n【 Max 】3% 回帖知识+1 血液+2",
  "GHOST": "GHOST\n【勋章类型】资产\n【入手条件】堕落≥10\n【商店售价】200金币\n【等级1】1% 回帖血液-1 金币+1、发帖血液-1 金币+1▕▏升级条件：消耗10血液\n【等级2】2% 回帖血液-1 金币+1、发帖血液-1 金币+1▕▏升级条件：消耗10血液\n【等级3】3% 回帖血液-1 金币+1、发帖血液-1 金币+1▕▏升级条件：消耗10血液\n【等级4】4% 回帖血液-1 金币+2、发帖血液-1 金币+2▕▏升级条件：消耗10血液\n【等级5】5% 回帖血液-1 金币+2、发帖血液-1 金币+2▕▏升级条件：消耗10血液\n【等级6】6% 回帖血液-1 金币+2、发帖血液-1 金币+2▕▏升级条件：消耗10血液\n【 Max 】7% 回帖血液-1 金币+3、发帖血液-1 金币+3",
  "GM論壇初心者勛章": "GM論壇初心者勛章\n【勋章类型】资产\n【入手条件】无\n【商店售价】100金币\n【等级1】15% 回帖金币+1 血液+1、发帖金币+3 血液+3▕▏升级条件：在线时间≥72\n【等级2】12% 回帖金币+1 血液+1、发帖金币+3 血液+3▕▏升级条件：在线时间≥150\n【等级3】8% 回帖金币+1 血液+1、发帖金币+3 血液+3▕▏升级条件：在线时间≥280\n【等级4】5% 回帖金币+1 血液+1、发帖金币+3 血液+3▕▏升级条件：在线时间≥560\n【 Max 】2% 回帖金币+1 血液+1、发帖金币+5 血液+5",
  "社畜专用闹钟": "社畜专用闹钟\n【勋章类型】资产\n【入手条件】无\n【商店售价】150金币\n【等级1】20% 回帖金币+1 血液-1▕▏升级条件：金币＞996\n【 Max 】20% 回帖金币-1 血液+1",
  "冒险用宝箱": "冒险用宝箱\n【勋章类型】资产\n【入手条件】无\n【商店售价】200金币\n【等级1】5% 回帖血液+1▕▏升级条件：旅程≥25\n【等级2】5% 回帖血液+1、发帖旅程+1▕▏升级条件：消耗15咒术\n【 Max 】10% 回帖血液+1、发帖知识+1",
  "羽毛笔": "羽毛笔\n【勋章类型】资产\n【入手条件】知识≥5\n【商店售价】280金币\n【等级1】1% 回帖旅程+1 血液+1▕▏升级条件：消耗33血液\n【等级2】2% 回帖旅程+1 血液+1▕▏升级条件：消耗66血液\n【等级3】3% 回帖旅程+1 血液+1▕▏升级条件：消耗99血液\n【 Max 】4% 回帖旅程+1 血液+1",
  "超级无敌名贵金卡": "超级无敌名贵金卡\n【勋章类型】资产\n【入手条件】无\n【商店售价】688金币\n【 Max 】100% 回帖金币+0",
  "One Ring": "One Ring\n【勋章类型】资产\n【入手条件】旅程≥25\n【商店售价】500金币\n【等级1】无属性▕▏升级条件：消耗200金币\n【等级2】无属性▕▏升级条件：消耗-1堕落\n【等级3】无属性▕▏升级条件：消耗-1堕落\n【等级4】无属性▕▏升级条件：消耗-1堕落\n【等级5】无属性▕▏升级条件：消耗-1堕落\n【等级6】无属性▕▏升级条件：消耗-1堕落\n【等级7】无属性▕▏升级条件：消耗-1堕落\n【等级8】无属性▕▏升级条件：消耗-1堕落\n【等级9】无属性▕▏升级条件：消耗-1堕落\n【等级10】无属性▕▏升级条件：消耗-1堕落\n【等级11】无属性▕▏升级条件：消耗10堕落\n【 Max 】50% 发帖血液+1 堕落-1",
  "秘密空瓶": "秘密空瓶\n【勋章类型】资产\n【入手条件】追随≥42\n【商店售价】666金币\n【等级1】15% 回帖血液+1▕▏升级条件：消耗1灵魂\n【等级2】100% 发帖血液+2▕▏升级条件：灵魂≥1\n【 Max 】100% 发帖金币+2 血液+1",
  "梦中的列车": "梦中的列车\n【勋章类型】资产\n【入手条件】旅程≥25\n【商店售价】350金币\n【等级1】无属性▕▏升级条件：消耗1旅程\n【等级2】无属性▕▏升级条件：消耗1旅程\n【等级3】无属性▕▏升级条件：消耗1旅程\n【等级4】无属性▕▏升级条件：消耗1旅程\n【等级5】无属性▕▏升级条件：消耗1旅程\n【等级6】无属性▕▏升级条件：消耗1旅程\n【等级7】无属性▕▏升级条件：消耗1旅程\n【等级8】无属性▕▏升级条件：消耗1旅程\n【等级9】无属性▕▏升级条件：消耗1旅程\n【等级10】无属性▕▏升级条件：消耗1旅程\n【等级11】无属性▕▏升级条件：消耗-10旅程\n【 Max 】3% 回帖知识+1、发帖知识+1",
  "冒险专用绳索": "冒险专用绳索\n【勋章类型】装备\n【入手条件】堕落＞10\n【商店售价】220金币\n【等级1】4% 回帖堕落+1▕▏升级条件：堕落≥66\n【 Max 】8% 回帖堕落+1 金币+1",
  "赫尔墨斯·看守者之杖": "赫尔墨斯·看守者之杖\n【勋章类型】装备\n【入手条件】知识≥8\n【商店售价】288金币\n【等级1】3% 回帖血液+1▕▏升级条件：消耗200血液\n【等级2】1% 回帖知识+1、发帖知识+1▕▏升级条件：知识≥20\n【等级3】8% 回帖知识+1 堕落+2、发帖知识+1 堕落+2▕▏升级条件：堕落≥30\n【等级4】6% 回帖知识+1 堕落+1、发帖知识+1 堕落+1▕▏升级条件：堕落≥60\n【等级5】4% 回帖知识+1 堕落+1、发帖知识+1 堕落+1▕▏升级条件：堕落≥90\n【 Max 】2% 回帖知识+1 血液-1、发帖知识+1 血液-1",
  "巴啦啦小魔仙棒": "巴啦啦小魔仙棒\n【勋章类型】装备\n【入手条件】无\n【商店售价】130金币\n【等级1】3% 回帖知识+1▕▏升级条件：知识≥13\n【等级2】6% 回帖咒术+1▕▏升级条件：消耗39咒术\n【 Max 】9% 回帖咒术+1、发帖咒术+1",
  "十字军护盾": "十字军护盾\n【勋章类型】装备\n【入手条件】无\n【商店售价】190金币\n【 Max 】8% 回帖金币+1、发帖金币+1",
  "龙血之斧": "龙血之斧\n【勋章类型】装备\n【入手条件】无\n【商店售价】210金币\n【 Max 】15% 发帖血液+3",
  "蔷薇骑士之刃": "蔷薇骑士之刃\n【勋章类型】装备\n【入手条件】追随≥10\n【商店售价】320金币\n【等级1】4% 回帖金币+1 血液+1、发帖知识+1▕▏升级条件：追随≥87\n【 Max 】8% 回帖金币+1 血液+1、发帖知识+1",
  "狩猎用小刀": "狩猎用小刀\n【勋章类型】装备\n【入手条件】知识≥10\n【商店售价】230金币\n【等级1】4% 回帖堕落+1▕▏升级条件：消耗100血液\n【等级2】8% 回帖血液+2▕▏升级条件：消耗-5堕落\n【等级3】8% 回帖金币+2▕▏升级条件：消耗-5堕落\n【 Max 】4% 回帖堕落+1、发帖旅程+1",
  "钢铁勇士弯刀": "钢铁勇士弯刀\n【勋章类型】装备\n【入手条件】无\n【商店售价】140金币\n【 Max 】4% 回帖金币+1 堕落-1",
  "海盗弯钩": "海盗弯钩\n【勋章类型】装备\n【入手条件】无\n【商店售价】130金币\n【 Max 】4% 回帖血液+1 堕落+1",
  "生锈的海盗刀枪": "生锈的海盗刀枪\n【勋章类型】装备\n【入手条件】堕落≥15\n【商店售价】310金币\n【等级1】5% 回帖堕落+1、发帖金币+3▕▏升级条件：旅程≥60\n【等级2】10% 回帖堕落+1、发帖金币+3▕▏升级条件：消耗180金币\n【 Max 】15% 回帖堕落+1、发帖金币+3",
  "日荒戒指": "日荒戒指\n【勋章类型】装备\n【入手条件】无\n【商店售价】300金币\n【等级1】5% 回帖金币+1、发帖金币+1▕▏升级条件：消耗88金币\n【 Max 】10% 回帖金币+1、发帖金币+2",
  "月陨戒指": "月陨戒指\n【勋章类型】装备\n【入手条件】无\n【商店售价】300金币\n【等级1】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗88血液\n【 Max 】10% 回帖血液+1、发帖血液+2",
  "星芒戒指": "星芒戒指\n【勋章类型】装备\n【入手条件】无\n【商店售价】300金币\n【等级1】4% 回帖咒术+1、发帖咒术+1▕▏升级条件：消耗18咒术\n【 Max 】8% 回帖咒术+1、发帖咒术+1",
  "琉璃玉坠": "琉璃玉坠\n【勋章类型】装备\n【入手条件】无\n【商店售价】180金币\n【等级1】7% 回帖金币+1▕▏升级条件：消耗88血液\n【 Max 】7% 回帖血液+1 金币+1",
  "武士之魂": "武士之魂\n【勋章类型】装备\n【入手条件】知识≥30\n【商店售价】999金币\n【等级1】10% 回帖血液+1、发帖血液+2▕▏升级条件：知识≥50\n【等级2】12% 回帖血液+2、发帖血液+3▕▏升级条件：消耗130血液\n【等级3】15% 回帖血液+2 金币+1、发帖旅程+1▕▏升级条件：消耗1灵魂\n【 Max 】2% 回帖金币+1 血液+3 旅程+1 知识+1 、发帖灵魂+1",
  "力量腕带": "力量腕带\n【勋章类型】装备\n【入手条件】无\n【商店售价】125金币\n【 Max 】8% 回帖金币+1",
  "物理学圣剑": "物理学圣剑\n【勋章类型】装备\n【入手条件】无\n【商店售价】188金币\n【 Max 】6% 回帖血液+1、发帖知识+1",
  "男用贞操带": "男用贞操带\n【勋章类型】装备\n【入手条件】无\n【商店售价】180金币\n【 Max 】3% 回帖旅程+1 堕落-1",
  "贤者头盔": "贤者头盔\n【勋章类型】装备\n【入手条件】知识≥1\n【商店售价】200金币\n【等级1】8% 回帖金币-1 咒术+1、发帖金币+1▕▏升级条件：消耗100血液\n【等级2】10% 回帖血液-1 咒术+1、发帖咒术+1▕▏升级条件：消耗100金币\n【 Max 】8% 回帖咒术+1、发帖知识+1",
  "恩惠护符": "恩惠护符\n【勋章类型】装备\n【入手条件】旅程≥10\n【商店售价】250金币\n【等级1】3% 回帖金币+1、发帖金币+1▕▏升级条件：消耗350金币\n【等级2】6% 回帖金币+2，发帖金币+3▕▏升级条件：旅程≥69\n【 Max 】9% 回帖金币+3、发帖金币+5",
  "超级幸运无敌辉石": "超级幸运无敌辉石\n【勋章类型】装备\n【入手条件】无\n【商店售价】1088金币\n【等级1】1% 回帖血液+5、发帖血液+5▕▏升级条件：消耗1188血液\n【 Max 】1% 回帖金币+20、发帖金币+20",
  "安杜因·乌瑞恩": "安杜因·乌瑞恩\n【勋章类型】游戏男从\n【入手条件】旅程≥10\n【商店售价】480金币\n【等级1】10% 发帖堕落-1▕▏升级条件：血液≥200\n【等级2】10% 回帖堕落-1、发帖堕落-1▕▏升级条件：咒术≥40\n【等级3】10% 回帖堕落-1 血液+1、发帖堕落-1 血液+1▕▏升级条件：消耗280血液\n【 Max 】13% 回帖堕落-1 血液+2、发帖堕落-1 血液+3",
  "羅素·托維": "羅素·托維\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】240金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗50血液\n【等级2】5% 回帖金币+1、发帖金币+1▕▏升级条件：金币≥100\n【等级3】15% 回帖金币+3、发帖金币+3▕▏升级条件：金币≥200\n【等级4】10% 回帖金币+2、发帖金币+2▕▏升级条件：金币≥300\n【 Max 】5% 回帖金币+1 血液+1、发帖金币+1 血液+1",
  "蓝礼·拜拉席恩": "蓝礼·拜拉席恩\n【勋章类型】真人男从\n【入手条件】堕落≤35\n【商店售价】520金币\n【等级1】5% 回帖金币+1、发帖金币+2▕▏升级条件：主题≥5\n【等级2】10% 回帖金币+1 血液+1、发帖金币+2▕▏升级条件：追随≥22\n【等级3】10% 回帖金币+1 血液+1、发帖旅程+1 金币+2▕▏升级条件：金币≥520\n【等级4】5% 回帖血液-1、发帖血液-1▕▏升级条件：消耗520金币\n【 Max 】13% 回帖金币+1 血液+1、发帖金币+2 血液+2",
  "阿列克西欧斯（Alexios）": "阿列克西欧斯（Alexios）\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】3% 发帖知识+1▕▏升级条件：主题≥10\n【等级2】3% 回帖堕落-1 血液+1、发帖知识+1▕▏升级条件：消耗200金币\n【等级3】5% 回帖堕落-1 血液+1、发帖知识+1▕▏升级条件：消耗220血液\n【等级4】7% 回帖堕落-1 血液+2、发帖知识+1▕▏升级条件：消耗-3知识\n【等级5】7% 发帖知识+1▕▏升级条件：知识≥50\n【 Max 】10% 回帖堕落-1 血液+2、发帖堕落-1 血液+3",
  "莱因哈特·威尔海姆": "莱因哈特·威尔海姆\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】449金币\n【等级1】勋章博物馆资料暂缺▕▏升级条件：旅程≥31\n【等级2】7% 回帖血液+1、发帖旅程+1 血液+2▕▏升级条件：旅程≥61\n【 Max 】10% 回帖血液+2、发帖旅程+1 血液+3",
  "尼克斯·乌尔里克": "尼克斯·乌尔里克\n【勋章类型】游戏男从\n【入手条件】堕落≤39\n【商店售价】520金币\n【等级1】6% 回帖血液+1、发帖咒术+1▕▏升级条件：咒术≥15\n【等级2】6% 回帖血液+2、发帖咒术+1▕▏升级条件：消耗333金币\n【等级3】10% 回帖血液+2、发帖咒术+2▕▏升级条件：消耗333血液\n【等级4】14% 回帖血液+1 堕落-1、发帖血液+1 堕落-2▕▏升级条件：血液≥1000\n【等级5】18% 回帖咒术+1 血液-2、发帖咒术+3 血液-3▕▏升级条件：堕落≥13\n【 Max 】10% 发帖堕落-1",
  "乔纳森·里德": "乔纳森·里德\n【勋章类型】游戏男从\n【入手条件】血液≥50\n【商店售价】360金币\n【等级1】3% 回帖血液+1▕▏升级条件：消耗200血液\n【等级2】6% 回帖血液+1▕▏升级条件：发帖数≥300\n【等级3】9% 回帖血液+2▕▏升级条件：知识≥40\n【等级4】12% 回帖血液+2 金币-1、发帖知识+1▕▏升级条件：消耗365血液\n【 Max 】15% 回帖血液+3 金币-1、发帖知识+1",
  "藤田優馬": "藤田優馬\n【勋章类型】真人男从\n【入手条件】主题≥3\n【商店售价】298金币\n【等级1】3% 回帖金币+1、发帖金币+2▕▏升级条件：消耗129金币\n【等级2】9% 回帖堕落+1 金币-1、发帖堕落+2▕▏升级条件：消耗129血液\n【 Max 】9% 回帖血液+1 堕落-1、发帖血液+2",
  "莫瑞甘": "莫瑞甘\n【勋章类型】女从\n【入手条件】咒术≥40\n【商店售价】360金币\n【等级1】3% 回帖堕落+1▕▏升级条件：消耗150血液\n【等级2】5% 回帖咒术+1▕▏升级条件：消耗220金币\n【等级3】7% 回帖咒术+1 堕落+1 血液-1▕▏升级条件：旅程≥60\n【等级4】10% 回帖咒术+1 血液-2、发帖血液-2 咒术+2▕▏升级条件：消耗200血液\n【 Max 】10% 回帖咒术+1 血液-1、发帖血液-3 咒术+3",
  "迈克尔迈尔斯": "迈克尔迈尔斯\n【勋章类型】真人男从\n【入手条件】旅程≥10\n【商店售价】450金币\n【等级1】3% 回帖血液+1、发帖堕落+1▕▏升级条件：消耗200血液\n【等级2】5% 回帖血液+2、发帖堕落+1 血液+1▕▏升级条件：主题≥20\n【等级3】7% 回帖血液+1 堕落+1、发帖堕落+1 血液+2▕▏升级条件：堕落≥40\n【 Max 】10% 回帖血液+2 堕落+1、发帖堕落+1 血液+3",
  "Doc": "Doc\n【勋章类型】游戏男从\n【入手条件】旅程≥10\n【商店售价】500金币\n【等级1】3% 回帖金币+1、发帖金币+1▕▏升级条件：旅程≥20\n【等级2】5% 回帖金币+2 血液-1、发帖金币+2 血液-1▕▏升级条件：知识≥10\n【等级3】8% 回帖金币+2、发帖金币+2▕▏升级条件：消耗250金币\n【 Max 】10% 回帖金币+2 血液+1、发帖金币+2 知识+1",
  "杰克·莫里森/士兵 76": "杰克·莫里森/士兵 76\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】476金币\n【等级1】3% 回帖血液+1▕▏升级条件：堕落≥30\n【等级2】6% 回帖血液+1、发帖旅程+1▕▏升级条件：堕落≥76\n【等级3】76% 回帖血液+3 堕落+2、发帖血液+7 金币+6 堕落+2▕▏升级条件：堕落≥77\n【等级4】12% 回帖血液+2 金币-1、发帖旅程+1▕▏升级条件：消耗376血液\n【 Max 】15% 回帖血液+3 金币-1、发帖旅程+1",
  "索林·橡木盾": "索林·橡木盾\n【勋章类型】真人男从\n【入手条件】追随≥14\n【商店售价】520金币\n【等级1】5% 回帖血液+1、发帖旅程+1▕▏升级条件：旅程≥15\n【等级2】7% 回帖血液+1、发帖旅程+1▕▏升级条件：消耗333血液\n【等级3】10% 回帖血液+2、发帖旅程+1▕▏升级条件：旅程-1\n【等级4】12% 回帖血液+2、发帖旅程+1▕▏升级条件：金币≥1314\n【等级5】15% 回帖金币+1 血液-1、发帖金币+3 血液-3▕▏升级条件：消耗1314金币\n【 Max 】18% 回帖血液+3、发帖血液+5",
  "陷阱杀手": "陷阱杀手\n【勋章类型】游戏男从\n【入手条件】堕落≥10\n【商店售价】280金币\n【等级1】3% 回帖血液+1、发帖血液+1▕▏升级条件：消耗134金币\n【等级2】5% 回帖堕落+1、发帖堕落+1▕▏升级条件：堕落≥30\n【 Max 】8% 回帖血液+2、发帖血液+2",
  "吉姆·霍普": "吉姆·霍普\n【勋章类型】真人男从\n【入手条件】在线时间≥100\n【商店售价】300金币\n【等级1】25% 回帖血液+1▕▏升级条件：血液≥35\n【等级2】6% 回帖血液+1 堕落+1▕▏升级条件：消耗100血液\n【等级3】8% 回帖血液+1 堕落+1▕▏升级条件：追随≥50\n【等级4】10% 回帖血液+1 堕落+1▕▏升级条件：消耗250金币\n【 Max 】12% 回帖血液+2 堕落+1",
  "沃特·沙利文": "沃特·沙利文\n【勋章类型】游戏男从\n【入手条件】堕落≥24\n【商店售价】302金币\n【等级1】2% 回帖血液+2▕▏升级条件：消耗173金币\n【等级2】4% 回帖血液+2、发帖咒术+1▕▏升级条件：追随≥40\n【等级3】6% 回帖咒术+1、发帖咒术+1▕▏升级条件：旅程≥60\n【等级4】8% 回帖咒术+1、发帖咒术+1▕▏升级条件：知识≥60\n【等级5】8% 回帖咒术+1、发帖咒术+1 血液+1▕▏升级条件：堕落≥91\n【 Max 】8% 回帖咒术+1 血液+1、发帖咒术+1 血液+1",
  "塞巴斯蒂安·斯坦": "塞巴斯蒂安·斯坦\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】450金币\n【等级1】5% 回帖金币+1▕▏升级条件：追随≥50\n【 Max 】8% 回帖金币+2、发帖金币+2",
  "魯杰羅·弗雷迪": "魯杰羅·弗雷迪\n【勋章类型】真人男从\n【入手条件】堕落≥30\n【商店售价】300金币\n【等级1】1% 回帖血液+3 堕落-1、发帖血液+3 堕落-1▕▏升级条件：消耗100血液\n【等级2】2% 回帖血液+3 堕落-1、发帖知识+1 堕落-1▕▏升级条件：主题≥30\n【等级3】2% 回帖知识+1 血液+1 堕落-1、发帖知识+1 血液+3▕▏升级条件：主题≥60\n【 Max 】3% 回帖知识+1 血液+1 堕落-1、发帖知识+1 血液+5",
  "莎伦": "莎伦\n【勋章类型】女从\n【入手条件】知识≥10\n【商店售价】350金币\n【等级1】4% 回帖血液+1▕▏升级条件：堕落≥50\n【等级2】4% 回帖血液+1 咒术+1▕▏升级条件：消耗50咒术\n【等级3】8% 回帖金币+2 咒术+1▕▏升级条件：咒术≥15\n【 Max 】10% 回帖金币+2",
  "疾风剑豪": "疾风剑豪\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】450金币\n【等级1】3% 回帖血液+1▕▏升级条件：旅程≥20\n【等级2】6% 回帖血液+1▕▏升级条件：堕落≥50\n【等级3】8% 回帖血液+1、发帖旅程+1▕▏升级条件：追随≥100\n【等级4】10% 回帖血液+1 金币+1、发帖旅程+1▕▏升级条件：消耗300金币\n【等级5】13% 回帖血液+1 金币+1、发帖旅程+1▕▏升级条件：灵魂≥1\n【 Max 】5% 回帖旅程+1",
  "【新手友好】昆進": "【新手友好】昆進\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】250金币\n【等级1】20% 回帖金币+1 血液+1、发帖金币+3 血液+3▕▏升级条件：总积分≥20\n【等级2】15% 回帖金币+1 血液+1、发帖金币+3 血液+3▕▏升级条件：总积分≥35\n【等级3】10% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：总积分≥50\n【 Max 】5% 回帖金币+1 血液+1、发帖金币+1 血液+1",
  "萨菲罗斯": "萨菲罗斯\n【勋章类型】游戏男从\n【入手条件】在线时间＞150小时\n【商店售价】440金币\n【等级1】4% 回帖血液+1、发帖血液+1▕▏升级条件：消耗130金币\n【等级2】7% 回帖堕落+1、发帖血液+2▕▏升级条件：消耗40咒术\n【等级3】8% 回帖血液+1 堕落+1、发帖血液+2 堕落+1▕▏升级条件：堕落≥100\n【等级4】10% 回帖血液+1 堕落+1、发帖血液+3▕▏升级条件：堕落≥150\n【等级5】12% 回帖血液+3 金币-1、发帖堕落+3▕▏升级条件：旅程≥100\n【等级6】14% 回帖血液+2 堕落+1、发帖血液+3▕▏升级条件：知识≥150\n【 Max 】16% 回帖血液+3 堕落+1、发帖血液+4",
  "丹·安博尔": "丹·安博尔\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】300金币\n【等级1】3% 回帖血液+1、发帖血液+2▕▏升级条件：追随≥10\n【等级2】4% 回帖金币+1 血液+1、发帖金币+1 血液+2▕▏升级条件：追随≥50\n【等级3】5% 回帖金币+1 血液+1、发帖金币+2 血液+2▕▏升级条件：消耗260血液\n【 Max 】6% 回帖金币+1 血液+2、发帖金币+2 血液+3",
  "汤姆·赫兰德": "汤姆·赫兰德\n【勋章类型】真人男从\n【入手条件】知识≥5\n【商店售价】450金币\n【等级1】4% 回帖血液+1▕▏升级条件：消耗50金币\n【等级2】8% 回帖血液+1、发帖知识+1▕▏升级条件：知识≥20\n【等级3】10% 回帖血液+1、发帖知识+1▕▏升级条件：旅程≥50\n【 Max 】12% 回帖血液+1、发帖知识+1",
  "超人": "超人\n【勋章类型】真人男从\n【入手条件】堕落＜500\n【商店售价】450金币\n【等级1】4% 回帖金币+1、堕落-1、发帖血液+1▕▏升级条件：血液≥200\n【等级2】6% 回帖金币+1、堕落-1、发帖血液+1▕▏升级条件：血液≥400\n【等级3】8% 回帖金币+2、发帖血液+2▕▏升级条件：血液≥600\n【 Max 】10% 回帖金币+3、发帖血液+3",
  "阿尔伯特·威斯克": "阿尔伯特·威斯克\n【勋章类型】游戏男从\n【入手条件】知识≥10\n【商店售价】400金币\n【等级1】3% 回帖血液+1、发帖血液+1▕▏升级条件：消耗100金币\n【等级2】5% 回帖血液+1、发帖血液+1▕▏升级条件：堕落≥30\n【等级3】8% 回帖血液+1、发帖血液+2▕▏升级条件：知识≥50\n【等级4】10% 回帖血液+1、发帖知识+1▕▏升级条件：知识≥100\n【 Max 】13% 回帖血液+2、发帖知识+1",
  "鬼王酒吞童子": "鬼王酒吞童子\n【勋章类型】游戏男从\n【入手条件】在线时间≥100小时\n【商店售价】300金币\n【等级1】4% 回帖血液+1▕▏升级条件：消耗50金币\n【等级2】勋章博物馆资料暂缺\n【等级3】8% 回帖血液+1、发帖堕落+1▕▏升级条件：堕落≥50\n【 Max 】10% 回帖血液+1、发帖堕落+2",
  "Scott Ryder": "Scott Ryder\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】4% 回帖金币+2▕▏升级条件：消耗80血液\n【等级2】4% 回帖金币+2、发帖旅程+1▕▏升级条件：追随≥40\n【等级3】6% 回帖金币+2、发帖旅程+1▕▏升级条件：消耗120金币\n【 Max 】8% 回帖金币+2、发帖旅程+1",
  "汉克/Hank": "汉克/Hank\n【勋章类型】游戏男从\n【入手条件】主题≥5\n【商店售价】500金币\n【等级1】5% 回帖血液+1▕▏升级条件：知识≥20\n【等级2】7% 回帖血液+1、发帖血液+1▕▏升级条件：追随≥40\n【等级3】9% 回帖血液+1、发帖血液+2▕▏升级条件：旅程≥60\n【等级4】11% 回帖血液+1、发帖血液+3▕▏升级条件：消耗500血液\n【等级5】13% 回帖血液+1、发帖旅程+1▕▏升级条件：消耗500金币\n【 Max 】25% 回帖血液+1、发帖旅程+1",
  "三角头": "三角头\n【勋章类型】真人男从\n【入手条件】主题≥20\n【商店售价】450金币\n【等级1】6% 回帖血液+1、发帖堕落+1▕▏升级条件：主题≥20\n【等级2】8% 回帖血液+2、发帖堕落+1▕▏升级条件：堕落≥50\n【 Max 】11% 回帖血液+2、发帖堕落+1 血液+2",
  "幻象": "幻象\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】350金币\n【等级1】3% 回帖金币+1、发帖金币+1▕▏升级条件：追随≥40\n【等级2】6% 回帖金币+1、发帖金币+2▕▏升级条件：知识≥40\n【等级3】8% 回帖金币+2、发帖金币+2▕▏升级条件：消耗200金币\n【 Max 】10% 回帖金币+2、发帖金币+3",
  "金刚狼": "金刚狼\n【勋章类型】真人男从\n【入手条件】在线时间≥240小时\n【商店售价】400金币\n【等级1】5% 回帖堕落+1▕▏升级条件：堕落≥10\n【等级2】5% 回帖金币+1 血液+1▕▏升级条件：金币≥500\n【等级3】8% 回帖金币+1 血液+1▕▏升级条件：血液≥500\n【 Max 】10% 回帖金币+1 血液+1 堕落-1",
  "克苏鲁": "克苏鲁\n【勋章类型】真人男从\n【入手条件】堕落≥15\n【商店售价】666金币\n【等级1】4% 回帖堕落+1、发帖堕落+1▕▏升级条件：堕落≥33\n【等级2】6% 回帖堕落+1、发帖知识+1▕▏升级条件：消耗666血液\n【等级3】8% 回帖堕落+2、发帖堕落+1 堕落+1▕▏升级条件：知识≥33\n【等级4】10% 回帖堕落+1 血液+1、发帖堕落+1▕▏升级条件：消耗1灵魂\n【 Max 】33% 回帖堕落+1 血液+1、发帖堕落+2 血液+2",
  "士官长": "士官长\n【勋章类型】游戏男从\n【入手条件】旅程≥10\n【商店售价】450金币\n【等级1】6% 回帖金币+1、发帖旅程+1▕▏升级条件：消耗200血液\n【等级2】8% 回帖金币+2 血液-1、发帖旅程+1▕▏升级条件：旅程≥50\n【 Max 】11% 回帖血液+3 金币-1、发帖旅程+1",
  "托尼·史塔克": "托尼·史塔克\n【勋章类型】真人男从\n【入手条件】旅程≥20\n【商店售价】650金币\n【等级1】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗50血液\n【等级2】6% 回帖血液+2、发帖血液+2▕▏升级条件：消耗100金币\n【等级3】7% 回帖血液+2、发帖血液+3▕▏升级条件：旅程≥40\n【等级4】8% 回帖血液+2、发帖知识+1▕▏升级条件：消耗150金币\n【等级5】9% 回帖血液+2、发帖知识+1▕▏升级条件：知识≥20\n【等级6】10% 回帖血液+2、发帖知识+1 血液+1▕▏升级条件：消耗200金币\n【等级7】11% 回帖血液+3 金币-1、发帖知识+1 血液+2▕▏升级条件：消耗200血液\n【等级8】12% 回帖血液+3 金币-1、发帖知识+1 血液+3▕▏升级条件：知识≥50\n【等级9】15% 发帖知识+1 血液+5▕▏升级条件：消耗999血液\n【 Max 】6% 回帖知识+1 血液+3、发帖知识+1 血液+5",
  "加勒特·霍克": "加勒特·霍克\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】300金币\n【等级1】4% 回帖金币+1、发帖金币+2▕▏升级条件：消耗150金币\n【等级2】6% 回帖金币+1、发帖金币+3▕▏升级条件：消耗200血液\n【等级3】8% 回帖金币+1 血液+1、发帖金币+3▕▏升级条件：追随≥100\n【 Max 】10% 回帖金币+1 血液+1、发帖旅程+1 金币+3",
  "艾吉奥": "艾吉奥\n【勋章类型】游戏男从\n【入手条件】旅程≥15\n【商店售价】500金币\n【等级1】4% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：消耗200金币\n【等级2】6% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：消耗200血液\n【等级3】8% 回帖金币+2 血液+1、发帖金币+2 血液+1▕▏升级条件：消耗50咒术\n【等级4】3% 回帖金币+1 旅程+1 血液+1、发帖金币+1 旅程+1 血液+1▕▏升级条件：知识≥50\n【等级5】4% 回帖金币+2 旅程+1 血液+1、发帖金币+2 旅程+1 血液+1▕▏升级条件：知识≥120\n【 Max 】4% 回帖金币+3 旅程+1 血液+1、发帖金币+3 旅程+1 血液+1",
  "Chris Mazdzer": "Chris Mazdzer\n【勋章类型】真人男从\n【入手条件】主题数≥5\n【商店售价】300金币\n【等级1】5% 回帖血液+1▕▏升级条件：消耗150金币\n【等级2】7% 回帖血液+2▕▏升级条件：消耗200金币\n【 Max 】10% 回帖血液+2",
  "索尔·奥丁森": "索尔·奥丁森\n【勋章类型】真人男从\n【入手条件】知识≥5\n【商店售价】500金币\n【等级1】2% 回帖咒术+1、发帖旅程+1▕▏升级条件：消耗88金币\n【等级2】3% 回帖咒术+1 金币+1、发帖旅程+1▕▏升级条件：消耗88血液\n【等级3】4% 回帖咒术+1 金币+1、发帖旅程+1▕▏升级条件：消耗120金币\n【等级4】5% 回帖咒术+1 金币+2、发帖旅程+1▕▏升级条件：消耗66咒术\n【等级5】6% 回帖咒术+1 金币+2、发帖旅程+1▕▏升级条件：消耗188金币\n【等级6】7% 回帖咒术+1 金币+3、发帖旅程+1▕▏升级条件：旅程≥88\n【 Max 】8% 回帖咒术+1 金币+3、发帖旅程+1",
  "绯红女巫": "绯红女巫\n【勋章类型】女从\n【入手条件】堕落≥10\n【商店售价】400金币\n【等级1】2% 回帖堕落+1、发帖咒术+1▕▏升级条件：消耗100血液\n【等级2】4% 回帖堕落+1、发帖咒术+1▕▏升级条件：堕落≥33\n【等级3】4% 回帖堕落+1 金币+1、发帖咒术+2▕▏升级条件：消耗66咒术\n【等级4】6% 回帖堕落+1 金币+1、发帖咒术+2▕▏升级条件：堕落≥88\n【等级5】8% 回帖堕落+1 金币+2、发帖咒术+2▕▏升级条件：知识≥88\n【 Max 】10% 回帖堕落+1 金币+2、发帖咒术+2",
  "泰比里厄斯": "泰比里厄斯\n【勋章类型】游戏男从\n【入手条件】咒术≥35\n【商店售价】555金币\n【等级1】5% 回帖血液+1、发帖咒术+1▕▏升级条件：消耗355金币\n【等级2】10% 回帖金币+1、发帖咒术+1▕▏升级条件：消耗255血液\n【等级3】12% 回帖血液+1、发帖咒术+1▕▏升级条件：咒术≥155\n【 Max 】15% 回帖血液+2、发帖旅程+1 咒术+1",
  "大古": "大古\n【勋章类型】真人男从\n【入手条件】主题数≥20\n【商店售价】300金币\n【等级1】2% 回帖金币+1 血液+1▕▏升级条件：血液≥100\n【等级2】4% 回帖金币+1 血液+1▕▏升级条件：消耗100血液\n【等级3】6% 回帖金币+1 血液+1▕▏升级条件：堕落≥50\n【等级4】8% 回帖血液+1 堕落+1▕▏升级条件：消耗200血液\n【等级5】10% 回帖金币+1 血液+1▕▏升级条件：血液≥666\n【 Max 】12% 回帖金币+1 血液+1、发帖旅程+1",
  "炙热的格拉迪欧拉斯": "炙热的格拉迪欧拉斯\n【勋章类型】游戏男从\n【入手条件】主题数≥15\n【商店售价】666金币\n【等级1】25% 回帖血液+2▕▏升级条件：咒术≥66\n【 Max 】50% 回帖血液+1",
  "格拉迪欧拉斯": "格拉迪欧拉斯\n【勋章类型】游戏男从\n【入手条件】血液≥100\n【商店售价】450金币\n【等级1】2% 回帖血液+1▕▏升级条件：消耗80金币\n【等级2】4% 回帖血液+1▕▏升级条件：追随≥50\n【等级3】8% 回帖血液+1、发帖血液+1▕▏升级条件：消耗120金币\n【等级4】11% 回帖血液+2、发帖血液+2▕▏升级条件：消耗330金币\n【 Max 】14% 回帖血液+2、发帖知识+1 血液+2",
  "卡洛斯·奥利维拉": "卡洛斯·奥利维拉\n【勋章类型】游戏男从\n【入手条件】追随≥10\n【商店售价】600金币\n【等级1】4% 回帖金币+1▕▏升级条件：消耗200血液\n【等级2】4% 回帖咒术+1▕▏升级条件：知识≥20\n【等级3】6% 回帖金币+1 咒术+1▕▏升级条件：消耗500金币\n【等级4】8% 回帖金币+1 咒术+1、发帖旅程+1▕▏升级条件：知识≥100\n【 Max 】10% 回帖金币+2 咒术+1 血液-1、发帖旅程+1",
  "巴基 (猎鹰与冬兵)": "巴基 (猎鹰与冬兵)\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】500金币\n【等级1】4% 回帖血液+1、发帖血液+1▕▏升级条件：消耗100血液\n【等级2】6% 回帖血液+1、发帖血液+1▕▏升级条件：消耗200血液\n【等级3】8% 回帖血液+2、发帖血液+2▕▏升级条件：消耗400血液\n【 Max 】12% 回帖血液+3、发帖血液+3",
  "Joker": "Joker\n【勋章类型】真人男从\n【入手条件】追随≥1\n【商店售价】400金币\n【等级1】6% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：追随≥30\n【等级2】8% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：追随≥60\n【等级3】10% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：追随≥90\n【等级4】2% 回帖旅程+1 知识+1、发帖旅程+1 知识+1▕▏升级条件：追随≥180\n【 Max 】3% 回帖旅程+1 知识+1、发帖旅程+1 知识+1",
  "V (DMC5)": "V (DMC5)\n【勋章类型】游戏男从\n【入手条件】堕落≤42\n【商店售价】500金币\n【等级1】4% 回帖堕落-1、发帖血液+2▕▏升级条件：金币≥200\n【等级2】6% 回帖堕落-1、发帖血液+2▕▏升级条件：知识≥30\n【等级3】8% 回帖血液+1 堕落-1、发帖血液+2▕▏升级条件：消耗150金币\n【等级4】10% 回帖血液+1 堕落-1、发帖血液+2▕▏升级条件：消耗150血液\n【等级5】13% 回帖血液+1 堕落-1、发帖血液+2▕▏升级条件：消耗66咒术\n【 Max 】13% 回帖血液+2 堕落-1、发帖血液+3",
  "Dante": "Dante\n【勋章类型】游戏男从\n【入手条件】主题数≥5\n【商店售价】666金币\n【等级1】4% 回帖血液+1、发帖血液+1▕▏升级条件：消耗150金币\n【等级2】6% 回帖血液+1、发帖血液+1▕▏升级条件：消耗150血液\n【等级3】8% 回帖血液+1、发帖血液+1▕▏升级条件：旅程≥50\n【等级4】10% 回帖血液+1、发帖血液+1▕▏升级条件：消耗88咒术\n【等级5】12% 回帖血液+2、发帖血液+2▕▏升级条件：血液≥666\n【等级6】15% 回帖血液+2 堕落-1、发帖血液+2▕▏升级条件：血液≥999\n【 Max 】13% 回帖金币+3、发帖血液+3",
  "Vergil": "Vergil\n【勋章类型】游戏男从\n【入手条件】咒术≥15\n【商店售价】500金币\n【等级1】4% 回帖血液+1、发帖血液+2▕▏升级条件：消耗150金币\n【等级2】4% 回帖血液+1、发帖咒术+2▕▏升级条件：追随≥50\n【等级3】5% 回帖咒术+1、发帖咒术+2▕▏升级条件：消耗150血液\n【等级4】6% 回帖咒术+1、发帖咒术+2▕▏升级条件：消耗88咒术\n【等级5】8% 回帖咒术+1、发帖咒术+2▕▏升级条件：咒术≥130\n【 Max 】15% 回帖血液+2、发帖咒术+3",
  "威克多尔·克鲁姆": "威克多尔·克鲁姆\n【勋章类型】真人男从\n【入手条件】血液≥100\n【商店售价】300金币\n【等级1】6% 回帖金币+1、发帖金币+1 咒术+1▕▏升级条件：消耗100金币\n【等级2】8% 回帖金币+1、发帖金币+1 咒术+1▕▏升级条件：消耗150金币\n【等级3】10% 回帖金币+1、发帖金币+1 咒术+1▕▏升级条件：消耗30咒术\n【 Max 】10% 回帖金币+2、发帖金币+2 咒术+1",
  "赫敏·格兰杰": "赫敏·格兰杰\n【勋章类型】女从\n【入手条件】发帖数≥30\n【商店售价】300金币\n【等级1】6% 回帖血液+1、发帖 血液+1 咒术+1▕▏升级条件：消耗100血液\n【等级2】8% 回帖血液+1、发帖血液+1 咒术+1▕▏升级条件：消耗150血液\n【等级3】10% 回帖血液+1、发帖血液+1 咒术+1▕▏升级条件：消耗30咒术\n【 Max 】10% 回帖血液+2、发帖血液+2 咒术+1",
  "阿拉贡": "阿拉贡\n【勋章类型】真人男从\n【入手条件】主题数≥15\n【商店售价】680金币\n【等级1】10% 回帖血液+1、发帖血液+3▕▏升级条件：消耗365血液\n【等级2】12% 回帖血液+1、发帖血液+3▕▏升级条件：旅程≥88\n【等级3】12% 回帖血液+2、发帖旅程+1▕▏升级条件：追随≥365\n【等级4】16% 回帖血液+2、发帖旅程+1▕▏升级条件：消耗1413金币\n【 Max 】18% 回帖血液+3、发帖旅程+1",
  "瑟兰迪尔": "瑟兰迪尔\n【勋章类型】真人男从\n【入手条件】在线时间≥99小时\n【商店售价】400金币\n【等级1】5% 回帖金币+1、发帖金币+3▕▏升级条件：消耗130金币\n【等级2】8% 回帖金币+1、发帖金币+3▕▏升级条件：消耗130血液\n【等级3】10% 回帖金币+1、发帖金币+3▕▏升级条件：知识≥40\n【 Max 】10% 回帖金币+2、发帖金币+3",
  "异形": "异形\n【勋章类型】真人男从\n【入手条件】堕落≥20\n【商店售价】580金币\n【等级1】6% 回帖血液+1▕▏升级条件：堕落≥44\n【等级2】8% 回帖血液+1▕▏升级条件：消耗188血液\n【等级3】10% 回帖血液+2▕▏升级条件：追随≥108\n【等级4】12% 回帖血液+2 金币-1▕▏升级条件：金币＞999\n【 Max 】15% 回帖血液+3 金币-1、发帖旅程+1",
  "岛田源氏": "岛田源氏\n【勋章类型】游戏男从\n【入手条件】旅程≥20\n【商店售价】600金币\n【等级1】6% 回帖血液+1、发帖血液+1▕▏升级条件：消耗300血液\n【等级2】8% 回帖血液+1、发帖血液+1▕▏升级条件：消耗300金币\n【等级3】10% 回帖血液+1、发帖血液+2▕▏升级条件：旅程≥60\n【等级4】11% 回帖血液+1、发帖血液+3▕▏升级条件：消耗300血液\n【等级5】13% 回帖血液+2、发帖血液+4▕▏升级条件：知识≥130\n【 Max 】15% 回帖血液+3、发帖血液+5",
  "小天狼星·布莱克": "小天狼星·布莱克\n【勋章类型】真人男从\n【入手条件】堕落≤30\n【商店售价】520金币\n【等级1】10% 回帖血液+1、发帖血液+3▕▏升级条件：消耗123咒术\n【等级2】8% 回帖血液-1 咒术+1、发帖血液-3 咒术+3▕▏升级条件：血液≥520\n【 Max 】10% 回帖咒术+1、发帖知识+1",
  "甘道夫": "甘道夫\n【勋章类型】真人男从\n【入手条件】知识≥18\n【商店售价】700金币\n【等级1】8% 回帖血液+1 、发帖知识+1▕▏升级条件：旅程≥60\n【等级2】10% 发帖血液+2 知识+1▕▏升级条件：灵魂≥2\n【等级3】1% 发帖灵魂+1▕▏升级条件：消耗1点堕落\n【等级4】12% 回帖血液+2、发帖咒术+1▕▏升级条件：消耗111咒术\n【 Max 】15% 回帖血液+3、发帖咒术+2",
  "莱戈拉斯": "莱戈拉斯\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】300金币\n【等级1】5% 回帖金币+1、发帖血液+1▕▏升级条件：消耗50金币\n【等级2】6% 回帖金币+1、发帖血液+2▕▏升级条件：消耗50血液\n【等级3】7% 回帖金币+2、发帖血液+2▕▏升级条件：消耗100金币\n【 Max 】8% 回帖金币+2、发帖血液+3",
  "神灯": "神灯\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】300金币\n【等级1】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗100金币\n【等级2】8% 回帖血液+1、发帖血液+1▕▏升级条件：消耗150金币\n【等级3】10% 回帖血液+1、发帖血液+1▕▏升级条件：消耗200金币\n【等级4】12% 回帖血液+2、发帖血液+2▕▏升级条件：消耗300金币\n【 Max 】8% 回帖血液+2 咒术+1、发帖血液+2 咒术+1",
  "黑豹": "黑豹\n【勋章类型】真人男从\n【入手条件】知识≥5\n【商店售价】320金币\n【等级1】6% 回帖血液+1 堕落-1、发帖咒术+1▕▏升级条件：消耗100血液\n【等级2】8% 回帖血液+1 堕落-1、发帖咒术+1▕▏升级条件：消耗200金币\n【等级3】10% 回帖血液+1 堕落-1、发帖咒术+1▕▏升级条件：消耗50咒术\n【 Max 】12% 回帖血液+2 堕落-1、发帖咒术+1",
  "莱托·厄崔迪": "莱托·厄崔迪\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】350金币\n【等级1】6% 回帖血液+2▕▏升级条件：消耗50金币\n【等级2】8% 回帖血液+2、发帖旅程+1▕▏升级条件：消耗50血液\n【等级3】10% 回帖血液+2、发帖旅程+1▕▏升级条件：消耗150金币\n【 Max 】12% 回帖血液+2、发帖知识+1",
  "西弗勒斯·斯内普": "西弗勒斯·斯内普\n【勋章类型】真人男从\n【入手条件】旅程≥38\n【商店售价】666金币\n【等级1】12% 回帖血液+1、发帖旅程+1▕▏升级条件：消耗99血液\n【等级2】10% 回帖血液+2、发帖旅程+1▕▏升级条件：消耗99金币\n【等级3】8% 回帖血液+3、发帖旅程+1▕▏升级条件：消耗1314血液\n【等级4】6% 回帖旅程+1▕▏升级条件：金币≥999\n【 Max 】2% 回帖旅程+1、发帖灵魂+1",
  "阿不思·邓布利多": "阿不思·邓布利多\n【勋章类型】真人男从\n【入手条件】旅程≥10\n【商店售价】250金币\n【等级1】6% 回帖血液+1、发帖血液+3▕▏升级条件：消耗333金币\n【等级2】8% 回帖血液+1、发帖血液+3▕▏升级条件：消耗444血液\n【等级3】10% 回帖血液+2、发帖血液+3▕▏升级条件：消耗77咒术\n【等级4】15% 回帖血液+2、发帖知识+1▕▏升级条件：旅程≥115\n【 Max 】16% 回帖血液+3、发帖知识+1",
  "普隆普特·阿金塔姆": "普隆普特·阿金塔姆\n【勋章类型】游戏男从\n【入手条件】知识≥10\n【商店售价】500金币\n【等级1】3% 回帖金币+1、发帖旅程+1▕▏升级条件：知识≥30\n【等级2】6% 回帖金币+2、发帖旅程+1▕▏升级条件：消耗100金币\n【等级3】9% 回帖金币+2、发帖旅程+1▕▏升级条件：消耗200金币\n【等级4】12% 回帖金币+2、发帖旅程+1▕▏升级条件：咒术≥188\n【 Max 】15% 回帖金币+2、发帖旅程+1",
  "诺克提斯·路西斯·伽拉姆": "诺克提斯·路西斯·伽拉姆\n【勋章类型】游戏男从\n【入手条件】旅程≥15\n【商店售价】666金币\n【等级1】4% 回帖血液+1、发帖旅程+1▕▏升级条件：旅程≥30\n【等级2】8% 回帖血液+1、发帖旅程+1▕▏升级条件：消耗200血液\n【等级3】12% 回帖血液+1、发帖旅程+1▕▏升级条件：消耗350血液\n【等级4】15% 回帖血液+2、发帖旅程+1▕▏升级条件：消耗1灵魂\n【 Max 】18% 回帖血液+3、发帖旅程+1",
  "豹王": "豹王\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】2% 回帖金币+2▕▏升级条件：追随≥50\n【等级2】5% 回帖金币+2▕▏升级条件：消耗100金币\n【等级3】8% 回帖金币+2、发帖旅程+1▕▏升级条件：消耗100血液\n【 Max 】11% 回帖金币+2、发帖旅程+1",
  "不灭狂雷-沃利贝尔": "不灭狂雷-沃利贝尔\n【勋章类型】游戏男从\n【入手条件】堕落≥20\n【商店售价】500金币\n【等级1】2% 回帖血液+1、发帖血液+2 堕落+1▕▏升级条件：消耗200血液\n【等级2】5% 回帖血液+1 堕落+1、发帖血液+3 堕落+1▕▏升级条件：咒术≥100\n【 Max 】10% 回帖血液+2 堕落+1、发帖血液+3 堕落+1",
  "桐生一马": "桐生一马\n【勋章类型】游戏男从\n【入手条件】追随≥20\n【商店售价】500金币\n【等级1】8% 回帖金币+1▕▏升级条件：消耗200血液\n【等级2】8% 回帖金币+1 血液+1▕▏升级条件：追随≥88\n【 Max 】15% 回帖金币+1 血液+1",
  "大黄蜂（ChevroletCamaro）": "大黄蜂（ChevroletCamaro）\n【勋章类型】真人男从\n【入手条件】旅程≥10\n【商店售价】300金币\n【等级1】4% 回帖金币+1 血液+1、发帖咒术+1▕▏升级条件：消耗100金币\n【等级2】6% 回帖金币+1 血液+1、发帖咒术+1▕▏升级条件：消耗100血液\n【等级3】8% 回帖金币+1 血液+1、发帖咒术+1▕▏升级条件：消耗30咒术\n【 Max 】10% 回帖血液+1 金币+1、发帖咒术+1",
  "博伊卡": "博伊卡\n【勋章类型】真人男从\n【入手条件】追随≥20\n【商店售价】550金币\n【等级1】5% 回帖金币+1、发帖金币+2▕▏升级条件：消耗200血液\n【等级2】10% 回帖金币+1、发帖金币+2▕▏升级条件：消耗400血液\n【等级3】10% 回帖金币+2 血液-1、发帖金币+3 血液-1▕▏升级条件：消耗-100金币\n【等级4】13% 回帖金币+3 血液-1、发帖金币+5 血液-1▕▏升级条件：消耗600血液\n【 Max 】15% 回帖金币+3、发帖金币+5",
  "史蒂文·格兰特": "史蒂文·格兰特\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】500金币\n【等级1】4% 回帖金币+1、发帖血液+1▕▏升级条件：消耗100血液\n【等级2】6% 回帖金币+1、发帖血液+1▕▏升级条件：消耗150金币\n【等级3】8% 回帖金币+1、发帖血液+1▕▏升级条件：消耗200血液\n【等级4】10% 回帖金币+2、发帖血液+2▕▏升级条件：消耗251金币\n【等级5】12% 回帖金币+2、发帖血液+2▕▏升级条件：消耗400血液\n【 Max 】14% 回帖金币+3、发帖血液+3",
  "马克·史贝特": "马克·史贝特\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】500金币\n【等级1】4% 回帖血液+1、发帖金币+1▕▏升级条件：消耗100金币\n【等级2】6% 回帖血液+1、发帖金币+1▕▏升级条件：消耗150血液\n【等级3】8% 回帖血液+1、发帖金币+1▕▏升级条件：消耗200金币\n【等级4】10% 回帖血液+2、发帖金币+2▕▏升级条件：消耗251血液\n【等级5】12% 回帖血液+2、发帖金币+2▕▏升级条件：消耗400金币\n【 Max 】14% 回帖血液+3、发帖金币+3",
  "果体76": "果体76\n【勋章类型】游戏男从\n【入手条件】活动产出\n【商店售价】766\n【 Max 】76% 回帖血液+1",
  "竹村五郎": "竹村五郎\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】400金币\n【等级1】5% 回帖咒术+1▕▏升级条件：消耗50咒术\n【等级2】8% 回帖咒术+1▕▏升级条件：消耗100咒术\n【 Max 】10% 回帖咒术+1",
  "光之战士": "光之战士\n【勋章类型】游戏男从\n【入手条件】旅程≥15\n【商店售价】600金币\n【等级1】5% 回帖金币+1 发帖血液+2▕▏升级条件：消耗200金币\n【等级2】8% 回帖金币+2 发帖血液+3▕▏升级条件：消耗300金币\n【等级3】10% 回帖金币+2 发帖血液+3▕▏升级条件：消耗300血液\n【等级4】12% 回帖血液+2 发帖金币+3▕▏升级条件：消耗400金币\n【等级5】4% 回帖旅程+1 发帖金币+3▕▏升级条件：消耗500金币\n【 Max 】5% 回帖旅程+1 发帖金币+5",
  "Drover": "Drover\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】380金币\n【等级1】4% 回帖血液+1、发帖血液+1▕▏升级条件：消耗80血液\n【等级2】4% 回帖血液+2、发帖血液+2▕▏升级条件：消耗180血液\n【等级3】6% 回帖血液+2、发帖血液+2▕▏升级条件：消耗280血液\n【 Max 】8% 回帖血液+3、发帖血液+3",
  "乔治·迈克尔": "乔治·迈克尔\n【勋章类型】真人男从\n【入手条件】旅程≥30\n【商店售价】600金币\n【等级1】6% 回帖血液+2、发帖血液+5▕▏升级条件：追随≥99\n【等级2】9% 回帖血液+2、发帖血液+5▕▏升级条件：在线时间≥666\n【等级3】12% 回帖血液+2、发帖血液+5▕▏升级条件：消耗666金币\n【等级4】15% 回帖血液+2、发帖血液+5▕▏升级条件：消耗888血液\n【 Max 】18% 回帖血液+3、发帖血液+5",
  "荒野大镖客：救赎 II": "荒野大镖客：救赎 II\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】1% 回帖旅程+1",
  "雾都血医": "雾都血医\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖血液+1",
  "寶可夢 Pokémon": "寶可夢 Pokémon\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖金币+1",
  "最终幻想XIV": "最终幻想XIV\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】1% 回帖旅程+1",
  "赛博朋克2077": "赛博朋克2077\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】1% 回帖旅程+1",
  "TRPG版塊": "TRPG版塊\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖咒术+1",
  "恶魔城": "恶魔城\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖血液+1",
  "英雄联盟": "英雄联盟\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖金币+1",
  "男巫之歌": "男巫之歌\n【勋章类型】板块\n【入手条件】金币≥888 追随≥100 知识≥20\n【商店售价】100金币\n【等级1】无属性▕▏升级条件：消耗150金币\n【等级2】无属性▕▏升级条件：消耗150金币\n【等级3】无属性▕▏升级条件：消耗150金币\n【等级4】无属性▕▏升级条件：消耗150金币\n【等级5】无属性▕▏升级条件：消耗150金币\n【等级6】无属性▕▏升级条件：消耗150金币\n【等级7】无属性▕▏升级条件：消耗-1旅程\n【等级8】15% 回帖血液+1、发帖血液+1▕▏升级条件：消耗1灵魂\n【 Max 】50% 回帖血液+1、发帖血液+2",
  "男色诱惑": "男色诱惑\n【勋章类型】天赋\n【入手条件】堕落≥100、知识≥50\n【商店售价】无\n【等级1】11% 发帖血液+3▕▏升级条件：堕落≥200\n【等级2】22% 发帖血液+4▕▏升级条件：堕落≥300\n【 Max 】33% 发帖血液+5",
  "美恐：启程": "美恐：启程\n【勋章类型】板块\n【入手条件】旅程≥15、知识≥15、血液≥200\n【商店售价】100金币\n【等级1】1% 回帖旅程+1▕▏升级条件：消耗1旅程\n【等级2】2% 回帖血液+1▕▏升级条件：消耗50血液\n【等级3】1% 回帖知识+1▕▏升级条件：消耗1旅程\n【等级4】1% 回帖血液+2▕▏升级条件：消耗50金币\n【等级5】1% 回帖旅程+1▕▏升级条件：消耗1旅程\n【等级6】1% 回帖血液+2▕▏升级条件：消耗50血液\n【等级7】2% 回帖血液+1▕▏升级条件：消耗50金币\n【等级8】1% 回帖知识+1▕▏升级条件：消耗-1知识\n【等级9】2% 回帖血液+2▕▏升级条件：消耗-3旅程\n【 Max 】3% 回帖旅程+1",
  "街头霸王": "街头霸王\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【等级1】2% 回帖金币+1",
  "海边的邻居": "海边的邻居\n【勋章类型】天赋\n【入手条件】金币≥1500、追随≥150\n【商店售价】无\n【等级1】5% 发帖咒术+1▕▏升级条件：在线时间≥888\n【等级2】15% 发帖咒术+1▕▏升级条件：知识≥60\n【 Max 】30% 发帖咒术+1",
  "四季之歌": "四季之歌\n【勋章类型】天赋\n【入手条件】旅程≥40、知识≥40、追随≥40、咒术≥40\n【商店售价】无\n【等级1】10% 回帖血液+1、发帖咒术+2▕▏升级条件：消耗1咒术\n【等级2】10% 回帖金币+1、发帖旅程+1▕▏升级条件：消耗1咒术\n【等级3】10% 回帖血液+1、发帖知识+1▕▏升级条件：消耗1咒术\n【 Max 】10% 回帖金币+1、发帖知识+1",
  "风雪之家": "风雪之家\n【勋章类型】天赋\n【入手条件】注册天数>=365、追随>=50\n【商店售价】无\n【 Max 】1% 回帖血液+5 金币+5、发帖灵魂+1",
  "龙鳞石": "龙鳞石\n【勋章类型】奖品\n【入手条件】春节活动获取\n【商店售价】无\n【 Max 】1% 发帖金币+6",
  "『不败之花』": "『不败之花』\n【勋章类型】剧情\n【入手条件】灵魂>=1并且堕落>=1\n【商店售价】1血液\n【 Max 】无属性",
  "约书亚・罗兹菲尔德": "约书亚・罗兹菲尔德\n【勋章类型】游戏男从\n【入手条件】堕落>=160\n【商店售价】500金币\n【等级1】1% 回帖血液+1▕▏升级条件：1600金币\n【等级2】2% 回帖血液+1 堕落+2、发帖灵魂+1 堕落+3▕▏升级条件：堕落>=16\n【等级3】10% 回帖血液+3 堕落+1、发帖血液+3▕▏升级条件：堕落>=32\n【等级4】10% 回帖血液+2、发帖血液+2▕▏升级条件：堕落>=80\n【等级5】6% 回帖堕落-1、发帖堕落-1▕▏升级条件：堕落>=160\n【等级6】8% 回帖堕落-1、发帖堕落-1▕▏升级条件：堕落>=250\n【 Max 】10% 回帖堕落-2、发帖堕落-2",
  "克莱夫・罗兹菲尔德": "克莱夫・罗兹菲尔德\n【勋章类型】游戏男从\n【入手条件】主题数>=16\n【商店售价】622金币\n【等级1】3% 回帖金币+1▕▏升级条件：消耗320金币\n【等级2】5% 回帖金币+1▕▏升级条件：消耗480血液\n【等级3】7% 回帖金币+2、发帖咒术+1▕▏升级条件：消耗640血液\n【等级4】9% 回帖金币+2、发帖咒术+1▕▏升级条件：消耗640血液\n【等级5】13% 回帖金币+2、发帖咒术+1▕▏升级条件：消耗1灵魂\n【 Max 】16% 回帖金币+2 血液+1、发帖咒术+1",
  "苇名弦一郎": "苇名弦一郎\n【勋章类型】游戏男从\n【入手条件】追随>=15\n【商店售价】666金币\n【等级1】3% 回帖咒术+1、发帖旅程+1▕▏升级条件：消耗666血液\n【等级2】5% 回帖咒术+1、发帖旅程+1▕▏升级条件：消耗999血液\n【等级3】8% 回帖咒术+1、发帖旅程+1▕▏升级条件：消耗169咒术\n【 Max 】5% 回帖旅程+1、发帖咒术+2",
  "里昂（RE4）": "里昂（RE4）\n【勋章类型】游戏男从\n【入手条件】主题数>=5\n【商店售价】600金币\n【等级1】3% 回帖金币+1▕▏升级条件：消耗300血液\n【等级2】5% 回帖金币+1 发帖咒术+1▕▏升级条件：消耗600血液\n【等级3】10% 回帖金币+2 发帖咒术+1▕▏升级条件：消耗600金币\n【等级4】18% 回帖金币+3▕▏升级条件：金币≥444\n【 Max 】12% 回帖金币+3 发帖咒术+1",
  "尼克·王尔德": "尼克·王尔德\n【勋章类型】真人男从\n【入手条件】堕落≥30\n【商店售价】500金币\n【等级1】5% 回帖血液+1、发帖金币+1▕▏升级条件：知识≥50\n【等级2】7% 回帖血液+1、发帖金币+1▕▏升级条件：消耗550血液\n【等级3】10% 回帖血液+2、发帖金币+2▕▏升级条件：消耗600金币\n【 Max 】13% 回帖血液+2 金币+1、发帖金币+1",
  "极客的晚宴": "极客的晚宴\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 发帖知识+1",
  "雄躯的昇格": "雄躯的昇格\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 发帖知识+1",
  "波纹蓝蛋": "波纹蓝蛋\n【勋章类型】宠物\n【入手条件】旅程>=25\n【商店售价】500金币\n【等级1】1% 回帖知识+1▕▏升级条件：消耗400血液\n【等级2】2% 回帖知识+1、发帖知识+1▕▏升级条件：消耗800血液\n【 Max 】4% 回帖知识+1、发帖知识+1",
  "崩朽龙卵": "崩朽龙卵\n【勋章类型】宠物\n【入手条件】主题数>=10\n【商店售价】500金币\n【等级1】3% 回帖咒术+1▕▏升级条件：消耗400血液\n【等级2】5% 回帖咒术+1▕▏升级条件：消耗800血液\n【 Max 】2% 回帖咒术+1、发帖灵魂+1",
  "冰海钓竿": "冰海钓竿\n【勋章类型】资产\n【入手条件】追随>=30\n【商店售价】500金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗300金币\n【等级2】8% 回帖金币+2▕▏升级条件：消耗600金币\n【等级3】10% 回帖金币+3▕▏升级条件：消耗900金币\n【 Max 】15% 回帖金币+3",
  "射手的火枪": "射手的火枪\n【勋章类型】装备\n【入手条件】咒术≥30\n【商店售价】550金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗100金币\n【等级2】7% 回帖金币+1▕▏升级条件：消耗200金币\n【等级3】5% 回帖金币+2▕▏升级条件：消耗300金币\n【等级4】10% 回帖金币+2▕▏升级条件：消耗650金币\n【 Max 】12% 回帖金币+3",
  "“米凯拉的锋刃”玛莲妮亚": "“米凯拉的锋刃”玛莲妮亚\n【勋章类型】女从\n【入手条件】血液>=200\n【商店售价】480金币\n【等级1】5% 回帖堕落+1、发帖堕落+1▕▏升级条件：堕落≥50\n【等级2】6% 回帖血液+1 堕落+1、发帖堕落+1▕▏升级条件：堕落≥100\n【等级3】6% 回帖血液+2 堕落+1、发帖堕落+1▕▏升级条件：消耗222血液\n【等级4】8% 回帖血液+3、发帖堕落+1▕▏升级条件：消耗444血液\n【等级5】10% 回帖堕落-1▕▏升级条件：消耗111咒术\n【 Max 】10% 回帖血液+3 堕落+1、发帖血液+3 堕落+1",
  "朱迪·霍普斯": "朱迪·霍普斯\n【勋章类型】女从\n【入手条件】知识>=15\n【商店售价】500金币\n【等级1】5% 回帖金币+1、发帖血液+1▕▏升级条件：知识≥50\n【等级2】7% 回帖金币+1、发帖血液+1▕▏升级条件：消耗550金币\n【等级3】10% 回帖金币+2、发帖血液+2▕▏升级条件：消耗600血液\n【 Max 】13% 回帖金币+2 血液+1、发帖旅程+1",
  "征服之王": "征服之王\n【勋章类型】奖品\n【入手条件】2024年论坛活动【七日之屿】资源数排名前列\n【商店售价】无\n【 Max 】10% 回帖金币+1 血液-1",
  "岛屿探险家": "岛屿探险家\n【勋章类型】奖品\n【入手条件】2024年论坛活动【七日之屿】全程参与\n【商店售价】无\n【 Max 】1% 回帖金币+1、发帖旅程+1",
  "『列车长』": "『列车长』\n【入手条件】【极地特快】活动中，拼车成功的列车长奖励等级\n【商店售价】无\n【等级 初级】无属性▕▏升级条件：好友数≥6\n【等级1】1% 回帖旅程+1 血液+1、发帖旅程+1 血液+1▕▏升级条件：消耗12旅程\n【等级2】1% 回帖旅程+1 血液+1、发帖旅程+1 血液+1▕▏升级条件：消耗-12旅程\n【等级3】1% 回帖旅程+1 血液+1、发帖旅程+1 血液+1▕▏升级条件：灵魂≥1\n【 Max 】1% 回帖旅程+1 血液+1、发帖旅程+1 血液+1",
  "特供热巧": "特供热巧\n【勋章类型】奖品\n【入手条件】【极地特快】活动中，拼车成功的车厢乘务员奖励\n【商店售价】无\n【等级1】1% 回帖血液+1、发帖血液+1▕▏升级条件：消耗1金币\n【 Max 】1% 回帖血液+1、发帖血液+1",
  "六出冰花": "六出冰花\n【勋章类型】奖品\n【入手条件】【雪中飞舞】发帖奖励、【极地特快】未拼车成功的列车长和车厢乘务员留念\n【商店售价】无\n【 Max 】1% 发帖旅程+1",
  "『金色车票』": "『金色车票』\n【入手条件】2023年【极地特快】活动隐藏奖励，成功触发隐藏剧情：平安夜前发出“相信”的声音\n【商店售价】无\n【等级 初级】无属性▕▏升级条件：消耗1旅程\n【等级1】无属性▕▏升级条件：消耗-1旅程\n【等级2】无属性▕▏升级条件：灵魂≥1\n【 Max 】无属性",
  "最终幻想XVI": "最终幻想XVI\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】1% 回帖旅程+1",
  "可疑的肉蛋": "可疑的肉蛋\n【勋章类型】宠物\n【入手条件】堕落≥50\n【商店售价】500金币\n【等级1】 3% 回帖堕落+1▕▏升级条件：消耗49咒术\n【等级2】 5% 回帖堕落+1▕▏升级条件：消耗-1知识\n【等级3】 7% 回帖堕落+1▕▏升级条件：知识≥49\n【等级4】10% 回帖堕落+1▕▏升级条件：消耗1灵魂\n【 Max 】2% 回帖知识+1、发帖灵魂+1",
  "无垠": "无垠\n【勋章类型】资产\n【入手条件】无\n【商店售价】2000金币\n【等级1】1% 回帖血液+1、发帖血液+1▕▏升级条件:血液≥1\n【等级2】1% 回帖旅程+1、发帖旅程+1▕▏升级条件:堕落≥800\n【等级3】无属性▕▏升级条件:消耗1血液\n【等级4】5% 回帖旅程+1、发帖旅程+1▕▏升级条件:堕落≥100\n【等级5】4% 回帖旅程+1 血液+1、发帖旅程+1 血液+1▕▏升级条件:堕落≥200\n【等级6】3% 回帖旅程+1 血液+1、发帖旅程+1 血液+1▕▏升级条件:堕落≥300\n【等级7】3% 回帖旅程+1、发帖旅程+1▕▏升级条件:堕落≥400\n【等级8】2% 回帖旅程+1 血液+1、发帖旅程+1 血液+1▕▏升级条件:堕落≥500\n【等级9】4% 回帖旅程+1、发帖旅程+1▕▏升级条件:堕落≥600\n【等级10】2% 回帖旅程+1、发帖旅程+1▕▏升级条件:堕落≥700\n【 Max 】1% 回帖血液+1 旅程+1、发帖血液+1 旅程+1",
  "黑暗水晶": "黑暗水晶\n【勋章类型】资产\n【入手条件】堕落≥36\n【商店售价】300金币\n【等级1】6% 回帖堕落+1、发帖堕落+1▕▏升级条件：消耗200金币\n【等级2】6% 回帖堕落+1 金币+1、发帖堕落+1 金币+1▕▏升级条件：消耗200血液\n【等级3】12% 回帖堕落+1 金币+1、发帖堕落+1 金币+1▕▏升级条件：消耗66咒术\n【等级4】12% 回帖堕落+2 金币+1、发帖堕落+2 金币+1▕▏升级条件：灵魂≥1\n【等级5】18% 回帖金币+1、发帖金币+1",
  "棱镜": "棱镜\n【勋章类型】装备\n【入手条件】咒术≥20\n【商店售价】666金币\n【等级1】10% 回帖金币+2▕▏升级条件：堕落≥100\n【 Max 】12% 回帖堕落-1",
  "天使之赐": "天使之赐\n【勋章类型】装备\n【入手条件】堕落≥30\n【商店售价】288金币\n【等级1】4% 回帖咒术+1、发帖咒术+1▕▏升级条件：消耗48咒术\n【等级2】10% 回帖金币+2▕▏升级条件：消耗300金币\n【 Max 】8% 回帖咒术+1、发帖咒术+1",
  "爱丽丝·盖恩斯巴勒": "爱丽丝·盖恩斯巴勒\n【勋章类型】女从\n【入手条件】知识≥30\n【商店售价】400金币\n【等级1】5% 回帖金币+1、发帖金币+1▕▏升级条件：消耗30咒术\n【等级2】5% 发帖金币+2▕▏升级条件：消耗400血液\n【等级3】8% 回帖金币+2、发帖金币+2▕▏升级条件：消耗66咒术\n【等级4】14% 回帖血液+3▕▏升级条件：血液≥199\n【等级5】9% 回帖金币+3、发帖金币+3▕▏升级条件：消耗1金币\n【 Max 】9% 回帖金币+3、发帖金币+3▕▏升级条件：消耗1金币",
  "凯特尼斯·伊夫狄恩": "凯特尼斯·伊夫狄恩\n【勋章类型】女从\n【入手条件】堕落≥10\n【商店售价】450金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗150血液\n【等级2】勋章博物馆资料暂缺\n【等级3】28% 回帖血液-1 堕落+1▕▏升级条件：堕落≥444\n【 Max 】18% 回帖金币+1",
  "克劳斯·迈克尔森": "克劳斯·迈克尔森\n【勋章类型】真人男从\n【入手条件】堕落≥5\n【商店售价】516金币\n【等级1】10% 回帖堕落+1▕▏升级条件：消耗111血液\n【等级2】5% 回帖血液+1▕▏升级条件：消耗222血液\n【等级3】8% 回帖血液+1▕▏升级条件：消耗333血液\n【等级4】10% 回帖血液+2、发帖金币+1▕▏升级条件：金币≥920\n【等级5】11% 回帖血液+2、发帖金币+2▕▏升级条件：血液≥920\n【等级6】12% 回帖血液+2、发帖金币+3▕▏升级条件：消耗1灵魂\n【 Max 】15% 回帖血液+3、发帖金币+5",
  "虎头怪": "虎头怪\n【勋章类型】游戏男从\n【入手条件】旅程≥20\n【商店售价】500金币\n【等级1】8% 回帖血液+1、发帖旅程+1▕▏升级条件：追随≥50\n【等级2】1% 回帖旅程+1、发帖旅程+1▕▏升级条件：消耗600血液\n【等级3】2% 回帖旅程+1、发帖旅程+1▕▏升级条件：知识≥60\n【等级4】3% 回帖旅程+1、发帖旅程+1▕▏升级条件：消耗1灵魂\n【等级5】4% 回帖旅程+1、发帖旅程+1▕▏升级条件：在线时间≥2400\n【 Max 】2% 回帖旅程+1、发帖灵魂+1",
  "重建熊屋": "重建熊屋\n【勋章类型】奖品\n【入手条件】【丰饶山麓】重建村镇大厅的兑换奖品\n【商店售价】无\n【 Max 】2% 回帖金币+1",
  "图腾饼干": "图腾饼干\n【勋章类型】奖品\n【入手条件】参与【丰饶山麓】救援重建熊人村的谢礼\n【商店售价】无\n【 Max 】1% 回帖血液+2",
  "龙之秘宝": "龙之秘宝\n【勋章类型】奖品\n【入手条件】在【秋园庆丰】活动第一阶段的发帖中消耗-10追随\n【商店售价】无\n【等级1】无属性▕▏升级条件：消耗1血液\n【等级2】2% 发帖堕落+1、回帖堕落+1▕▏升级条件：消耗1血液\n【等级3】2% 发帖血液+1、回帖血液+1▕▏升级条件：消耗1血液\n【等级4】2% 发帖金币+1、回帖金币+1▕▏升级条件：消耗1血液\n【 Max 】2% 发帖咒术+1、回帖咒术+1",
  "长花的蛋": "长花的蛋\n【勋章类型】宠物\n【入手条件】知识≥10\n【商店售价】366金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗72咒术\n【等级2】10% 回帖金币+1▕▏升级条件：咒术≥36\n【 Max 】15% 回帖金币+2",
  "棕色条纹蛋": "棕色条纹蛋\n【勋章类型】宠物\n【入手条件】追随≥35\n【商店售价】520金币\n【等级1】1% 回帖咒术+1▕▏升级条件：消耗80血液\n【等级2】3% 回帖咒术+1▕▏升级条件：消耗150血液\n【等级3】5% 回帖咒术+1▕▏升级条件：消耗-1旅程\n【等级4】8% 回帖咒术+1▕▏升级条件：消耗150金币\n【等级5】3% 回帖知识+1▕▏升级条件：在线时间≥1999\n【 Max 】1% 回帖知识+1、发帖灵魂+1",
  "被尘封之书": "被尘封之书\n【勋章类型】资产\n【入手条件】堕落≥99\n【商店售价】999金币\n【等级1】3% 回帖血液+2▕▏升级条件：消耗1000血液\n【等级2】6% 回帖血液+2▕▏升级条件：消耗1000血液\n【等级3】勋章博物馆资料暂缺\n【等级4】12% 回帖血液+2▕▏升级条件：堕落≥200\n【等级5】15% 回帖血液+2▕▏升级条件：堕落≥300\n【等级6】18% 回帖血液+2▕▏升级条件：堕落≥400\n【等级7】21% 回帖血液+2▕▏升级条件：堕落≥500\n【等级8】24% 回帖血液+2▕▏升级条件：堕落≥600\n【等级9】27% 回帖血液+2▕▏升级条件：堕落≥700\n【等级10】30% 回帖血液+2▕▏升级条件：堕落≥800\n【等级11】35% 回帖血液+2▕▏升级条件：消耗1灵魂\n【 Max 】50% 回帖血液+2",
  "和谐圣杯": "和谐圣杯\n【勋章类型】装备\n【入手条件】知识≥15\n【商店售价】300金币\n【等级1】5% 回帖堕落-1、发帖堕落-2▕▏升级条件：消耗88咒术\n【等级2】10% 回帖堕落-2、发帖堕落-3▕▏升级条件：消耗400血液\n【 Max 】10% 回帖血液+2 堕落+1、发帖血液+3 堕落+3",
  "双项圣杯": "双项圣杯\n【勋章类型】资产\n【入手条件】咒术＞=20\n【商店售价】100金币\n【等级1】无属性▕▏升级条件：消耗1金币\n【等级2】1% 回帖血液+1、发帖血液+1▕▏升级条件：消耗1金币\n【等级3】1% 回帖咒术+1、发帖咒术+1▕▏升级条件：消耗10金币\n【等级4】2% 回帖血液+2、发帖血液+2▕▏升级条件：消耗10金币\n【等级5】2% 回帖咒术+2、发帖咒术+2▕▏升级条件：金币≥1000\n【等级6】8% 回帖血液+1、发帖血液+1▕▏升级条件：消耗1000金币\n【 Max 】8% 回帖血液+2 咒术+2、发帖血液+2 咒术+2",
  "露娜弗蕾亚·诺克斯·芙尔雷": "露娜弗蕾亚·诺克斯·芙尔雷\n【勋章类型】女从\n【入手条件】知识≥20\n【商店售价】500金币\n【等级1】4% 回帖咒术+1、发帖血液+1▕▏升级条件：消耗150金币\n【等级2】6% 回帖血液+1、发帖血液+2▕▏升级条件：消耗200金币\n【等级3】8% 回帖血液+1、发帖血液+2▕▏升级条件：消耗300金币\n【等级4】10% 回帖血液+2、发帖血液+3▕▏升级条件：消耗120咒术\n【 Max 】13% 回帖血液+3、发帖血液+5",
  "凯尔": "凯尔\n【勋章类型】女从\n【入手条件】主题数>=10\n【商店售价】300金币\n【等级1】8% 回帖血液+1、发帖旅程+1▕▏升级条件：灵魂≥1\n【等级2】15% 回帖血液+1、发帖旅程+2▕▏升级条件：消耗300金币\n【等级3】15% 回帖血液+2、发帖旅程+1▕▏升级条件：堕落≥10\n【 Max 】12% 回帖血液+2、发帖旅程+1",
  "莫甘娜": "莫甘娜\n【勋章类型】女从\n【入手条件】主题数>=10\n【商店售价】300金币\n【等级1】8% 回帖金币+1、发帖旅程+1▕▏升级条件：灵魂≥1\n【等级2】10% 回帖金币+1、发帖旅程+1▕▏升级条件：消耗300血液\n【等级3】15% 回帖金币+2、发帖旅程+1▕▏升级条件：堕落≥10\n【 Max 】12% 回帖金币+2、发帖旅程+1",
  "阿尔瓦罗·索莱尔": "阿尔瓦罗·索莱尔\n【勋章类型】真人男从\n【入手条件】追随≥10\n【商店售价】500金币\n【等级1】6% 回帖金币+1▕▏升级条件：消耗50金币\n【等级2】6% 回帖金币+2▕▏升级条件：追随≥50\n【等级3】6% 回帖金币+3▕▏升级条件：追随≥150\n【等级4】8% 回帖金币+3▕▏升级条件：消耗150金币\n【 Max 】12% 回帖金币+2、发帖金币+3",
  "纣王·子受": "纣王·子受\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】500金币\n【等级1】5% 回帖金币+1、发帖金币+2▕▏升级条件：消耗400金币\n【等级2】10% 回帖金币+1、发帖金币+2▕▏升级条件：消耗400血液\n【等级3】10% 回帖金币+1 堕落+1、发帖金币+3▕▏升级条件：消耗100咒术\n【等级4】15% 回帖金币+1 堕落-1、发帖旅程+1▕▏升级条件：消耗1灵魂\n【等级5】20% 回帖金币+2、发帖旅程+1▕▏升级条件：堕落≥100\n【 Max 】3% 回帖旅程+1、发帖旅程+1",
  "阿齐斯": "阿齐斯\n【勋章类型】真人男从\n【入手条件】堕落≥60\n【商店售价】480金币\n【等级1】8% 回帖血液+1、发帖血液+2▕▏升级条件：消耗300金币\n【等级2】10% 回帖血液+2 金币-1、发帖堕落+1▕▏升级条件：消耗600金币\n【等级3】12% 回帖血液+3 金币-1▕▏升级条件：消耗900金币\n【 Max 】18% 回帖血液+3、发帖堕落+1",
  "百相千面": "百相千面\n【勋章类型】游戏男从\n【入手条件】发帖数≥100\n【商店售价】1000金币\n【等级1】5% 回帖金币+1 血液+1、发帖旅程+1▕▏升级条件：消耗100金币\n【等级2】5% 回帖金币+1 血液+1、发帖旅程+1▕▏升级条件：消耗1000血液\n【等级3】5% 回帖金币+2 血液+2、发帖金币+3 血液+3▕▏升级条件：消耗1000金币\n【 Max 】5% 回帖旅程+1、发帖金币+3 血液+3",
  "肉垫手套": "肉垫手套\n【勋章类型】奖品\n【入手条件】参与【河谷寻奇】活动，用70枚游戏金币兑换\n【商店售价】无\n【 Max 】2% 回帖金币+1、发帖金币+1",
  "绿茵宝钻": "绿茵宝钻\n【勋章类型】奖品\n【入手条件】参与【河谷寻奇】活动，获得的游戏金币数量排名前10%\n【商店售价】无\n【 Max 】1% 回帖金币+6、发帖金币+6",
  "猛虎贴贴": "猛虎贴贴\n【勋章类型】奖品\n【入手条件】参与【巧夕之月】，发布的帖子在各板块中追随数排名前五，或者在活动贴留言抽奖中奖\n【商店售价】无\n【等级1】1% 回帖血液+1▕▏升级条件：消耗1血液\n【等级2】3% 回帖血液+1▕▏升级条件：消耗1血液\n【 Max 】5% 回帖血液+1",
  "白巧克力蛋": "白巧克力蛋\n【勋章类型】奖品\n【入手条件】参与【巧夕之月】，发布符合活动主题条件的帖子LV1\n【商店售价】无\n【等级1】1% 回帖金币+1▕▏升级条件：1金币\n【 Max 】3% 回帖金币+1",
  "灵藤蛋": "灵藤蛋\n【勋章类型】宠物\n【入手条件】知识≥5\n【商店售价】350金币\n【等级1】5% 回帖血液+1▕▏升级条件：知识≥30\n【 Max 】3% 回帖知识+1",
  "令人不安的契约书": "令人不安的契约书\n【勋章类型】资产\n【入手条件】咒术≥5\n【商店售价】200金币\n【等级1】1% 回帖咒术+1▕▏升级条件：消耗15咒术\n【等级2】2% 回帖咒术+1▕▏升级条件：消耗-20金币\n【 Max 】3% 回帖咒术+1 金币+1",
  "星籁歌姬": "星籁歌姬\n【勋章类型】女从\n【入手条件】追随≥10\n【商店售价】500金币\n【等级1】3% 回帖血液+1▕▏升级条件：消耗30咒术\n【等级2】6% 回帖血液+1、发帖血液+1▕▏升级条件：消耗270金币\n【等级3】9% 回帖血液+1、发帖血液+2▕▏升级条件：追随≥100\n【等级4】12% 回帖血液+2、发帖血液+3▕▏升级条件：追随≥200\n【 Max 】15% 回帖血液+2、发帖知识+1 血液-3",
  "维涅斯": "维涅斯\n【勋章类型】女从\n【入手条件】旅程≥14\n【商店售价】700金币\n【等级1】4% 回帖咒术+1、发帖知识+1▕▏升级条件：主题数≥14\n【等级2】6% 回帖咒术+1、发帖知识+1▕▏升级条件：消耗140金币\n【等级3】8% 回帖咒术+1、发帖知识+1▕▏升级条件：知识≥140\n【等级4】3% 回帖旅程+1、发帖知识+1▕▏升级条件：消耗1144血液\n【 Max 】2% 回帖知识+1、发帖灵魂+1",
  "刀锋女王": "刀锋女王\n【勋章类型】女从\n【入手条件】咒术≥50\n【商店售价】600金币\n【等级1】3% 回帖咒术+1、发帖咒术+1▕▏升级条件：咒术≥120\n【等级2】6% 回帖咒术+1、发帖咒术+1▕▏升级条件：消耗60咒术\n【等级3】9% 回帖血液+2、发帖旅程+1▕▏升级条件：消耗200金币\n【等级4】12% 回帖血液+2、发帖旅程+1▕▏升级条件：消耗200血液\n【等级5】15% 回帖血液+2、发帖旅程+1▕▏升级条件：消耗350血液\n【 Max 】15% 回帖血液+3、发帖旅程+1",
  "天照大神": "天照大神\n【勋章类型】游戏男从\n【入手条件】追随≥13\n【商店售价】500金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗130血液\n【等级2】7% 回帖金币+1、发帖知识+1▕▏升级条件：血液≥1300\n【等级3】9% 回帖金币+1、发帖知识+1▕▏升级条件：消耗130血液\n【等级4】12% 回帖金币+2、发帖知识+1▕▏升级条件：消耗1300血液\n【 Max 】12% 回帖咒术+1、发帖知识+1",
  "桑克瑞德·沃特斯": "桑克瑞德·沃特斯\n【勋章类型】游戏男从\n【入手条件】旅程≥21\n【商店售价】520金币\n【等级1】1% 回帖血液+7▕▏升级条件：消耗52血液\n【等级2】1% 回帖血液+8▕▏升级条件：堕落≥52\n【等级3】2% 回帖血液+8▕▏升级条件：消耗52咒术\n【等级4】2% 回帖血液+9▕▏升级条件：咒术≥180\n【 Max 】3% 回帖血液+9",
  "『星河碎片』": "『星河碎片』\n【入手条件】灵魂≥1且主题数≥1且知识≥7且旅程≥7\n【商店售价】1知识\n【等级1】无属性▕▏升级条件：消耗-1知识\n【 Max 】无属性",
  "探险三杰士": "探险三杰士\n【勋章类型】奖品\n【入手条件】扮演战士、游侠、法师三大职业，参与【迷翳森林】探险活动\n【商店售价】无\n【 Max 】1% 回帖血液+2",
  "『迷翳森林回忆录』": "『迷翳森林回忆录』\n【勋章类型】剧情\n【入手条件】在『迷翳森林』探险活动中，收集15枚迷翳结晶兑换获得\n【商店售价】无\n【 Max 】2% 回帖金币+2、发帖旅程+1",
  "『迷翳之中』": "『迷翳之中』\n【勋章类型】剧情\n【入手条件】灵魂≥1并且旅程≥10并且主题数≥1\n【商店售价】10旅程\n【等级 初级】无属性▕▏升级条件：消耗-2旅程\n【等级1】无属性▕▏升级条件：消耗-3旅程\n【等级2】无属性▕▏升级条件：消耗-5旅程\n【等级3】无属性▕▏升级条件：旅程≥50\n【等级4】无属性▕▏升级条件：消耗1知识\n【 Max 】无属性",
  "『灰域来音』": "『灰域来音』\n【勋章类型】剧情\n【入手条件】灵魂≥1\n【商店售价】10旅程\n【等级1】无属性▕▏升级条件：消耗-10旅程\n【 Max 】无属性",
  "狱炎蛋": "狱炎蛋\n【勋章类型】宠物\n【入手条件】堕落≥66\n【商店售价】266血液\n【等级1】4% 回帖血液+1 堕落+1、发帖血液+1 堕落+1▕▏升级条件：堕落≥100\n【 Max 】7% 回帖血液+1 堕落+1、发帖血液+1 堕落+1",
  "散佚的文集": "散佚的文集\n【勋章类型】资产\n【入手条件】知识≥10\n【商店售价】250金币\n【等级1】2% 回帖知识+1、发帖知识+1▕▏升级条件：知识≥25\n【等级2】5% 回帖血液+2 堕落-1、发帖血液+2 堕落-1▕▏升级条件：消耗200血液\n【等级3】6% 回帖血液+3 堕落-1、发帖血液+3 堕落-1▕▏升级条件：消耗300血液\n【 Max 】7% 回帖咒术+1 堕落-1、发帖咒术+1 堕落-1",
  "女神之泪": "女神之泪\n【勋章类型】装备\n【入手条件】咒术≥36\n【商店售价】500金币\n【等级1】5% 回帖血液+1、发帖血液+3▕▏升级条件：消耗360血液\n【等级2】10% 回帖血液+2、发帖血液+3▕▏升级条件：咒术≥360\n【 Max 】10% 回帖血液+3、发帖血液+5",
  "希尔瓦娜斯·风行者": "希尔瓦娜斯·风行者\n【勋章类型】女从\n【入手条件】知识≥30\n【商店售价】500金币\n【等级1】3% 回帖血液+2、发帖金币+3▕▏升级条件：追随≥50\n【等级2】6% 回帖血液+2、发帖金币+3▕▏升级条件：在线时间≥666\n【等级3】9% 回帖血液+2、发帖金币+3▕▏升级条件：消耗500金币\n【等级4】12% 回帖血液+2、发帖金币+3▕▏升级条件：消耗666血液\n【 Max 】15% 回帖血液+3、发帖金币+3",
  "死亡": "死亡\n【勋章类型】真人男从\n【入手条件】追随≥10\n【商店售价】444金币\n【等级1】13% 回帖金币-1▕▏升级条件：消耗130血液\n【等级2】13% 回帖血液+1 金币-1▕▏升级条件：知识≥80\n【等级3】13% 回帖血液+1 金币+1、发帖旅程+1▕▏升级条件：消耗1300血液\n【等级4】2% 回帖旅程+1、发帖灵魂+1▕▏升级条件：灵魂≥2\n【 Max 】15% 回帖金币+3、发帖旅程+1",
  "弗图博士": "弗图博士\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】1024金币\n【等级1】10% 回帖金币+1 血液+1▕▏升级条件：旅程≥40\n【等级2】10% 回帖金币+1 血液+1、发帖知识+1▕▏升级条件：堕落≥40\n【等级3】10% 回帖金币+2 血液+1、发帖知识+1▕▏升级条件：咒术≥40\n【 Max 】2% 回帖知识+1 血液+1、发帖灵魂+1",
  "不屈之枪·阿特瑞斯": "不屈之枪·阿特瑞斯\n【勋章类型】游戏男从\n【入手条件】堕落≤450\n【商店售价】450金币\n【等级1】3% 回帖旅程+1▕▏升级条件：旅程≥88\n【等级2】1% 发帖灵魂+1▕▏升级条件：灵魂≥1\n【 Max 】8% 回帖血液+2、发帖旅程+1",
  "【周年限定】克里斯(8)": "【周年限定】克里斯(8)\n【勋章类型】游戏男从\n【入手条件】主题数≥5\n【商店售价】600金币\n【等级1】10% 回帖金币+1▕▏升级条件：消耗200金币\n【等级2】20% 回帖金币+1▕▏升级条件：消耗200血液\n【等级3】30% 回帖金币+1▕▏升级条件：消耗1灵魂\n【 Max 】66% 回帖金币+1",
  "破旧打火机": "破旧打火机\n【勋章类型】装备\n【入手条件】无\n【商店售价】25金币\n【等级1】无属性▕▏升级条件：消耗1金币\n【等级2】无属性▕▏升级条件：消耗1血液\n【等级3】无属性▕▏升级条件：消耗1咒术\n【等级4】无属性▕▏升级条件：消耗1知识\n【等级5】无属性▕▏升级条件：消耗1旅程\n【等级6】5% 回帖金币+1▕▏升级条件：灵魂≥1\n【 Max 】15% 回帖金币+1",
  "山村贞子": "山村贞子\n【勋章类型】女从\n【入手条件】咒术≥44\n【商店售价】300金币\n【等级1】10% 回帖血液+1、发帖咒术+1▕▏升级条件：消耗250血液\n【等级2】15% 回帖血液+1、发帖咒术+1▕▏升级条件：消耗500血液\n【 Max 】15% 回帖血液+2、发帖咒术+2",
  "蒂法·洛克哈特": "蒂法·洛克哈特\n【勋章类型】女从\n【入手条件】堕落≥30\n【商店售价】400金币\n【等级1】1% 回帖旅程+1、发帖旅程+1▕▏升级条件：消耗100血液\n【等级2】2% 回帖旅程+1、发帖旅程+1▕▏升级条件：消耗200血液\n【等级3】3% 回帖旅程+1、发帖旅程+1▕▏升级条件：消耗300血液\n【 Max 】4% 回帖旅程+1、发帖旅程+1",
  "九尾妖狐·阿狸": "九尾妖狐·阿狸\n【勋章类型】女从\n【入手条件】血液≥350\n【商店售价】350金币\n【等级1】4% 回帖血液+1、发帖咒术+1▕▏升级条件：旅程≥89\n【等级2】8% 回帖血液+2、发帖咒术+1▕▏升级条件：消耗89咒术\n【 Max 】13% 回帖血液+2、发帖咒术+2",
  "勒维恩·戴维斯": "勒维恩·戴维斯\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】400金币\n【等级1】10% 无属性▕▏升级条件：消耗1金币\n【等级2】10% 回帖金币+2、发帖金币+2▕▏升级条件：消耗1金币\n【等级3】10% 回帖金币+2、发帖金币+2▕▏升级条件：消耗1金币\n【等级4】10% 回帖金币+2、发帖金币+2▕▏升级条件：消耗1金币\n【等级5】10% 回帖金币+2、发帖金币+2▕▏升级条件：消耗1金币\n【等级6】10% 回帖金币+2、发帖金币+2▕▏升级条件：消耗1金币\n【等级7】10% 回帖金币+2、发帖金币+2▕▏金币＜85\n【等级8】10% 回帖金币+2、发帖金币+2▕▏金币≥85且金币＜233\n【 Max 】10% 回帖金币+2、发帖金币+2▕▏≥233金币",
  "擎天柱（Peterbilt389）": "擎天柱（Peterbilt389）\n【勋章类型】真人男从\n【入手条件】追随≥20\n【商店售价】400金币\n【等级1】4% 回帖血液+1、发帖咒术+1▕▏升级条件：消耗233血液\n【等级2】5% 回帖咒术+1 血液+1、发帖咒术+1▕▏升级条件：消耗233金币\n【等级3】6% 回帖咒术+1 血液+1、发帖咒术+1▕▏升级条件：消耗66咒术\n【等级4】7% 回帖咒术+1 血液+2、发帖旅程+1▕▏升级条件：旅程≥80\n【 Max 】8% 回帖咒术+1 血液+3、发帖旅程+1",
  "丹·雷诺斯": "丹·雷诺斯\n【勋章类型】真人男从\n【入手条件】主题数≥5\n【商店售价】400金币\n【等级1】4% 回帖堕落-1▕▏升级条件：消耗100金币\n【等级2】6% 回帖堕落-1▕▏升级条件：消耗200金币\n【等级3】8% 回帖堕落-1▕▏升级条件：消耗40咒术\n【等级4】11% 回帖堕落-1 血液+2▕▏升级条件：消耗80咒术\n【等级5】13% 回帖堕落-1 血液+2、发帖血液+2▕▏升级条件：追随≥250\n【等级6】15% 回帖堕落-1 血液+2、发帖堕落-1 血液+2▕▏升级条件：消耗1金币（切换卡面）\n【 Max 】15% 回帖堕落-1 血液+2、发帖堕落-1 血液+2",
  "謎の男": "謎の男\n【勋章类型】奖品\n【入手条件】募捐活动奖励，凡募捐一次即可获得。\n【商店售价】无\n【 Max 】100% 回帖血液+1、发帖金币+1",
  "Chris Redfield in Uroboros": "Chris Redfield in Uroboros\n【入手条件】\"重口味学习小组\"群组勋章，由群主颁发。\n【商店售价】无\n【 Max 】1% 回帖知识+1、发帖知识+1",
  "杀意人偶": "杀意人偶\n【勋章类型】咒术\n【入手条件】在线时间(小时)≥72\n【商店售价】8咒术\n【持续时间】7天\n【等级1】1% 回帖血液+1▕▏升级条件：消耗2咒术\n【等级2】10% 回帖血液+2▕▏升级条件：旅程≥59\n【等级3】11% 回帖血液+3▕▏升级条件：旅程≥219\n【 Max 】8% 回帖堕落+1 金币-1",
  "雷霆晶球": "雷霆晶球\n【勋章类型】咒术\n【入手条件】咒术≥14\n【商店售价】7咒术\n【持续时间】7天\n【等级1】无属性▕▏升级条件：消耗7咒术\n【等级2】7% 回帖血液+1 咒术+1、发帖血液+1 咒术+1▕▏升级条件：消耗-7咒术\n【 Max 】无属性",
  "思绪骤聚": "思绪骤聚\n【勋章类型】咒术\n【入手条件】咒术≥20 知识≥10\n【商店售价】15咒术\n【持续时间】5天\n【等级1】3% 回帖知识+1、发帖知识+1▕▏升级条件：消耗-1知识\n【等级2】无属性▕▏升级条件：知识≥500\n【 Max 】无属性",
  "闪光糖果盒": "闪光糖果盒\n【勋章类型】赠礼\n【入手条件】无\n【商店售价】70金币\n【 Max 】17% 发帖咒术+1 血液+3",
  "茉香啤酒": "茉香啤酒\n【勋章类型】赠礼\n【入手条件】无\n【商店售价】25金币\n【等级1】无属性▕▏升级条件：消耗-1血液\n【 Max 】15% 回帖血液+1 堕落-1、发帖血液-1 堕落+1",
  "太空列车票": "太空列车票\n【勋章类型】咒术\n【入手条件】知识>=10\n【商店售价】5咒术\n【持续时间】5天\n【等级1】1% 回帖血液+1▕▏升级条件：旅程≥30\n【等级2】5% 回帖血液+1▕▏升级条件：旅程≥50\n【等级3】7% 回帖血液+1 金币+1▕▏升级条件：知识≥40\n【等级4】12% 回帖血液+1 金币+1▕▏升级条件：消耗30金币\n【 Max 】1% 回帖旅程+1",
  "艾利克斯": "艾利克斯\n【勋章类型】真人男从\n【入手条件】堕落≥50\n【商店售价】600金币\n【等级1】4% 回帖血液+1 堕落+1▕▏升级条件：消耗150金币\n【等级2】6% 回帖血液+2 堕落+1▕▏升级条件：堕落≥120\n【等级3】8% 回帖血液+2 堕落+1▕▏升级条件：消耗150金币\n【等级4】10% 回帖血液+2 堕落+1▕▏升级条件：消耗-1旅程\n【等级5】11% 回帖血液+2 堕落+1▕▏升级条件：血液≥666\n【 Max 】12% 回帖血液+3 金币-1、发帖旅程+1",
  "巴比伦辞典": "巴比伦辞典\n【勋章类型】故事\n【入手条件】<a href=\"/thread-162802-1-1.html\" target=\"_blank\">单机游戏区激励活动（点击跳转）</a>\n【商店售价】不可购买\n【 Max 】100% 回帖金币+1、发帖金币+1",
  "金翼使": "金翼使\n【勋章类型】奖品\n【入手条件】<a href=\"/thread-167739-1-1.html\" target=\"_blank\">单机游戏区激励活动（点击跳转）</a>\n【商店售价】不可购买\n【 Max 】20% 回帖金币+1 血液+1",
  "巨力橡果": "巨力橡果\n【勋章类型】奖品\n【入手条件】<a href=\"/thread-167764-1-1.html\" target=\"_blank\">动漫区激励活动（点击跳转）</a>\n【商店售价】不可购买\n【 Max 】20% 回帖金币+1 血液+1",
  "照相机": "照相机\n【勋章类型】奖品\n【入手条件】【瞬时映景】活动奖励\n【商店售价】无\n【 Max 】2% 回帖血液+1",
  "阿怪": "阿怪\n【勋章类型】奖品\n【入手条件】【夜影迷踪】活动参与奖\n【商店售价】无\n【 Max 】2% 发帖血液+1",
  "吉尔·沃瑞克": "吉尔·沃瑞克\n【勋章类型】女从\n【入手条件】知识>=16\n【商店售价】460金币\n【等级1】无属性▕▏升级条件： 消耗80金币\n【等级2】6% 回帖金币+2、发帖血液+1▕▏升级条件：消耗80血液\n【等级3】5% 回帖金币+3、发帖血液+2▕▏升级条件：消耗160血液\n【等级4】4% 回帖金币+4、发帖血液+3▕▏升级条件：消耗320血液\n【 Max 】8% 回帖金币+3、发帖血液+2",
  "希德法斯·特拉蒙": "希德法斯·特拉蒙\n【勋章类型】游戏男从\n【入手条件】旅程>=16\n【商店售价】720金币\n【等级1】无属性▕▏升级条件：消耗48咒术\n【等级2】3% 回帖咒术+1 血液+1、发帖咒术+1▕▏升级条件：消耗160血液\n【等级3】5% 回帖咒术+1 血液+1、发帖咒术+1▕▏升级条件：消耗320血液\n【 Max 】8% 回帖咒术+1 血液+2、发帖咒术+1",
  "神秘挑战书": "神秘挑战书\n【勋章类型】资产\n【入手条件】旅程>=28\n【商店售价】348金币\n【等级1】8% 回帖血液+1▕▏升级条件：消耗2旅程\n【等级2】8% 回帖血液+1▕▏升级条件：消耗2旅程\n【等级3】8% 回帖血液+1▕▏升级条件：消耗2旅程\n【等级4】8% 回帖血液+1▕▏升级条件：消耗2旅程\n【等级5】8% 回帖血液+1▕▏升级条件：消耗-2旅程\n【等级6】8% 回帖血液+1▕▏升级条件：消耗-2旅程\n【等级7】8% 回帖血液+1▕▏升级条件：消耗-2旅程\n【等级8】8% 回帖血液+1▕▏升级条件：消耗-2旅程\n【等级9】8% 回帖血液+1▕▏升级条件：在线时间≥2888\n【 Max 】1% 回帖血液+8、发帖血液+8",
  "坏掉的月亮提灯": "坏掉的月亮提灯\n【勋章类型】装备\n【入手条件】无\n【商店售价】380金币\n【等级1】4% 回帖咒术+1 血液+1、发帖咒术+1▕▏升级条件：咒术≥100\n【 Max 】6% 回帖咒术+1 血液+1、发帖咒术+2",
  "吸血猫蛋": "吸血猫蛋\n【勋章类型】宠物\n【入手条件】无\n【商店售价】333金币\n【等级1】5% 回帖血液+1▕▏升级条件：消耗66血液\n【等级2】6% 回帖血液+2、发帖血液+1▕▏升级条件：消耗666血液\n【等级3】7% 回帖血液+2、发帖血液+2▕▏升级条件：咒术≥123\n【 Max 】8% 回帖血液+2 咒术+1、发帖咒术+3",
  "装了衣物的纸盒": "装了衣物的纸盒\n【勋章类型】装备\n【入手条件】无\n【商店售价】99金币\n【等级1】无属性▕▏升级条件：消耗69血液\n【等级2】6% 回帖堕落+1、发帖堕落+1▕▏升级条件：旅程≥69\n【 Max 】10% 回帖血液-1 堕落+1、发帖血液-1 堕落+1",
  "狄翁・勒萨若": "狄翁・勒萨若\n【勋章类型】游戏男从\n【入手条件】在线时间（小时）>=160\n【商店售价】480金币\n【等级1】无属性▕▏升级条件：消耗160金币\n【等级2】2% 回帖血液+1 发帖血液+1▕▏升级条件：消耗160金币\n【等级3】4% 回帖血液+1 发帖咒术+1▕▏升级条件：消耗320血液\n【等级4】6% 回帖血液+2 发帖咒术+1▕▏升级条件：消耗48咒术\n【等级5】8% 回帖血液+2 金币+1、发帖咒术+2 金币+3▕▏升级条件：消耗64咒术\n【 Max 】10% 回帖血液+2 金币+2、发帖知识+1 金币+3",
  "五谷丰年": "五谷丰年\n【勋章类型】天赋\n【入手条件】在线时间>=120小时，注册天数>=24，旅程>=24\n【商店售价】无\n【等级1】4% 回帖血液+2 金币-1、发帖血液+4▕▏升级条件：血液≥365\n【等级2】5% 回帖血液+2 金币-1、发帖血液+4▕▏升级条件：知识≥365\n【等级3】12% 回帖血液+2 金币-1、发帖血液+4▕▏升级条件：旅程≥365\n【 Max 】24% 回帖金币+1、发帖金币+4",
  "库伦 (审判)": "库伦 (审判)\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】450金币\n【等级1】5% 回帖血液+1▕▏升级条件：知识≥16\n【等级2】7% 回帖血液+1、发帖血液+1▕▏升级条件：消耗350血液\n【等级3】15% 回帖血液+2、发帖血液+2▕▏升级条件：血液≥200\n【 Max 】10% 回帖血液+1、发帖血液+1 金币+1 旅程+1",
  "库伦 (起源)": "库伦 (起源)\n【勋章类型】游戏男从\n【入手条件】咒术≤25\n【商店售价】480金币\n【等级1】18% 回帖血液+2、发帖血液+6▕▏升级条件：咒术≥11\n【 Max 】7% 发帖血液+6 咒术-1",
  "变身器": "变身器\n【勋章类型】奖品\n【入手条件】【细语欢歌】活动期间根据抽取到的主题在音乐区发布翻唱歌曲\n【商店售价】无\n【等级1】2% 回帖咒术+1、发帖咒术+1▕▏升级条件：咒术≥1000\n【 Max 】2% 回帖咒术+1、发帖咒术+1",
  "GM村蛋糕": "GM村蛋糕\n【勋章类型】奖品\n【入手条件】<a href=\"/thread-80460-1-1.html\" target=\"_blank\">符合发放条件的用户（点击跳转）</a>\n【商店售价】无\n【 Max 】30% 回帖金币+1 血液+1\n【特别说明】兔兔只按大家站内资料填写的出生日期的月日来给大家制作蛋糕哦\n【特别说明】所以记得把站内资料填写完整并且公开",
  "近地夜航": "近地夜航\n【勋章类型】奖品\n【入手条件】在【十一周年】活动期间于论坛任意符合规定的版块发表主题帖，符合活动规定即可免费获得\n【商店售价】无\n【 Max 】11% 发帖知识+1",
  "半生黄金种": "半生黄金种\n【勋章类型】资产\n【入手条件】无\n【商店售价】120金币\n【等级1】2% 回帖金币+1、发帖金币+1▕▏升级条件：消耗1知识\n【等级2】2% 回帖咒术+1、发帖咒术+1▕▏升级条件：堕落≥22\n【等级3】20% 回帖金币+2 堕落+1、发帖金币+3▕▏升级条件：堕落≥23\n【等级4】3% 回帖咒术+1、发帖咒术+1▕▏升级条件：堕落≥123\n【 Max 】12% 回帖金币+1、发帖金币+1",
  "Zootopia": "Zootopia\n【勋章类型】板块\n【入手条件】旅程 >= 15\n【商店售价】100金币\n【等级1】3% 回帖咒术+1▕▏升级条件：消耗1旅程\n【等级2】3% 回帖咒术+1▕▏升级条件：消耗1旅程\n【等级3】3% 回帖咒术+1▕▏升级条件：消耗1旅程\n【等级4】3% 回帖咒术+1▕▏升级条件：消耗1旅程\n【等级5】3% 回帖咒术+1▕▏升级条件：消耗-4旅程\n【 Max 】3% 回帖咒术+1",
  "肃弓": "肃弓\n【勋章类型】装备\n【入手条件】无\n【商店售价】299金币\n【等级1】3% 回帖金币+1 堕落-1▕▏升级条件：消耗188血液\n【等级2】5% 回帖金币+1 堕落-1▕▏升级条件：消耗66咒术\n【 Max 】10% 回帖金币+1 血液-1 堕落-1",
  "叶卡捷琳娜": "叶卡捷琳娜\n【勋章类型】女从\n【入手条件】主题数>=3\n【商店售价】480金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗200金币\n【等级2】8% 回帖金币+1▕▏升级条件：消耗350血液\n【等级3】12% 回帖金币+1 血液+1、发帖金币+1 血液+1 金币+2▕▏升级条件：消耗66咒术\n【 Max 】12% 回帖金币+2 血液+1、发帖血液+1",
  "莱昂纳多·迪卡普里奥": "莱昂纳多·迪卡普里奥\n【勋章类型】真人男从\n【入手条件】知识 >= 10\n【商店售价】666金币\n【等级1】8% 回帖金币+1、发帖金币+2▕▏升级条件：金币≥666\n【等级2】18% 回帖金币+3、发帖金币+3  升级条件：金币≥688\n【等级3】12% 回帖金币+2、发帖金币+2▕▏升级条件：消耗1灵魂\n【 Max 】15% 回帖金币+3、发帖旅程+1",
  "迷之瓶": "迷之瓶\n【勋章类型】资产\n【入手条件】注册天数>=1700（天）, 堕落>=20, 旅程>=38,知识>=4\n【商店售价】120金币\n【 Max 】5% 回帖堕落+1、发帖金币+1 堕落+1\n【特别备注】另有一个奖品类型的同名勋章，效果十分强力，但是不在二手市场出现，故不在这里列出。",
  "布莱恩‧欧康纳": "布莱恩‧欧康纳\n【勋章类型】真人男从\n【入手条件】追随≥20\n【商店售价】400金币\n【等级1】4% 回帖金币+1▕▏升级条件：消耗100金币\n【等级2】6% 回帖金币+1▕▏升级条件：消耗100血液\n【等级3】8% 回帖金币+1▕▏升级条件：追随≥101\n【 Max 】10% 回帖金币+2、发帖旅程+1",
  "老旧的怀表": "老旧的怀表\n【勋章类型】资产\n【入手条件】在线时间>=280\n【商店售价】280金币\n【等级1】5% 回帖血液+1、发帖知识+1▕▏升级条件：消耗120金币\n【等级2】10% 回帖血液+1、发帖知识+1▕▏升级条件：消耗200血液\n【 Max 】2% 回帖知识+1、发帖灵魂+1",
  "辐射：新维加斯": "辐射：新维加斯\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖堕落+1",
  "上古卷轴V：天际": "上古卷轴V：天际\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】2% 回帖金币+2",
  "『居住证: Lv2~6』": "『居住证: Lv2~6』\n【勋章类型】剧情\n【入手条件】Lv2＜=等级＜=Lv6，旅程＞=10，发帖数＞=2\n【商店售价】无\n【时限】7天\n【等级 初级】无属性▕▏升级条件：消耗-10金币\n【 Max 】1% 回帖旅程+1、发帖旅程+1，",
  "『户口本: Lv7+』": "『户口本: Lv7+』\n【勋章类型】剧情\n【入手条件】等级＞=Lv7 灵魂＞=1\n【商店售价】无\n【时限】30天\n【等级 初级】无属性▕▏升级条件：消耗-100金币\n【 Max 】1% 回帖金币+1 血液+1 咒术+1 知识+1 堕落+1、发帖旅程+1 灵魂+1",
  "里昂‧S‧甘乃迪": "里昂‧S‧甘乃迪\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】450金币\n【等级1】8% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥100\n【等级2】10% 回帖金币+1、发帖金币+1▕▏升级条件：血液≥180\n【等级3】10% 回帖金币+1 血液+1、发帖金币+1 血液+1▕▏升级条件：消耗300金币\n【等级4】12% 回帖金币+2 血液+1、发帖金币+2 血液+1▕▏升级条件：血液≥300\n【 Max 】16% 回帖血液+3、发帖血液+3",
  "维克多‧天火": "维克多‧天火\n【勋章类型】游戏男从\n【入手条件】堕落≤50\n【商店售价】460金币\n【等级1】3% 回帖金币-1 血液+2 ▕▏升级条件：需求150血液\n【等级2】3% 回帖金币+1、发帖血液+2▕▏升级条件：消耗55堕落\n【等级3】10% 回帖金币+1 血液+1、发帖血液+2▕▏升级条件：知识≥30\n【等级4】25% 发帖金币+10▕▏升级条件：旅程≥99\n【 Max 】1% 回帖血液+5、发帖灵魂+1",
  "『任天堂Switch』灰黑√": "『任天堂Switch』灰黑√\n【勋章类型】剧情\n【入手条件】咒术＞=20\n【商店售价】100金币\n【等级 初级】无属性▕▏升级条件：消耗0灵魂\n【等级1】2% 回帖旅程+1▕▏升级条件：灵魂≥1\n【 Max 】2% 回帖旅程+1、发帖灵魂+1",
  "『任天堂Switch』红蓝√": "『任天堂Switch』红蓝√\n【勋章类型】剧情\n【入手条件】咒术＞=20\n【商店售价】100金币\n【等级 初级】无属性▕▏升级条件：消耗0灵魂\n【等级1】2% 回帖知识+1▕▏升级条件：灵魂≥1\n【 Max 】2% 回帖知识+1、发帖灵魂+1",
  "丹妮莉丝·坦格利安": "丹妮莉丝·坦格利安\n【勋章类型】女从\n【入手条件】旅程＞＝30\n【商店售价】400金币\n【等级1】6% 回帖金币+1、发帖血液+1▕▏升级条件：消耗150血液\n【等级2】8% 回帖金币+1、发帖血液+1▕▏升级条件：消耗300血液\n【等级3】10% 回帖金币+2、发帖血液+2▕▏升级条件：消耗400血液\n【 Max 】12% 回帖金币+3、发帖血液+3",
  "时间变异管理局": "时间变异管理局\n【勋章类型】板块\n【入手条件】在线时间≥199 且 知识≥19 且 旅程≥19\n【商店售价】100金币\n【等级1】2% 回帖血液+1、发帖金币+1▕▏升级条件：在线时间≥999\n【等级2】2% 回帖金币+2、发帖血液+2▕▏升级条件：在线时间≥1999\n【等级3】3% 回帖咒术+1、发帖咒术+1▕▏升级条件：在线时间≥2999\n【 Max 】3% 回帖旅程+1、发帖旅程+1",
  "白猪猪储蓄罐": "白猪猪储蓄罐\n【勋章类型】储蓄\n【入手条件】无，但是有时限，过期消失\n【商店售价】100金币\n【等级1】50% 回帖金币+1、发帖金币+1▕▏升级条件：消耗-101金币\n【 Max 】无属性",
  "粉猪猪储蓄罐": "粉猪猪储蓄罐\n【勋章类型】储蓄\n【入手条件】无，但是有时限，过期消失\n【商店售价】1000金币\n【等级1】50% 回帖金币+1、发帖金币+1▕▏升级条件：消耗-1025金币\n【 Max 】无属性",
  "金猪猪储蓄罐": "金猪猪储蓄罐\n【勋章类型】储蓄\n【入手条件】无，但是有时限，过期消失\n【商店售价】10000金币\n【等级1】50% 回帖金币+1、发帖金币+1▕▏升级条件：消耗-10100金币\n【 Max 】无属性",
  "不起眼的空瓶": "不起眼的空瓶\n【勋章类型】储蓄\n【入手条件】无，但是有时限，过期消失\n【商店售价】10咒术\n【等级1】8% 回帖咒术+1、发帖咒术+1▕▏升级条件：消耗-13咒术\n【 Max 】无属性",
  "古烈": "古烈\n【勋章类型】游戏男从\n【入手条件】旅程≥20\n【商店售价】560金币\n【等级1】2% 回帖咒术+1▕▏升级条件：追随≥33\n【等级2】4% 回帖血液+1 咒术+1▕▏升级条件：消耗88咒术\n【等级3】5% 回帖血液+1 咒术+1▕▏升级条件：消耗360血液\n【 Max 】8% 回帖血液+2 咒术+1",
  "奇异博士": "奇异博士\n【勋章类型】真人男从\n【入手条件】旅程≥14\n【商店售价】700金币\n【等级1】2% 回帖咒术+1、发帖咒术+1▕▏升级条件：消耗777血液\n【等级2】6% 回帖咒术+1、发帖咒术+1▕▏升级条件：消耗777血液\n【等级3】8% 回帖咒术+1、发帖咒术+1▕▏升级条件：消耗1400血液\n【 Max 】14% 回帖咒术+1、发帖咒术+1",
  "托比·马奎尔": "托比·马奎尔\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】800金币\n【等级1】10% 回帖金币+2、发帖金币+2▕▏升级条件：消耗10咒术\n【等级2】10% 回帖堕落+2、发帖堕落+2▕▏升级条件：消耗10咒术\n【等级3】10% 回帖堕落-2、发帖堕落-2▕▏升级条件：消耗10咒术\n【 Max 】12% 回帖金币+2、发帖金币+2",
  "阿丽塔": "阿丽塔\n【勋章类型】女从\n【入手条件】知识≥20\n【商店售价】700金币\n【等级1】6% 回帖血液+1、发帖血液+1▕▏升级条件：消耗450金币\n【等级2】12% 回帖血液+1、发帖金币+2▕▏升级条件：消耗450血液\n【等级3】12% 回帖血液+2、发帖金币+2▕▏升级条件：消耗1灵魂\n【 Max 】16% 回帖血液+3、发帖旅程+1",
  "圣诞有铃": "圣诞有铃\n【勋章类型】装备\n【入手条件】无\n【商店售价】100金币\n【等级1】2% 回帖血液+1 堕落+1、发帖堕落+1▕▏升级条件：堕落≥99\n【 Max 】6% 回帖血液+1 堕落-1、发帖堕落-1 咒术+1",
  "羽毛胸针": "羽毛胸针\n【勋章类型】资产\n【入手条件】无\n【商店售价】149金币\n【等级1】1% 回帖血液+1 旅程+1、发帖血液+1▕▏升级条件：好友数≥30\n【等级2】2% 回帖血液+1 旅程+1、发帖血液+1 旅程+1▕▏升级条件：好友数≥100\n【 Max 】3% 回帖血液+1 旅程+1、发帖血液+1 旅程+1",
  "都市：天际线2": "都市：天际线2\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】1% 回帖旅程+1",
  "【新春限定】果体 隆": "【新春限定】果体 隆\n【勋章类型】游戏男从\n【入手条件】无\n【商店售价】555金币\n【等级1】5%  回帖金币+1、发帖金币+1▕▏升级条件：消耗100血液\n【等级2】15% 回帖金币+1、发帖金币+1▕▏升级条件：消耗555血液\n【 Max 】50% 回帖金币+1、发帖金币+1",
  "汉尼拔": "汉尼拔\n【勋章类型】真人男从\n【入手条件】追随 >= 10\n【商店售价】650金币\n【等级1】3%  回帖血液+1▕▏升级条件：消耗400金币\n【等级2】5%  回帖血液+1、发帖堕落+1▕▏升级条件：知识≥101\n【等级3】10% 回帖血液+1、发帖堕落+1▕▏升级条件：追随≥202\n【等级4】13% 回帖血液+2、发帖知识+1▕▏升级条件：消耗600血液\n【等级5】16% 回帖血液+2、发帖知识+1▕▏升级条件：消耗1灵魂\n【 Max 】18% 回帖血液+3、发帖知识+1",
  "梅琳娜Melina": "梅琳娜Melina\n【勋章类型】女从\n【入手条件】无\n【商店售价】999血液\n【等级1】8%  回帖金币+1 血液+1▕▏升级条件：消耗1灵魂\n【等级2】10% 回帖金币+1 血液+1▕▏升级条件：消耗-888血液\n【等级3】2%  回帖旅程+1、发帖灵魂+1▕▏升级条件：消耗1金币\n【等级4】25% 回帖金币+1▕▏升级条件：消耗1金币\n【等级5】20% 回帖金币+2 血液-1▕▏升级条件：消耗1金币\n【 Max 】25% 回帖血液+1",
  "贝儿(Belle)": "贝儿(Belle)\n【勋章类型】女从\n【入手条件】旅程 >= 20\n【商店售价】666金币\n【等级1】5%  回帖金币+2、发帖血液+2▕▏升级条件：消耗99咒术\n【等级2】10% 回帖金币+2、发帖血液+2▕▏升级条件：消耗1314血液\n【等级3】15% 回帖金币+2、发帖血液+2▕▏升级条件：消耗1灵魂\n【 Max 】16% 回帖金币+2 血液+1、发帖旅程+1",
  "普通羊毛球": "普通羊毛球\n【勋章类型】装备\n【入手条件】无\n【商店售价】200金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗120血液\n【等级2】7% 回帖金币+1▕▏升级条件：消耗180血液\n【等级3】9% 回帖金币+1▕▏升级条件：在线时间≥1200\n【 Max 】11% 回帖金币+1 血液+1、发帖旅程+1",
  "神秘天球": "神秘天球\n【勋章类型】资产\n【入手条件】知识 >= 10\n【商店售价】650金币\n【等级1】5%  回帖血液+1、发帖血液+1▕▏升级条件：消耗66咒术\n【等级2】10% 回帖血液+1、发帖血液+2▕▏升级条件：消耗66咒术\n【等级3】15% 回帖血液+1、发帖血液+2▕▏升级条件：消耗66咒术\n【等级4】20% 回帖血液+1、发帖血液+2▕▏升级条件：消耗-1知识\n【等级5】1%  回帖知识+1、发帖知识+1▕▏升级条件：知识≥100\n【等级6】20% 回帖堕落+1、发帖堕落+1▕▏升级条件：消耗1旅程\n【等级7】20% 回帖堕落-1、发帖堕落-1▕▏升级条件：消耗-1旅程\n【等级8】20% 回帖血液+1、发帖血液+2▕▏升级条件：消耗1灵魂\n【等级9】30% 回帖血液+1、发帖血液+2▕▏升级条件：血液≥666\n【 Max 】2% 发帖灵魂+1",
  "婚姻登记册": "婚姻登记册\n【勋章类型】资产\n【入手条件】血液 >= 52\n【商店售价】52金币\n【等级1】2% 回帖血液+1▕▏升级条件：消耗9血液\n【等级2】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗11血液\n【等级3】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗11血液\n【等级4】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗21血液\n【等级5】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗21血液\n【等级6】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗52血液\n【等级7】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗52血液\n【 Max 】5% 回帖血液+2、发帖血液+2",
  "健忘礼物盒": "健忘礼物盒\n【勋章类型】资产\n【入手条件】无\n【商店售价】123金币\n【等级1】10% 回帖血液+1▕▏升级条件：消耗1知识\n【等级2】8% 回帖血液+1▕▏升级条件：消耗1知识\n【等级3】6% 回帖血液+1▕▏升级条件：知识≥50\n【等级4】4% 回帖血液+1▕▏升级条件：消耗-2知识\n【 Max 】3% 回帖血液+2 知识+1",
  "脏兮兮的蛋": "脏兮兮的蛋\n【勋章类型】宠物\n【入手条件】发帖数 >= 200\n【商店售价】404金币\n【等级1】10% 回帖血液+1▕▏升级条件：消耗404金币\n【等级2】10% 回帖血液-1 堕落+1▕▏升级条件：发帖数≥404\n【等级3】10% 回帖血液+4 咒术-1▕▏升级条件：血液≥404\n【等级4】10% 回帖血液-3 金币+3▕▏升级条件：金币≥404\n【等级5】无属性▕▏升级条件：主题数≥4\n【等级6】10% 回帖血液-4 咒术+1▕▏升级条件：发帖数≥1000\n【 Max 】10% 回帖血液+3",
  "猫咪点唱机": "猫咪点唱机\n【勋章类型】奖品\n【入手条件】<a href=\"/thread-157767-1-1.html\" target=\"_blank\">音乐交流区激励活动（点击跳转）</a>\n【商店售价】无\n【 Max 】10% 回帖金币+1",
  "John Reese": "John Reese\n【勋章类型】真人男从\n【入手条件】金币 >= 1500\n【商店售价】1000金币\n【等级1】5% 回帖血液+1、发帖血液+1▕▏升级条件：金币≥2000\n【等级2】10% 回帖血液+1、发帖血液+1▕▏升级条件：消耗600血液\n【等级3】10% 回帖血液+2、发帖血液+1▕▏升级条件：金币≥2500\n【等级4】12% 回帖血液+1、发帖血液+1▕▏升级条件：消耗700金币\n【等级5】12% 回帖血液+2、发帖血液+1▕▏升级条件：金币≥3000\n【 Max 】15% 回帖血液+3、发帖血液+3",
  "穿靴子的猫": "穿靴子的猫\n【勋章类型】真人男从\n【入手条件】旅程 >= 9\n【商店售价】666金币\n【等级1】6% 回帖金币+1、发帖金币+1▕▏升级条件：消耗444血液\n【等级2】9% 回帖金币+1、发帖金币+1▕▏升级条件：消耗444金币\n【等级3】13% 回帖金币+3 血液-2、发帖金币+3 血液-2▕▏升级条件：在线时间≥365\n【等级4】12% 回帖血液+2、发帖血液+2▕▏升级条件：消耗666血液\n【等级5】12% 回帖血液+2 金币+1、发帖血液+2 金币+1▕▏升级条件：消耗88咒术\n【 Max 】12% 回帖血液+2 金币+2、发帖血液+2 金币+2",
  "狗狗": "狗狗\n【勋章类型】真人男从\n【入手条件】主题数 >= 1\n【商店售价】300金币\n【等级1】5% 回帖血液+1、发帖旅程+1▕▏升级条件：消耗300金币\n【等级2】8% 回帖血液+2、发帖旅程+1▕▏升级条件：消耗550血液\n【等级3】10% 回帖血液+2、发帖旅程+1▕▏升级条件：金币≥1314\n【 Max 】12% 回帖血液+3、发帖血液+3",
  "阿加莎·哈克尼斯": "阿加莎·哈克尼斯\n【勋章类型】女从\n【入手条件】堕落 >= 94\n【商店售价】400金币\n【等级1】1% 回帖咒术+1、发帖知识+1▕▏升级条件：消耗500血液\n【等级2】4% 回帖咒术+1 血液+1、发帖知识+1▕▏升级条件：消耗88咒术\n【等级3】8% 回帖咒术+1 血液+1、发帖知识+1▕▏升级条件：知识≥69\n【 Max 】8% 回帖金币+1 血液+2 堕落+1、发帖知识+1",
  "圣水瓶": "圣水瓶\n【勋章类型】装备\n【入手条件】无\n【商店售价】999金币\n【等级1】2% 回帖金币+1、发帖金币+1▕▏升级条件：消耗1血液\n【等级2】4% 回帖金币+1、发帖金币+1▕▏升级条件：消耗10血液\n【等级3】8% 回帖金币+1、发帖金币+1▕▏升级条件：主题数≥100\n【等级4】25% 回帖金币+1、发帖金币+1▕▏升级条件：消耗1000血液\n【 Max 】50% 回帖金币+1、发帖金币+1",
  "弯钩与连枷": "弯钩与连枷\n【勋章类型】资产\n【入手条件】旅程 >= 20\n【商店售价】330金币\n【等级1】3% 回帖堕落+1、发帖堕落+1▕▏升级条件：消耗60血液\n【等级2】6% 回帖堕落+1、发帖血液+1 堕落+1▕▏升级条件：消耗90金币\n【等级3】9% 回帖血液+1 堕落+1、发帖血液+1 堕落+1▕▏升级条件：消耗80咒术\n【等级4】12% 回帖血液+1 堕落+1、发帖血液+1 堕落+1▕▏升级条件：消耗180血液\n【 Max 】18% 回帖血液+1、发帖血液+1",
  "黑神话:悟空": "黑神话:悟空\n【勋章类型】板块\n【入手条件】无\n【商店售价】100金币\n【 Max 】5% 发帖旅程+1",
  "巴哈姆特": "巴哈姆特\n【勋章类型】游戏男从\n【入手条件】知识 >= 20\n【商店售价】700金币\n【等级1】5% 回帖血液+1、发帖血液+1▕▏升级条件：好友数≥24\n【等级2】8% 回帖血液+1、发帖血液+1▕▏升级条件：消耗500血液\n【等级3】8% 回帖血液+2、发帖血液+2▕▏升级条件：知识≥100\n【等级4】12% 回帖血液+2、发帖血液+2▕▏升级条件：消耗100咒术\n【等级5】13% 回帖血液+3、发帖血液+3▕▏升级条件：消耗1堕落\n【等级6】13% 回帖血液+3、发帖血液+3▕▏升级条件：消耗1灵魂\n【 Max 】16% 回帖血液+3、发帖血液+3",
  "约翰·康斯坦丁": "约翰·康斯坦丁\n【勋章类型】真人男从\n【入手条件】堕落 >= 25\n【商店售价】450金币\n【等级1】3% 回帖血液+1▕▏升级条件：堕落≥50\n【等级2】10% 回帖血液+2、发帖堕落+1▕▏升级条件：消耗1血液\n【等级3】5% 回帖血液+1▕▏升级条件：堕落≥100\n【等级4】9% 回帖血液+1、发帖堕落+1▕▏升级条件：堕落≥150\n【等级5】13% 回帖血液+1、发帖堕落+1▕▏升级条件：堕落≥200\n【等级6】15% 回帖血液+1、发帖堕落+1▕▏升级条件：堕落≥350\n【 Max 】20% 回帖血液+1、发帖咒术+1",
  "牛局长博戈": "牛局长博戈\n【勋章类型】真人男从\n【入手条件】咒术 >= 20\n【商店售价】200金币\n【等级1】5% 回帖金币+1、发帖金币+1▕▏升级条件：消耗350血液\n【等级2】8% 回帖咒术+1、发帖咒术+1▕▏升级条件：咒术≥40\n【 Max 】40% 发帖堕落+1",
  "亨利.卡维尔": "亨利.卡维尔\n【勋章类型】真人男从\n【入手条件】无\n【商店售价】500金币\n【等级1】1% 回帖金币+5▕▏升级条件：消耗400血液\n【等级2】10% 回帖金币+3▕▏升级条件：消耗1金币\n【等级3】10% 回帖金币+3▕▏升级条件：消耗1金币\n【等级4】10% 回帖金币+3▕▏升级条件：消耗1金币\n【等级5】10% 回帖金币+3▕▏升级条件：消耗1金币\n【等级6】10% 回帖金币+3▕▏升级条件：消耗1金币\n【等级7】10% 回帖金币+3▕▏升级条件：消耗1金币\n【等级8】10% 回帖金币+3▕▏升级条件：消耗1金币\n【等级9】10% 回帖金币+3▕▏升级条件：消耗1金币\n【 Max 】10% 回帖金币+3",
  "红夫人": "红夫人\n【勋章类型】女从\n【入手条件】追随 >= 15\n【商店售价】688金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗99咒术\n【等级2】9% 回帖金币+2▕▏升级条件：消耗399血液\n【等级3】13% 回帖金币+2▕▏升级条件：堕落≥399\n【 Max 】13% 回帖金币+3",
  "莉莉娅·考尔德（Lilia Calderu）": "莉莉娅·考尔德（Lilia Calderu）\n【勋章类型】女从\n【入手条件】咒术 >= 77\n【商店售价】777金币\n【等级1】无属性▕▏升级条件：消耗177咒术\n【等级2】1% 回帖咒术+1▕▏升级条件：消耗1旅程\n【等级3】2% 回帖旅程+1▕▏升级条件：消耗177金币\n【等级4】3% 回帖金币+1▕▏升级条件：消耗1旅程\n【等级5】4% 回帖金币+1▕▏升级条件：消耗177血液\n【等级6】5% 回帖金币+1▕▏升级条件：消耗-2旅程\n【等级7】6% 回帖金币+2 血液-1▕▏升级条件：在线时长≥777\n【等级8】7% 回帖金币+2▕▏升级条件：灵魂≥1\n【等级9】10% 回帖金币+2▕▏升级条件：灵魂≥3\n【 Max 】17% 回帖金币+2",
  "尼特公仔": "尼特公仔\n【勋章类型】资产\n【入手条件】无\n【商店售价】188金币\n【等级1】6% 回帖血液-1 金币+1、发帖血液-1 金币+1▕▏升级条件：消耗288金币\n【等级2】8% 回帖血液+2 金币-1、发帖血液-1 金币+2▕▏升级条件：消耗288血液\n【 Max 】10% 回帖金币+3 血液-1 堕落+1、发帖血液+3 金币-1 堕落-1",
  "黑暗之魂系列": "黑暗之魂系列\n【勋章类型】场景&版块\n【入手条件】无\n【商店售价】100金币\n【 Max 】1% 回帖血液+1 堕落+1、发帖血液+1 堕落-1",
  "女巫之路": "女巫之路\n【勋章类型】场景&版块\n【入手条件】咒术 >= 77\n【商店售价】100金币\n【等级1】1% 回帖咒术+1▕▏升级条件：消耗7旅程\n【等级2】1% 回帖旅程+1▕▏升级条件：消耗7咒术\n【等级3】2% 回帖咒术+1▕▏升级条件：消耗-7旅程\n【等级4】2% 回帖旅程+1▕▏升级条件：在线时间≥777\n【等级5】3% 回帖咒术+1▕▏升级条件：消耗7咒术\n【 Max 】3% 回帖旅程+1",
  "站员: 保卫领土": "站员: 保卫领土\n【勋章类型】薪俸与其他\n【入手条件】仅“站员”可领取\n【商店售价】无\n【时限】30天\n【等级1】无属性▕▏升级条件：消耗-100金币\n【 Max 】100% 回帖金币+1、发帖金币+1",
  "见习版主: 神的重量": "见习版主: 神的重量\n【勋章类型】薪俸与其他\n【入手条件】仅“见习版主”可领取\n【商店售价】无\n【时限】30天\n【等级1】无属性▕▏升级条件：消耗-130金币\n【 Max 】100% 回帖金币+1、发帖金币+1",
  "版主: 一国之主": "版主: 一国之主\n【勋章类型】薪俸与其他\n【入手条件】仅“版主”可领取\n【商店售价】无\n【时限】30天\n【等级1】无属性▕▏升级条件：消耗-200金币\n【 Max 】100% 回帖金币+1、发帖金币+1",
  "『还乡歌』": "『还乡歌』\n【勋章类型】剧情\n【入手条件】注册天数≥1825\n【商店售价】无\n【等级 初级】无属性▕▏升级条件：灵魂≥1\n【等级1】无属性▕▏升级条件：总积分≥300\n【等级2】无属性▕▏升级条件：在线时间≥1000\n【 Max 】无属性",
  "纯真护剑㊕": "纯真护剑㊕\n【勋章类型】装饰/特殊\n【入手条件】灵魂≥1（儿童节限时获取）\n【商店售价】无\n【等级1】无属性▕▏升级条件：？？？\n【等级2】无属性▕▏升级条件：？？？\n【等级3】无属性▕▏升级条件：？？？\n【 Max 】无属性",
  "『日心说』": "『日心说』\n【勋章类型】剧情\n【入手条件】【星楼钟塔】成功报名参加占星仪式\n【商店售价】无\n【等级 初级】无属性▕▏升级条件：消耗-1知识\n【 Max 】1% 回帖知识+1、发帖知识+1",
  "『圣洁化身』": "『圣洁化身』\n【勋章类型】剧情\n【入手条件】Level 10 max， 堕落≤1并且灵魂≥1\n【商店售价】1堕落\n【时限】1天\n【 Max 】1% 回帖堕落-1、发帖堕落-1",
  "『搓粉团珠』": "『搓粉团珠』\n【勋章类型】剧情\n【入手条件】灵魂≥1或者主题数≥15（2025年元宵节限时购买）\n【商店售价】15金币\n【 Max 】无属性",
  "『冰雕马拉橇』": "『冰雕马拉橇』\n【勋章类型】剧情\n【入手条件】在游戏大区，根据自身等级完成【冰上飞贼】或【名门圆舞】等发帖剧情任务\n【商店售价】无\n【 Max 】无属性",
  "『南瓜拿铁』": "『南瓜拿铁』\n【勋章类型】剧情\n【入手条件】主题数≥1并且发帖数≥1并且注册天数≥1并且在线时间(小时)≥1并且旅程≥1（2024年10月31日~11月10日 限时获取）\n【商店售价】5金币\n【时限】5天（可续期）\n【 Max 】无属性",
  "『逆境中的幸运女神』": "『逆境中的幸运女神』\n【勋章类型】剧情\n【入手条件】【英雄再聚】活动中战功卓著的小队获得\n【商店售价】无\n【 Max 】无属性",
  "『泥潭颂唱者』": "『泥潭颂唱者』\n【勋章类型】剧情\n【入手条件】灵魂≥1并且咒术≥10并且在线时间(小时)≥100\n【商店售价】1咒术\n【时限】7天（可续期）\n【 Max 】无属性",
  "『钟楼日暮』": "『钟楼日暮』\n【勋章类型】剧情\n【入手条件】灵魂≥1并且在线时间(小时)≥300并且发帖数≥30并且主题数≥3（限时获取）\n【商店售价】1旅程\n【时限】30天（可续期）\n【等级0】无属性▕▏升级条件：好友数≥3\n【等级1】无属性▕▏升级条件：消耗1金币\n【 Max 】无属性",
  "『流星赶月』": "『流星赶月』\n【勋章类型】剧情\n【入手条件】灵魂≥1并且发帖数≥5并且主题数≥1（2024年5月1日~5月31日 限时获取）\n【商店售价】1旅程（2024年5月1日~5月11日）/ 5旅程（2024年5月12日~5月31日）\n【时限】30天（可续期）\n【等级1】无属性▕▏升级条件：主题数≥10\n【等级2】无属性▕▏升级条件：主题数≥50\n【等级3】无属性▕▏升级条件：主题数≥100\n【 Max 】无属性",
  "『先知灵药』": "『先知灵药』\n【勋章类型】剧情\n【入手条件】灵魂≥1并且知识≥10（2024年4月2日~4月30日 限时获取）\n【商店售价】10金币\n【等级1】无属性▕▏升级条件：旅程≥100\n【 Max 】无属性",
  "『酒馆蛋煲』": "『酒馆蛋煲』\n【勋章类型】剧情\n【入手条件】灵魂≥1或者发帖数≥10（限时获取）\n【商店售价】10金币\n【时限】1天（可续期）\n【 Max 】无属性",
  "『活动代币』": "『活动代币』\n【勋章类型】剧情\n【入手条件】无（限时获取）\n【商店售价】1金币\n【时限】7天\n【等级 初级】无属性▕▏升级条件：消耗0金币\n【等级1】无属性▕▏升级条件：消耗0金币\n【等级2】无属性▕▏升级条件：消耗0金币\n【 Max 】无属性",
  "『伊黎丝的祝福』": "『伊黎丝的祝福』\n【勋章类型】剧情\n【入手条件】灵魂≥1（限时获取）\n【商店售价】1咒术\n【时限】30天（可续期）\n【等级 初级】无属性▕▏升级条件：消耗-1咒术\n【 Max 】无属性",
  "『 弗霖的琴』": "『弗霖的琴』\n【勋章类型】剧情\n【入手条件】【旷世奇珍】酒会：内定密谋，猜中成交价\n【商店售价】1旅程\n【 Max 】无属性",
  "『瓶中信』": "『瓶中信』\n【勋章类型】剧情\n【入手条件】发帖数≥1并且主题数≥1（限时获取）\n【商店售价】1金币\n【时限】21天\n【等级1】无属性▕▏升级条件：消耗1旅程\n【等级2】无属性▕▏升级条件：消消耗-1旅程\n【等级3】无属性▕▏升级条件：消耗-20金币\n【 Max 】无属性",
  "『林中过夜』": "『林中过夜』\n【勋章类型】剧情\n【入手条件】灵魂≥1并且主题数≥2并且旅程≥16\n【商店售价】1旅程\n【 Max 】无属性",
  "『凯旋诺书』": "『凯旋诺书』\n【勋章类型】剧情\n【入手条件】【派遣远征s1】派遣先锋的天选仪式\n【商店售价】无\n【 Max 】无属性",
  "『绿茵甘露』": "『绿茵甘露』\n【勋章类型】剧情\n【入手条件】【派遣远征s1】主题数≥1\n【商店售价】1金币\n【时限】14天\n【 Max 】无属性",
  "『道具超市』": "『道具超市』\n【勋章类型】剧情\n【入手条件】灵魂≥1并且在线时间(小时)≥100并且主题数≥1并且发帖数≥1\n【商店售价】1旅程\n【等级1】无属性▕▏升级条件：消耗75金币\n【 Max 】无属性",
  "『狄文卡德的残羽』": "『狄文卡德的残羽』\n【勋章类型】剧情\n【入手条件】无\n【商店售价】1堕落\n【 Max 】无属性",
  "『厢庭望远』": "『厢庭望远』\n【勋章类型】剧情\n【入手条件】无\n【商店售价】无\n【等级1】无属性▕▏升级条件：总积分≥1000\n【 Max 】无属性",
  "『转生经筒』": "『转生经筒』\n【勋章类型】剧情\n【入手条件】签到天数≥1460天并且灵魂≥1\n【商店售价】1旅程\n【 Max 】无属性",
  "“半狼”布莱泽": "“半狼”布莱泽\n【勋章类型】游戏男从\n【入手条件】旅程 >= 5\n【商店售价】496金币\n【等级1】5% 回帖金币+1、发帖金币+1▕▏升级条件：消耗99金币\n【等级2】9% 回帖金币+1、发帖金币+1▕▏升级条件：咒术≥399\n【等级3】9% 回帖金币+2、发帖金币+2▕▏升级条件：消耗1199金币\n【等级4】12% 回帖金币+2、发帖金币+2▕▏升级条件：血液≥521\n【 Max 】13% 回帖金币+3、发帖金币+3",
  "炽焰咆哮虎": "炽焰咆哮虎\n【勋章类型】游戏男从\n【入手条件】旅程 >= 30\n【商店售价】500金币\n【等级1】5% 回帖血液+1、发帖血液+1▕▏升级条件：消耗100金币\n【等级2】7% 回帖血液+1、发帖血液+1▕▏升级条件：消耗50咒术\n【等级3】8% 回帖血液+2、发帖血液+2▕▏升级条件：消耗300金币\n【 Max 】10% 回帖血液+3、发帖血液+3 堕落+2",
  "基努·里维斯": "基努·里维斯\n【勋章类型】真人男从\n【入手条件】堕落 <= 999\n【商店售价】666金币\n【等级1】1% 回帖血液+1▕▏升级条件：消耗99金币\n【等级2】5% 回帖血液+1▕▏升级条件：消耗99金币\n【等级3】15% 回帖血液+1 金币-1▕▏升级条件：血液≥1964\n【等级4】10% 回帖血液+2▕▏升级条件：消耗99血液\n【等级5】20% 回帖血液+1 金币-1▕▏升级条件：消耗99咒术\n【 Max 】2% 回贴堕落-1、发帖灵魂+1 堕落-3",
  "琴.葛蕾": "琴.葛蕾\n【勋章类型】女从\n【入手条件】咒术 >= 30\n【商店售价】500金币\n【等级1】10% 回帖堕落+1▕▏升级条件：堕落≥600\n【等级2】10% 回帖堕落+2▕▏升级条件：消耗66咒术\n【等级3】勋章博物馆资料暂缺\n【等级4】3% 回帖旅程+1▕▏升级条件：堕落≥600\n【 Max 】10% 回帖堕落-2",
  "Honey B Lovely": "Honey B Lovely\n【勋章类型】女从\n【入手条件】无\n【商店售价】520金币\n【等级1】勋章博物馆资料暂缺\n【等级2】10% 回帖金币+1、发帖金币+1▕▏升级条件：消耗710金币\n【等级3】11% 回帖金币+2、发帖金币+2▕▏升级条件：消耗198金币\n【 Max 】12% 回帖金币+3、发帖金币+3",
  "黑暗封印": "黑暗封印\n【勋章类型】装备\n【入手条件】堕落 >= 130\n【商店售价】350金币\n【等级1】3% 回帖咒术+1 血液-3▕▏升级条件：消耗1250金币\n【等级2】10% 回帖咒术+1 血液-3▕▏升级条件：好友数≥13\n【等级3】8% 回帖血液-3 咒术+1▕▏升级条件：好友数≥130\n【等级4】6% 回帖血液-3 咒术+1▕▏升级条件：知识≥130\n【等级5】5% 回帖咒术+2、发帖咒术+2▕▏升级条件：灵魂≥25\n【 Max 】8% 回帖咒术+2 堕落+1",
  "枯木法杖": "枯木法杖\n【勋章类型】装备\n【入手条件】无\n【商店售价】500金币\n【等级1】5% 回帖金币+1▕▏升级条件：消耗1000血液\n【等级2】10% 回帖金币+1▕▏升级条件：堕落≥100\n【等级3】10% 回帖金币+3 堕落-1▕▏升级条件：堕落≥300\n【等级4】10% 回帖金币+2▕▏升级条件：堕落≥500\n【 Max 】10% 回帖金币+2 堕落+1",
  "千年积木": "千年积木\n【勋章类型】装备\n【入手条件】无\n【商店售价】200金币\n【等级1】3% 回帖咒术+1▕▏升级条件：消耗30咒术\n【 Max 】4% 回帖咒术+2",
  "辉夜姬的五难题": "辉夜姬的五难题\n【勋章类型】资产\n【入手条件】无\n【商店售价】500金币\n【等级1】5% 回帖血液+1▕▏升级条件：消耗125金币\n【等级2】5% 回帖金币+1▕▏升级条件：消耗125血液\n【等级3】5% 回帖血液+2、发帖旅程+1▕▏升级条件：消耗125金币\n【等级4】5% 回帖金币+3、发帖旅程+1▕▏升级条件：消耗125血液\n【等级5】5% 回帖血液+4、发帖旅程+1▕▏升级条件：消耗500金币\n【 Max 】5% 回帖金币+5",
  "末影珍珠": "末影珍珠\n【勋章类型】资产\n【入手条件】知识 >= 16\n【商店售价】256金币\n【等级1】1% 回帖咒术+1▕▏升级条件：消耗64血液\n【等级2】3% 回帖咒术+1▕▏升级条件：消耗128血液\n【等级3】2% 回帖旅程+1▕▏升级条件：在线时间≥999\n【 Max 】3% 回帖旅程+1",
  "双生蛋": "双生蛋\n【勋章类型】宠物\n【入手条件】堕落 >= 177\n【商店售价】377金币\n【等级1】3% 回帖堕落+1▕▏升级条件：消耗377血液\n【等级2】15% 回帖堕落+1 金币+3▕▏升级条件：堕落≥17\n【 Max 】10% 回帖堕落-1 金币+1"
}


def calculate_income(text: str) -> Dict:
    """
    Calculate medal income based on description text.
    
    Args:
        text: Medal description text
        
    Returns:
        Dictionary containing medal name, income details, and maximum income
    """
    lines = text.split('\n')
    max_income = {"reply_income": 0, "topic_income": 0, "total_income": 0, "level": 0}

    for line in lines:
        total_income = 0
        reply_income = 0
        topic_income = 0

        # Match trigger probability
        probability_match = re.search(r'】(\d+)%', line)
        if probability_match:
            probability = float(probability_match.group(1)) / 100

            # Match reply attributes
            reply_attr_match = re.search(r'回帖(.*?)(,|$|发帖|升级|▕|▕▏)', line)
            if reply_attr_match:
                attribute_matches = re.finditer(
                    r'(金币|血液|旅程|咒术|知识|灵魂|堕落)(\+|-)(\d+)',
                    reply_attr_match.group(1)
                )
                for match in attribute_matches:
                    attr_name = match.group(1)
                    sign = match.group(2)
                    value = int(match.group(3)) * (1 if sign == '+' else -1)
                    attr_en = ATTRIBUTE_CN_TO_EN.get(attr_name, attr_name)
                    weight = INCOME_WEIGHT_MAP.get(attr_en, 0)
                    income = probability * value * weight
                    reply_income += income

            # Match topic attributes
            topic_attr_match = re.search(r'发帖(.*?)(升级|▕ | ▕▏|$)', line)
            if topic_attr_match:
                attribute_matches = re.finditer(
                    r'(金币|血液|旅程|咒术|知识|灵魂|堕落)(\+|-)(\d+)',
                    topic_attr_match.group(1)
                )
                for match in attribute_matches:
                    attr_name = match.group(1)
                    sign = match.group(2)
                    value = int(match.group(3)) * (1 if sign == '+' else -1)

                    attr_en = ATTRIBUTE_CN_TO_EN.get(attr_name, attr_name)
                    weight = INCOME_WEIGHT_MAP.get(attr_en, 0)
                    income = probability * value * weight
                    topic_income += income

        # Calculate total income with weight adjustment
        total_income = (
            reply_income +
            topic_income * (PERSONAL_POST_WEIGHT["reply"] / PERSONAL_POST_WEIGHT["topic"])
        )

        # Update maximum income if current income is higher
        if total_income >= max_income["total_income"]:
            max_income["total_income"] = total_income
            max_income["topic_income"] = topic_income
            max_income["reply_income"] = reply_income
            level_match = re.search(r'【等级(\d+)】', line)
            max_income["level"] = level_match.group(1) if level_match else 'Max'

    return {
        "medal_name": lines[0],
        "max_income": max_income,
    }


def calculate_total_cost(content: str) -> Dict:
    """
    Calculate total cost to upgrade a medal.
    
    Args:
        content: Medal description content
        
    Returns:
        Dictionary containing cost amount and cost description
    """
    cost_statistics = {
        "gold": 0,
        "blood": 0,
        "journey": 0,
        "curse": 0,
        "knowledge": 0,
        "soul": 0,
        "corruption": 0,
    }

    # Extract upgrade conditions
    upgrade_conditions = re.findall(
        r'消耗([-\d]+)\s*(金币|血液|旅程|咒术|知识|灵魂|堕落)',
        content
    )

    if upgrade_conditions:
        for cost_value_str, resource_type in upgrade_conditions:
            cost_value = int(cost_value_str)
            resource_en = ATTRIBUTE_CN_TO_EN.get(resource_type, resource_type)
            cost_statistics[resource_en] += cost_value


    # Calculate upgrade cost with weights
    upgrade_cost = sum(
        cost_value * INCOME_WEIGHT_MAP.get(resource_type_en, 0)
        for resource_type_en, cost_value in cost_statistics.items()
    )

    # Extract shop price
    price_match = re.search(
        r'商店售价】(\d+)(金币|血液|旅程|咒术|知识|灵魂|堕落)?',
        content
    )
    price = int(price_match.group(1)) if price_match else 0
    price_unit = price_match.group(2) if price_match and price_match.group(2) else None
    price_unit_en = ATTRIBUTE_CN_TO_EN.get(price_unit, price_unit) if price_unit else None
    price = price * INCOME_WEIGHT_MAP.get(price_unit_en, 0) if price_unit_en else 0

    total_cost = upgrade_cost + price

    return total_cost


def main():
    """
    Main function to calculate and rank medals by cost-effectiveness and income.
    """
    results = []

    for medal_name, content in MEDAL_MAPPINGS.items():
        
        # 判断过滤勋章
        bContinue = False
        for fm in FILTER_MEDALS:
            if fm in medal_name:
                bContinue = True
                break
        if bContinue:
            continue

        income_result = calculate_income(content)
        total_cost = calculate_total_cost(content)

        max_income = income_result["max_income"]["total_income"]
        cost_effectiveness = max_income / total_cost if total_cost != 0 else 0

        results.append({
            "name": medal_name,
            "income": max_income,
            "cost_effectiveness": cost_effectiveness,
        })

    lines = []
    # Sort by cost-effectiveness and display top TOP_N
    print(f"性价比 Top {TOP_N}:")
    print("-" * 80)
    results.sort(key=lambda x: x["cost_effectiveness"], reverse=True)
    for index, item in enumerate(results[:TOP_N], start=1):
        lines.append(f"""性价比排名: {index}
        勋章: {item['name']:<30}
        性价比: {item['cost_effectiveness']:.4f}
        博物馆: https://www.gamemale.com/thread-{MEDAL_LIBRARY.get(item['name'], 'unknown')}-1-1.html"""
        )

    # Sort by income and display top TOP_INCOME_N
    print(f"\n\n收益 Top {TOP_INCOME_N}:")
    print("-" * 80)
    results.sort(key=lambda x: x["income"], reverse=True)
    for index, item in enumerate(results[:TOP_INCOME_N], start=1):
        lines.append(f"""收益排名: {index}
        勋章: {item['name']:<30} 
        收益: {item['income']:.4f}
        博物馆: https://www.gamemale.com/thread-{MEDAL_LIBRARY.get(item['name'], 'unknown')}-1-1.html"""
        )

    with open('output', 'w') as f:
        f.writelines('\n'.join(lines))


if __name__ == "__main__":
    main()





性价比 Top 1000:
--------------------------------------------------------------------------------


收益 Top 1000:
--------------------------------------------------------------------------------


: 